Wikipedia analysis using Latent Dirichlet Allocation (LDA)
==========================================================

Purpose
-------

Results
-------

References
----------

-   LDA
-   Koden
-   Annat?

Setup
=====

Init
----

Always run this when attaching to cluster

In [ ]:
val group = "05_lda"  // just a directory name to separate file structure
val localpath=s"file:/databricks/driver/$group/"
val dir = s"dbfs:/datasets/$group/"


  

>     group: String = 05_lda
>     localpath: String = file:/databricks/driver/05_lda/
>     dir: String = dbfs:/datasets/05_lda/

In [ ]:
cd /databricks/driver/projects
pwd
ls

  

>     /databricks/driver/projects
>     group12

In [ ]:
export group=05_lda
mkdir $group
cd $group
pwd

  

>     /databricks/driver/05_lda

  

Config
======

Update and run the config for the language you want

In [ ]:
val lang = "sv"
val freeze_date = "20201208"
val xml_file = lang + "wiki-" + freeze_date + "-pages-articles.xml"

  

>     lang: String = sv
>     freeze_date: String = 20201208
>     xml_file: String = svwiki-20201208-pages-articles.xml

  

Download
========

Download Stopwords
------------------

In [ ]:
wget https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv

  

>     --2020-12-04 09:42:29--  https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv
>     Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
>     Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1936 (1.9K) [text/plain]
>     Saving to: ‘stoppord.csv’
>
>          0K .                                                     100% 17.9M=0s
>
>     2020-12-04 09:42:29 (17.9 MB/s) - ‘stoppord.csv’ saved [1936/1936]

  

Move to DBFS

In [ ]:
val file ="file:/databricks/driver/stoppord.csv"
val dir = "dbfs:/datasets/05_lda/"

dbutils.fs.mkdirs(dir)

dbutils.fs.cp(localpath, dir)


  

>     localpath: String = file:/databricks/driver/stoppord.csv
>     dir: String = dbfs:/datasets/05_lda/
>     res26: Boolean = true

In [ ]:
val stopword_path = "dbfs:/datasets/05_lda/stoppord.csv"
display(dbutils.fs.ls(stopword_path))

  

[TABLE]

  

Download the Swedish Wikipedia dump
-----------------------------------

In [ ]:
wget https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2

  

>     --2020-11-30 09:16:33--  https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2
>     Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.155.106, 2620:0:861:4:208:80:155:106
>     Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.155.106|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1525903265 (1.4G) [application/octet-stream]
>     Saving to: ‘svwiki-20201120-pages-articles.xml.bz2’
>
>          0K .......... .......... .......... .......... ..........  0%  636K 39m2s
>         50K .......... .......... .......... .......... ..........  0%  636K 39m1s
>        100K .......... .......... .......... .......... ..........  0% 1.45M 31m36s
>        150K .......... .......... .......... .......... ..........  0%  116M 23m45s
>        200K .......... .......... .......... .......... ..........  0% 1.09M 23m27s
>        250K .......... .......... .......... .......... ..........  0% 92.2M 19m35s
>        300K .......... .......... .......... .......... ..........  0% 1.45M 19m11s
>        350K .......... .......... .......... .......... ..........  0%  131M 16m48s
>        400K .......... .......... .......... .......... ..........  0%  160M 14m57s
>        450K .......... .......... .......... .......... ..........  0% 1.12M 15m37s
>        500K .......... .......... .......... .......... ..........  0%  139M 14m13s
>        550K .......... .......... .......... .......... ..........  0%  129M 13m3s
>        600K .......... .......... .......... .......... ..........  0%  231M 12m3s
>        650K .......... .......... .......... .......... ..........  0% 1.49M 12m21s
>        700K .......... .......... .......... .......... ..........  0%  153M 11m32s
>        750K .......... .......... .......... .......... ..........  0%  128M 10m50s
>        800K .......... .......... .......... .......... ..........  0%  148M 10m12s
>        850K .......... .......... .......... .......... ..........  0%  155M 9m39s
>        900K .......... .......... .......... .......... ..........  0%  155M 9m9s
>        950K .......... .......... .......... .......... ..........  0% 1.13M 9m46s
>       1000K .......... .......... .......... .......... ..........  0%  125M 9m18s
>       1050K .......... .......... .......... .......... ..........  0%  168M 8m53s
>       1100K .......... .......... .......... .......... ..........  0%  202M 8m31s
>       1150K .......... .......... .......... .......... ..........  0%  178M 8m10s
>       1200K .......... .......... .......... .......... ..........  0%  186M 7m50s
>       1250K .......... .......... .......... .......... ..........  0%  172M 7m33s
>       1300K .......... .......... .......... .......... ..........  0% 18.3M 7m19s
>       1350K .......... .......... .......... .......... ..........  0% 1.66M 7m34s
>       1400K .......... .......... .......... .......... ..........  0%  131M 7m19s
>       1450K .......... .......... .......... .......... ..........  0%  166M 7m5s
>       1500K .......... .......... .......... .......... ..........  0%  147M 6m51s
>       1550K .......... .......... .......... .......... ..........  0%  125M 6m39s
>       1600K .......... .......... .......... .......... ..........  0%  154M 6m27s
>       1650K .......... .......... .......... .......... ..........  0% 3.33M 6m28s
>       1700K .......... .......... .......... .......... ..........  0% 3.60M 6m29s
>       1750K .......... .......... .......... .......... ..........  0% 2.83M 6m32s
>       1800K .......... .......... .......... .......... ..........  0%  150M 6m22s
>       1850K .......... .......... .......... .......... ..........  0% 3.19M 6m24s
>       1900K .......... .......... .......... .......... ..........  0% 3.47M 6m25s
>       1950K .......... .......... .......... .......... ..........  0% 5.15M 6m22s
>       2000K .......... .......... .......... .......... ..........  0% 3.91M 6m22s
>       2050K .......... .......... .......... .......... ..........  0%  111M 6m13s
>       2100K .......... .......... .......... .......... ..........  0% 3.09M 6m15s
>       2150K .......... .......... .......... .......... ..........  0% 4.13M 6m15s
>       2200K .......... .......... .......... .......... ..........  0% 4.42M 6m14s
>       2250K .......... .......... .......... .......... ..........  0%  158M 6m6s
>       2300K .......... .......... .......... .......... ..........  0% 3.96M 6m6s
>       2350K .......... .......... .......... .......... ..........  0% 2.94M 6m9s
>       2400K .......... .......... .......... .......... ..........  0% 3.54M 6m9s
>       2450K .......... .......... .......... .......... ..........  0% 5.16M 6m8s
>       2500K .......... .......... .......... .......... ..........  0%  129M 6m1s
>       2550K .......... .......... .......... .......... ..........  0% 4.13M 6m0s
>       2600K .......... .......... .......... .......... ..........  0% 3.04M 6m3s
>       2650K .......... .......... .......... .......... ..........  0% 2.89M 6m5s
>       2700K .......... .......... .......... .......... ..........  0%  148M 5m59s
>       2750K .......... .......... .......... .......... ..........  0% 2.94M 6m1s
>       2800K .......... .......... .......... .......... ..........  0% 2.81M 6m4s
>       2850K .......... .......... .......... .......... ..........  0% 3.18M 6m5s
>       2900K .......... .......... .......... .......... ..........  0% 2.94M 6m8s
>       2950K .......... .......... .......... .......... ..........  0%  169M 6m2s
>       3000K .......... .......... .......... .......... ..........  0% 3.03M 6m4s
>       3050K .......... .......... .......... .......... ..........  0% 3.30M 6m5s
>       3100K .......... .......... .......... .......... ..........  0% 4.11M 6m5s
>       3150K .......... .......... .......... .......... ..........  0% 3.04M 6m6s
>       3200K .......... .......... .......... .......... ..........  0%  149M 6m1s
>       3250K .......... .......... .......... .......... ..........  0% 3.15M 6m2s
>       3300K .......... .......... .......... .......... ..........  0% 3.06M 6m4s
>       3350K .......... .......... .......... .......... ..........  0% 3.13M 6m5s
>       3400K .......... .......... .......... .......... ..........  0%  125M 6m0s
>       3450K .......... .......... .......... .......... ..........  0% 3.11M 6m2s
>       3500K .......... .......... .......... .......... ..........  0% 3.92M 6m2s
>       3550K .......... .......... .......... .......... ..........  0% 3.18M 6m3s
>       3600K .......... .......... .......... .......... ..........  0% 3.19M 6m5s
>       3650K .......... .......... .......... .......... ..........  0%  181M 6m0s
>       3700K .......... .......... .......... .......... ..........  0% 3.13M 6m1s
>       3750K .......... .......... .......... .......... ..........  0% 3.38M 6m2s
>       3800K .......... .......... .......... .......... ..........  0% 3.58M 6m2s
>       3850K .......... .......... .......... .......... ..........  0%  155M 5m58s
>       3900K .......... .......... .......... .......... ..........  0% 3.21M 5m59s
>       3950K .......... .......... .......... .......... ..........  0% 3.62M 6m0s
>       4000K .......... .......... .......... .......... ..........  0% 4.40M 5m59s
>       4050K .......... .......... .......... .......... ..........  0% 4.02M 5m59s
>       4100K .......... .......... .......... .......... ..........  0%  146M 5m55s
>       4150K .......... .......... .......... .......... ..........  0% 4.20M 5m55s
>       4200K .......... .......... .......... .......... ..........  0% 3.84M 5m55s
>       4250K .......... .......... .......... .......... ..........  0% 4.71M 5m55s
>       4300K .......... .......... .......... .......... ..........  0%  144M 5m51s
>       4350K .......... .......... .......... .......... ..........  0% 3.87M 5m51s
>       4400K .......... .......... .......... .......... ..........  0% 3.22M 5m52s
>       4450K .......... .......... .......... .......... ..........  0% 3.02M 5m53s
>       4500K .......... .......... .......... .......... ..........  0% 3.98M 5m54s
>       4550K .......... .......... .......... .......... ..........  0%  122M 5m50s
>       4600K .......... .......... .......... .......... ..........  0% 2.95M 5m51s
>       4650K .......... .......... .......... .......... ..........  0% 3.25M 5m52s
>       4700K .......... .......... .......... .......... ..........  0% 3.10M 5m54s
>       4750K .......... .......... .......... .......... ..........  0% 1.94M 5m58s
>       4800K .......... .......... .......... .......... ..........  0%  139M 5m54s
>       4850K .......... .......... .......... .......... ..........  0% 4.61M 5m54s
>       4900K .......... .......... .......... .......... ..........  0% 4.13M 5m54s
>       4950K .......... .......... .......... .......... ..........  0% 3.11M 5m55s
>       5000K .......... .......... .......... .......... ..........  0%  180M 5m51s
>       5050K .......... .......... .......... .......... ..........  0% 4.27M 5m51s
>       5100K .......... .......... .......... .......... ..........  0% 4.26M 5m51s
>       5150K .......... .......... .......... .......... ..........  0% 3.04M 5m52s
>       5200K .......... .......... .......... .......... ..........  0% 6.14M 5m51s
>       5250K .......... .......... .......... .......... ..........  0%  126M 5m48s
>       5300K .......... .......... .......... .......... ..........  0% 3.15M 5m49s
>       5350K .......... .......... .......... .......... ..........  0% 4.92M 5m48s
>       5400K .......... .......... .......... .......... ..........  0% 4.31M 5m48s
>       5450K .......... .......... .......... .......... ..........  0%  124M 5m45s
>       5500K .......... .......... .......... .......... ..........  0% 4.16M 5m45s
>       5550K .......... .......... .......... .......... ..........  0% 3.92M 5m45s
>       5600K .......... .......... .......... .......... ..........  0% 3.63M 5m46s
>       5650K .......... .......... .......... .......... ..........  0% 3.73M 5m46s
>       5700K .......... .......... .......... .......... ..........  0%  159M 5m43s
>       5750K .......... .......... .......... .......... ..........  0% 3.83M 5m44s
>       5800K .......... .......... .......... .......... ..........  0% 3.16M 5m45s
>       5850K .......... .......... .......... .......... ..........  0% 3.65M 5m45s
>       5900K .......... .......... .......... .......... ..........  0%  140M 5m42s
>       5950K .......... .......... .......... .......... ..........  0% 3.06M 5m43s
>       6000K .......... .......... .......... .......... ..........  0% 4.60M 5m43s
>       6050K .......... .......... .......... .......... ..........  0% 4.08M 5m43s
>       6100K .......... .......... .......... .......... ..........  0% 4.14M 5m43s
>       6150K .......... .......... .......... .......... ..........  0%  123M 5m41s
>       6200K .......... .......... .......... .......... ..........  0% 4.11M 5m41s
>       6250K .......... .......... .......... .......... ..........  0% 3.34M 5m41s
>       6300K .......... .......... .......... .......... ..........  0% 5.49M 5m41s
>       6350K .......... .......... .......... .......... ..........  0% 3.28M 5m41s
>       6400K .......... .......... .......... .......... ..........  0%  154M 5m39s
>       6450K .......... .......... .......... .......... ..........  0% 3.41M 5m40s
>       6500K .......... .......... .......... .......... ..........  0% 4.11M 5m40s
>       6550K .......... .......... .......... .......... ..........  0% 3.34M 5m40s
>       6600K .......... .......... .......... .......... ..........  0%  161M 5m38s
>       6650K .......... .......... .......... .......... ..........  0% 1.65M 5m42s
>       6700K .......... .......... .......... .......... ..........  0% 4.39M 5m42s
>       6750K .......... .......... .......... .......... ..........  0% 3.26M 5m43s
>       6800K .......... .......... .......... .......... ..........  0% 3.33M 5m43s
>       6850K .......... .......... .......... .......... ..........  0%  144M 5m41s
>       6900K .......... .......... .......... .......... ..........  0% 3.34M 5m41s
>       6950K .......... .......... .......... .......... ..........  0% 3.35M 5m42s
>       7000K .......... .......... .......... .......... ..........  0% 3.30M 5m43s
>       7050K .......... .......... .......... .......... ..........  0%  182M 5m40s
>       7100K .......... .......... .......... .......... ..........  0% 3.35M 5m41s
>       7150K .......... .......... .......... .......... ..........  0% 4.28M 5m41s
>       7200K .......... .......... .......... .......... ..........  0% 5.78M 5m40s
>       7250K .......... .......... .......... .......... ..........  0% 3.36M 5m41s
>       7300K .......... .......... .......... .......... ..........  0%  154M 5m39s
>       7350K .......... .......... .......... .......... ..........  0% 3.39M 5m39s
>       7400K .......... .......... .......... .......... ..........  0% 3.37M 5m40s
>       7450K .......... .......... .......... .......... ..........  0% 4.40M 5m40s
>       7500K .......... .......... .......... .......... ..........  0%  156M 5m38s
>       7550K .......... .......... .......... .......... ..........  0% 3.44M 5m38s
>       7600K .......... .......... .......... .......... ..........  0% 3.45M 5m39s
>       7650K .......... .......... .......... .......... ..........  0% 3.49M 5m39s
>       7700K .......... .......... .......... .......... ..........  0% 3.49M 5m40s
>       7750K .......... .......... .......... .......... ..........  0%  114M 5m37s
>       7800K .......... .......... .......... .......... ..........  0% 3.42M 5m38s
>       7850K .......... .......... .......... .......... ..........  0%  712K 5m49s
>       7900K .......... .......... .......... .......... ..........  0% 2.18M 5m51s
>       7950K .......... .......... .......... .......... ..........  0% 16.9M 5m49s
>       8000K .......... .......... .......... .......... ..........  0%  167M 5m47s
>       8050K .......... .......... .......... .......... ..........  0% 3.22M 5m48s
>       8100K .......... .......... .......... .......... ..........  0% 3.35M 5m48s
>       8150K .......... .......... .......... .......... ..........  0% 3.64M 5m49s
>       8200K .......... .......... .......... .......... ..........  0%  123M 5m47s
>       8250K .......... .......... .......... .......... ..........  0% 4.12M 5m47s
>       8300K .......... .......... .......... .......... ..........  0% 4.85M 5m46s
>       8350K .......... .......... .......... .......... ..........  0% 3.37M 5m47s
>       8400K .......... .......... .......... .......... ..........  0% 4.76M 5m47s
>       8450K .......... .......... .......... .......... ..........  0%  133M 5m45s
>       8500K .......... .......... .......... .......... ..........  0% 3.65M 5m45s
>       8550K .......... .......... .......... .......... ..........  0% 4.26M 5m45s
>       8600K .......... .......... .......... .......... ..........  0% 3.45M 5m45s
>       8650K .......... .......... .......... .......... ..........  0%  167M 5m43s
>       8700K .......... .......... .......... .......... ..........  0% 3.47M 5m44s
>       8750K .......... .......... .......... .......... ..........  0% 3.48M 5m44s
>       8800K .......... .......... .......... .......... ..........  0% 3.45M 5m44s
>       8850K .......... .......... .......... .......... ..........  0% 3.44M 5m45s
>       8900K .......... .......... .......... .......... ..........  0%  122M 5m43s
>       8950K .......... .......... .......... .......... ..........  0% 4.21M 5m43s
>       9000K .......... .......... .......... .......... ..........  0% 4.50M 5m43s
>       9050K .......... .......... .......... .......... ..........  0% 3.32M 5m43s
>       9100K .......... .......... .......... .......... ..........  0%  141M 5m42s
>       9150K .......... .......... .......... .......... ..........  0% 5.64M 5m41s
>       9200K .......... .......... .......... .......... ..........  0% 3.26M 5m42s
>       9250K .......... .......... .......... .......... ..........  0% 4.49M 5m41s
>       9300K .......... .......... .......... .......... ..........  0% 3.30M 5m42s
>       9350K .......... .......... .......... .......... ..........  0% 87.0M 5m40s
>       9400K .......... .......... .......... .......... ..........  0% 3.37M 5m41s
>       9450K .......... .......... .......... .......... ..........  0% 4.02M 5m41s
>       9500K .......... .......... .......... .......... ..........  0% 3.26M 5m41s
>       9550K .......... .......... .......... .......... ..........  0% 4.00M 5m41s
>       9600K .......... .......... .......... .......... ..........  0%  154M 5m40s
>       9650K .......... .......... .......... .......... ..........  0% 3.80M 5m40s
>       9700K .......... .......... .......... .......... ..........  0% 3.99M 5m40s
>       9750K .......... .......... .......... .......... ..........  0% 3.18M 5m41s
>       9800K .......... .......... .......... .......... ..........  0%  234M 5m39s
>       9850K .......... .......... .......... .......... ..........  0% 3.06M 5m40s
>       9900K .......... .......... .......... .......... ..........  0% 3.33M 5m40s
>       9950K .......... .......... .......... .......... ..........  0% 3.95M 5m40s
>      10000K .......... .......... .......... .......... ..........  0% 3.14M 5m41s
>      10050K .......... .......... .......... .......... ..........  0%  175M 5m39s
>      10100K .......... .......... .......... .......... ..........  0% 3.24M 5m40s
>      10150K .......... .......... .......... .......... ..........  0% 3.06M 5m40s
>      10200K .......... .......... .......... .......... ..........  0% 3.21M 5m41s
>      10250K .......... .......... .......... .......... ..........  0% 99.8M 5m39s
>      10300K .......... .......... .......... .......... ..........  0% 3.23M 5m40s
>      10350K .......... .......... .......... .......... ..........  0% 3.26M 5m40s
>      10400K .......... .......... .......... .......... ..........  0% 3.14M 5m41s
>      10450K .......... .......... .......... .......... ..........  0% 3.08M 5m41s
>      10500K .......... .......... .......... .......... ..........  0%  146M 5m40s
>      10550K .......... .......... .......... .......... ..........  0% 3.11M 5m40s
>      10600K .......... .......... .......... .......... ..........  0% 3.21M 5m41s
>      10650K .......... .......... .......... .......... ..........  0% 4.03M 5m41s
>      10700K .......... .......... .......... .......... ..........  0%  125M 5m39s
>      10750K .......... .......... .......... .......... ..........  0% 4.31M 5m39s
>      10800K .......... .......... .......... .......... ..........  0% 3.16M 5m40s
>      10850K .......... .......... .......... .......... ..........  0% 3.21M 5m40s
>      10900K .......... .......... .......... .......... ..........  0% 3.19M 5m41s
>      10950K .......... .......... .......... .......... ..........  0%  136M 5m39s
>      11000K .......... .......... .......... .......... ..........  0% 3.18M 5m40s
>      11050K .......... .......... .......... .......... ..........  0% 3.19M 5m40s
>      11100K .......... .......... .......... .......... ..........  0% 5.20M 5m40s
>      11150K .......... .......... .......... .......... ..........  0% 4.31M 5m40s
>      11200K .......... .......... .......... .......... ..........  0%  131M 5m38s
>      11250K .......... .......... .......... .......... ..........  0% 3.20M 5m39s
>      11300K .......... .......... .......... .......... ..........  0% 3.07M 5m40s
>      11350K .......... .......... .......... .......... ..........  0% 3.16M 5m40s
>      11400K .......... .......... .......... .......... ..........  0%  136M 5m39s
>      11450K .......... .......... .......... .......... ..........  0% 4.18M 5m39s
>      11500K .......... .......... .......... .......... ..........  0% 3.91M 5m39s
>      11550K .......... .......... .......... .......... ..........  0% 4.12M 5m39s
>      11600K .......... .......... .......... .......... ..........  0% 4.41M 5m39s
>      11650K .......... .......... .......... .......... ..........  0%  135M 5m37s
>      11700K .......... .......... .......... .......... ..........  0% 3.17M 5m38s
>      11750K .......... .......... .......... .......... ..........  0% 3.20M 5m38s
>      11800K .......... .......... .......... .......... ..........  0% 3.24M 5m39s
>      11850K .......... .......... .......... .......... ..........  0%  197M 5m37s
>      11900K .......... .......... .......... .......... ..........  0% 3.26M 5m38s
>      11950K .......... .......... .......... .......... ..........  0% 4.31M 5m38s
>      12000K .......... .......... .......... .......... ..........  0% 4.12M 5m38s
>      12050K .......... .......... .......... .......... ..........  0% 4.00M 5m38s
>      12100K .......... .......... .......... .......... ..........  0%  149M 5m36s
>      12150K .......... .......... .......... .......... ..........  0% 3.35M 5m37s
>      12200K .......... .......... .......... .......... ..........  0% 4.87M 5m37s
>      12250K .......... .......... .......... .......... ..........  0% 3.28M 5m37s
>      12300K .......... .......... .......... .......... ..........  0%  196M 5m36s
>      12350K .......... .......... .......... .......... ..........  0% 3.22M 5m36s
>      12400K .......... .......... .......... .......... ..........  0% 3.59M 5m36s
>      12450K .......... .......... .......... .......... ..........  0% 3.13M 5m37s
>      12500K .......... .......... .......... .......... ..........  0% 4.09M 5m37s
>      12550K .......... .......... .......... .......... ..........  0%  114M 5m36s
>      12600K .......... .......... .......... .......... ..........  0% 4.58M 5m36s
>      12650K .......... .......... .......... .......... ..........  0% 4.27M 5m36s
>      12700K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      12750K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      12800K .......... .......... .......... .......... ..........  0%  180M 5m35s
>      12850K .......... .......... .......... .......... ..........  0% 3.14M 5m36s
>      12900K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      12950K .......... .......... .......... .......... ..........  0% 3.34M 5m36s
>      13000K .......... .......... .......... .......... ..........  0%  161M 5m35s
>      13050K .......... .......... .......... .......... ..........  0% 3.95M 5m35s
>      13100K .......... .......... .......... .......... ..........  0% 4.23M 5m35s
>      13150K .......... .......... .......... .......... ..........  0% 3.19M 5m36s
>      13200K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      13250K .......... .......... .......... .......... ..........  0%  138M 5m35s
>      13300K .......... .......... .......... .......... ..........  0% 3.22M 5m35s
>      13350K .......... .......... .......... .......... ..........  0% 3.27M 5m36s
>      13400K .......... .......... .......... .......... ..........  0% 3.25M 5m36s
>      13450K .......... .......... .......... .......... ..........  0%  141M 5m35s
>      13500K .......... .......... .......... .......... ..........  0% 4.49M 5m35s
>      13550K .......... .......... .......... .......... ..........  0% 3.24M 5m35s
>      13600K .......... .......... .......... .......... ..........  0% 3.25M 5m36s
>      13650K .......... .......... .......... .......... ..........  0% 3.28M 5m36s
>      13700K .......... .......... .......... .......... ..........  0%  144M 5m35s
>      13750K .......... .......... .......... .......... ..........  0% 3.31M 5m35s
>      13800K .......... .......... .......... .......... ..........  0% 3.68M 5m35s
>      13850K .......... .......... .......... .......... ..........  0% 5.33M 5m35s
>      13900K .......... .......... .......... .......... ..........  0%  165M 5m34s
>      13950K .......... .......... .......... .......... ..........  0% 3.15M 5m34s
>      14000K .......... .......... .......... .......... ..........  0% 2.97M 5m35s
>      14050K .......... .......... .......... .......... ..........  0% 3.11M 5m35s
>      14100K .......... .......... .......... .......... ..........  0% 3.16M 5m36s
>      14150K .......... .......... .......... .......... ..........  0%  145M 5m35s
>      14200K .......... .......... .......... .......... ..........  0% 3.09M 5m35s
>      14250K .......... .......... .......... .......... ..........  0% 3.16M 5m35s
>      14300K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      14350K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      14400K .......... .......... .......... .......... ..........  0%  188M 5m35s
>      14450K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      14500K .......... .......... .......... .......... ..........  0% 5.12M 5m35s
>      14550K .......... .......... .......... .......... ..........  0% 3.17M 5m36s
>      14600K .......... .......... .......... .......... ..........  0%  138M 5m35s
>      14650K .......... .......... .......... .......... ..........  0% 3.22M 5m35s
>      14700K .......... .......... .......... .......... ..........  0% 3.24M 5m35s
>      14750K .......... .......... .......... .......... ..........  0% 3.28M 5m36s
>      14800K .......... .......... .......... .......... ..........  0% 3.26M 5m36s
>      14850K .......... .......... .......... .......... ..........  0%  128M 5m35s
>      14900K .......... .......... .......... .......... ..........  1% 4.01M 5m35s
>      14950K .......... .......... .......... .......... ..........  1% 3.15M 5m35s
>      15000K .......... .......... .......... .......... ..........  1% 4.21M 5m35s
>      15050K .......... .......... .......... .......... ..........  1%  189M 5m34s
>      15100K .......... .......... .......... .......... ..........  1% 4.26M 5m34s
>      15150K .......... .......... .......... .......... ..........  1% 3.17M 5m35s
>      15200K .......... .......... .......... .......... ..........  1% 3.20M 5m35s
>      15250K .......... .......... .......... .......... ..........  1% 3.68M 5m35s
>      15300K .......... .......... .......... .......... ..........  1%  185M 5m34s
>      15350K .......... .......... .......... .......... ..........  1% 4.88M 5m34s
>      15400K .......... .......... .......... .......... ..........  1% 4.48M 5m34s
>      15450K .......... .......... .......... .......... ..........  1% 3.80M 5m34s
>      15500K .......... .......... .......... .......... ..........  1%  165M 5m33s
>      15550K .......... .......... .......... .......... ..........  1% 4.16M 5m33s
>
>     *** WARNING: skipped 2292666 bytes of output ***
>
>     1474000K .......... .......... .......... .......... .......... 98% 4.32M 4s
>     1474050K .......... .......... .......... .......... .......... 98%  140M 4s
>     1474100K .......... .......... .......... .......... .......... 98% 4.11M 4s
>     1474150K .......... .......... .......... .......... .......... 98% 3.05M 4s
>     1474200K .......... .......... .......... .......... .......... 98% 3.47M 4s
>     1474250K .......... .......... .......... .......... .......... 98%  124M 4s
>     1474300K .......... .......... .......... .......... .......... 98% 3.21M 4s
>     1474350K .......... .......... .......... .......... .......... 98% 3.16M 4s
>     1474400K .......... .......... .......... .......... .......... 98% 3.18M 4s
>     1474450K .......... .......... .......... .......... .......... 98% 3.65M 3s
>     1474500K .......... .......... .......... .......... .......... 98%  127M 3s
>     1474550K .......... .......... .......... .......... .......... 98% 3.19M 3s
>     1474600K .......... .......... .......... .......... .......... 98% 3.24M 3s
>     1474650K .......... .......... .......... .......... .......... 98% 3.23M 3s
>     1474700K .......... .......... .......... .......... .......... 98%  157M 3s
>     1474750K .......... .......... .......... .......... .......... 98% 3.24M 3s
>     1474800K .......... .......... .......... .......... .......... 98% 3.26M 3s
>     1474850K .......... .......... .......... .......... .......... 98% 3.28M 3s
>     1474900K .......... .......... .......... .......... .......... 98% 3.28M 3s
>     1474950K .......... .......... .......... .......... .......... 98%  126M 3s
>     1475000K .......... .......... .......... .......... .......... 98% 3.54M 3s
>     1475050K .......... .......... .......... .......... .......... 98% 3.57M 3s
>     1475100K .......... .......... .......... .......... .......... 98% 4.01M 3s
>     1475150K .......... .......... .......... .......... .......... 98% 2.86M 3s
>     1475200K .......... .......... .......... .......... .......... 99%  142M 3s
>     1475250K .......... .......... .......... .......... .......... 99% 3.38M 3s
>     1475300K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1475350K .......... .......... .......... .......... .......... 99% 3.74M 3s
>     1475400K .......... .......... .......... .......... .......... 99%  108M 3s
>     1475450K .......... .......... .......... .......... .......... 99% 4.64M 3s
>     1475500K .......... .......... .......... .......... .......... 99% 3.24M 3s
>     1475550K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1475600K .......... .......... .......... .......... .......... 99% 3.06M 3s
>     1475650K .......... .......... .......... .......... .......... 99%  134M 3s
>     1475700K .......... .......... .......... .......... .......... 99% 4.15M 3s
>     1475750K .......... .......... .......... .......... .......... 99% 3.80M 3s
>     1475800K .......... .......... .......... .......... .......... 99% 4.16M 3s
>     1475850K .......... .......... .......... .......... .......... 99%  154M 3s
>     1475900K .......... .......... .......... .......... .......... 99% 3.88M 3s
>     1475950K .......... .......... .......... .......... .......... 99% 4.28M 3s
>     1476000K .......... .......... .......... .......... .......... 99% 3.10M 3s
>     1476050K .......... .......... .......... .......... .......... 99% 4.47M 3s
>     1476100K .......... .......... .......... .......... .......... 99%  165M 3s
>     1476150K .......... .......... .......... .......... .......... 99% 3.14M 3s
>     1476200K .......... .......... .......... .......... .......... 99% 3.62M 3s
>     1476250K .......... .......... .......... .......... .......... 99% 3.84M 3s
>     1476300K .......... .......... .......... .......... .......... 99%  209M 3s
>     1476350K .......... .......... .......... .......... .......... 99% 4.15M 3s
>     1476400K .......... .......... .......... .......... .......... 99% 3.13M 3s
>     1476450K .......... .......... .......... .......... .......... 99% 4.19M 3s
>     1476500K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1476550K .......... .......... .......... .......... .......... 99%  114M 3s
>     1476600K .......... .......... .......... .......... .......... 99% 2.70M 3s
>     1476650K .......... .......... .......... .......... .......... 99% 2.84M 3s
>     1476700K .......... .......... .......... .......... .......... 99% 3.62M 3s
>     1476750K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476800K .......... .......... .......... .......... .......... 99%  123M 3s
>     1476850K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476900K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476950K .......... .......... .......... .......... .......... 99% 4.13M 3s
>     1477000K .......... .......... .......... .......... .......... 99%  144M 3s
>     1477050K .......... .......... .......... .......... .......... 99% 4.27M 3s
>     1477100K .......... .......... .......... .......... .......... 99% 3.65M 3s
>     1477150K .......... .......... .......... .......... .......... 99% 3.99M 3s
>     1477200K .......... .......... .......... .......... .......... 99% 2.76M 3s
>     1477250K .......... .......... .......... .......... .......... 99%  145M 3s
>     1477300K .......... .......... .......... .......... .......... 99% 4.00M 3s
>     1477350K .......... .......... .......... .......... .......... 99% 5.87M 3s
>     1477400K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477450K .......... .......... .......... .......... .......... 99%  194M 3s
>     1477500K .......... .......... .......... .......... .......... 99% 3.75M 3s
>     1477550K .......... .......... .......... .......... .......... 99% 4.40M 3s
>     1477600K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477650K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477700K .......... .......... .......... .......... .......... 99%  140M 3s
>     1477750K .......... .......... .......... .......... .......... 99% 4.33M 3s
>     1477800K .......... .......... .......... .......... .......... 99% 3.01M 3s
>     1477850K .......... .......... .......... .......... .......... 99% 4.95M 3s
>     1477900K .......... .......... .......... .......... .......... 99%  149M 3s
>     1477950K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1478000K .......... .......... .......... .......... .......... 99% 3.08M 3s
>     1478050K .......... .......... .......... .......... .......... 99% 3.11M 3s
>     1478100K .......... .......... .......... .......... .......... 99% 3.03M 3s
>     1478150K .......... .......... .......... .......... .......... 99%  123M 3s
>     1478200K .......... .......... .......... .......... .......... 99% 3.01M 3s
>     1478250K .......... .......... .......... .......... .......... 99% 3.18M 3s
>     1478300K .......... .......... .......... .......... .......... 99% 2.93M 3s
>     1478350K .......... .......... .......... .......... .......... 99% 3.03M 3s
>     1478400K .......... .......... .......... .......... .......... 99%  144M 3s
>     1478450K .......... .......... .......... .......... .......... 99% 3.81M 3s
>     1478500K .......... .......... .......... .......... .......... 99% 3.10M 3s
>     1478550K .......... .......... .......... .......... .......... 99% 3.08M 3s
>     1478600K .......... .......... .......... .......... .......... 99%  132M 3s
>     1478650K .......... .......... .......... .......... .......... 99% 3.18M 3s
>     1478700K .......... .......... .......... .......... .......... 99% 3.14M 3s
>     1478750K .......... .......... .......... .......... .......... 99% 3.16M 3s
>     1478800K .......... .......... .......... .......... .......... 99% 3.06M 3s
>     1478850K .......... .......... .......... .......... .......... 99%  167M 3s
>     1478900K .......... .......... .......... .......... .......... 99% 4.10M 3s
>     1478950K .......... .......... .......... .......... .......... 99% 2.89M 2s
>     1479000K .......... .......... .......... .......... .......... 99% 2.96M 2s
>     1479050K .......... .......... .......... .......... .......... 99%  143M 2s
>     1479100K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1479150K .......... .......... .......... .......... .......... 99% 3.86M 2s
>     1479200K .......... .......... .......... .......... .......... 99% 4.52M 2s
>     1479250K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1479300K .......... .......... .......... .......... .......... 99%  143M 2s
>     1479350K .......... .......... .......... .......... .......... 99% 4.01M 2s
>     1479400K .......... .......... .......... .......... .......... 99% 3.20M 2s
>     1479450K .......... .......... .......... .......... .......... 99% 3.88M 2s
>     1479500K .......... .......... .......... .......... .......... 99%  208M 2s
>     1479550K .......... .......... .......... .......... .......... 99% 3.73M 2s
>     1479600K .......... .......... .......... .......... .......... 99% 3.11M 2s
>     1479650K .......... .......... .......... .......... .......... 99% 4.10M 2s
>     1479700K .......... .......... .......... .......... .......... 99% 3.15M 2s
>     1479750K .......... .......... .......... .......... .......... 99%  140M 2s
>     1479800K .......... .......... .......... .......... .......... 99% 3.02M 2s
>     1479850K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1479900K .......... .......... .......... .......... .......... 99% 3.01M 2s
>     1479950K .......... .......... .......... .......... .......... 99% 3.58M 2s
>     1480000K .......... .......... .......... .......... .......... 99%  148M 2s
>     1480050K .......... .......... .......... .......... .......... 99% 2.95M 2s
>     1480100K .......... .......... .......... .......... .......... 99% 3.23M 2s
>     1480150K .......... .......... .......... .......... .......... 99% 3.26M 2s
>     1480200K .......... .......... .......... .......... .......... 99%  216M 2s
>     1480250K .......... .......... .......... .......... .......... 99% 4.17M 2s
>     1480300K .......... .......... .......... .......... .......... 99% 4.15M 2s
>     1480350K .......... .......... .......... .......... .......... 99% 3.64M 2s
>     1480400K .......... .......... .......... .......... .......... 99% 2.49M 2s
>     1480450K .......... .......... .......... .......... .......... 99%  131M 2s
>     1480500K .......... .......... .......... .......... .......... 99% 3.71M 2s
>     1480550K .......... .......... .......... .......... .......... 99% 3.16M 2s
>     1480600K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1480650K .......... .......... .......... .......... .......... 99%  160M 2s
>     1480700K .......... .......... .......... .......... .......... 99% 3.54M 2s
>     1480750K .......... .......... .......... .......... .......... 99% 2.00M 2s
>     1480800K .......... .......... .......... .......... .......... 99% 3.49M 2s
>     1480850K .......... .......... .......... .......... .......... 99% 2.98M 2s
>     1480900K .......... .......... .......... .......... .......... 99%  137M 2s
>     1480950K .......... .......... .......... .......... .......... 99% 3.82M 2s
>     1481000K .......... .......... .......... .......... .......... 99% 4.41M 2s
>     1481050K .......... .......... .......... .......... .......... 99% 5.45M 2s
>     1481100K .......... .......... .......... .......... .......... 99%  189M 2s
>     1481150K .......... .......... .......... .......... .......... 99% 3.35M 2s
>     1481200K .......... .......... .......... .......... .......... 99% 4.40M 2s
>     1481250K .......... .......... .......... .......... .......... 99% 4.99M 2s
>     1481300K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1481350K .......... .......... .......... .......... .......... 99%  123M 2s
>     1481400K .......... .......... .......... .......... .......... 99% 4.13M 2s
>     1481450K .......... .......... .......... .......... .......... 99% 4.15M 2s
>     1481500K .......... .......... .......... .......... .......... 99% 4.17M 2s
>     1481550K .......... .......... .......... .......... .......... 99% 2.08M 2s
>     1481600K .......... .......... .......... .......... .......... 99%  146M 2s
>     1481650K .......... .......... .......... .......... .......... 99% 3.60M 2s
>     1481700K .......... .......... .......... .......... .......... 99% 3.55M 2s
>     1481750K .......... .......... .......... .......... .......... 99% 4.10M 2s
>     1481800K .......... .......... .......... .......... .......... 99%  135M 2s
>     1481850K .......... .......... .......... .......... .......... 99% 3.69M 2s
>     1481900K .......... .......... .......... .......... .......... 99% 3.60M 2s
>     1481950K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1482000K .......... .......... .......... .......... .......... 99% 3.03M 2s
>     1482050K .......... .......... .......... .......... .......... 99%  122M 2s
>     1482100K .......... .......... .......... .......... .......... 99% 2.85M 2s
>     1482150K .......... .......... .......... .......... .......... 99% 2.97M 2s
>     1482200K .......... .......... .......... .......... .......... 99% 2.93M 2s
>     1482250K .......... .......... .......... .......... .......... 99%  177M 2s
>     1482300K .......... .......... .......... .......... .......... 99% 4.90M 2s
>     1482350K .......... .......... .......... .......... .......... 99% 3.44M 2s
>     1482400K .......... .......... .......... .......... .......... 99% 3.91M 2s
>     1482450K .......... .......... .......... .......... .......... 99% 2.90M 2s
>     1482500K .......... .......... .......... .......... .......... 99%  139M 2s
>     1482550K .......... .......... .......... .......... .......... 99% 2.97M 2s
>     1482600K .......... .......... .......... .......... .......... 99% 3.02M 2s
>     1482650K .......... .......... .......... .......... .......... 99% 3.40M 2s
>     1482700K .......... .......... .......... .......... .......... 99%  147M 2s
>     1482750K .......... .......... .......... .......... .......... 99% 3.87M 2s
>     1482800K .......... .......... .......... .......... .......... 99% 5.92M 2s
>     1482850K .......... .......... .......... .......... .......... 99% 2.90M 2s
>     1482900K .......... .......... .......... .......... .......... 99% 3.98M 2s
>     1482950K .......... .......... .......... .......... .......... 99%  134M 2s
>     1483000K .......... .......... .......... .......... .......... 99% 4.82M 2s
>     1483050K .......... .......... .......... .......... .......... 99% 4.05M 2s
>     1483100K .......... .......... .......... .......... .......... 99% 3.63M 2s
>     1483150K .......... .......... .......... .......... .......... 99% 3.52M 2s
>     1483200K .......... .......... .......... .......... .......... 99%  154M 2s
>     1483250K .......... .......... .......... .......... .......... 99% 3.95M 2s
>     1483300K .......... .......... .......... .......... .......... 99% 3.99M 2s
>     1483350K .......... .......... .......... .......... .......... 99% 3.32M 2s
>     1483400K .......... .......... .......... .......... .......... 99%  147M 1s
>     1483450K .......... .......... .......... .......... .......... 99% 5.69M 1s
>     1483500K .......... .......... .......... .......... .......... 99% 3.62M 1s
>     1483550K .......... .......... .......... .......... .......... 99% 2.80M 1s
>     1483600K .......... .......... .......... .......... .......... 99% 4.37M 1s
>     1483650K .......... .......... .......... .......... .......... 99%  124M 1s
>     1483700K .......... .......... .......... .......... .......... 99% 3.03M 1s
>     1483750K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1483800K .......... .......... .......... .......... .......... 99% 3.96M 1s
>     1483850K .......... .......... .......... .......... .......... 99%  161M 1s
>     1483900K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1483950K .......... .......... .......... .......... .......... 99% 3.07M 1s
>     1484000K .......... .......... .......... .......... .......... 99% 3.14M 1s
>     1484050K .......... .......... .......... .......... .......... 99% 3.14M 1s
>     1484100K .......... .......... .......... .......... .......... 99%  119M 1s
>     1484150K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1484200K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1484250K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1484300K .......... .......... .......... .......... .......... 99%  154M 1s
>     1484350K .......... .......... .......... .......... .......... 99% 2.74M 1s
>     1484400K .......... .......... .......... .......... .......... 99% 3.04M 1s
>     1484450K .......... .......... .......... .......... .......... 99% 3.06M 1s
>     1484500K .......... .......... .......... .......... .......... 99% 2.99M 1s
>     1484550K .......... .......... .......... .......... .......... 99%  110M 1s
>     1484600K .......... .......... .......... .......... .......... 99% 2.99M 1s
>     1484650K .......... .......... .......... .......... .......... 99% 3.04M 1s
>     1484700K .......... .......... .......... .......... .......... 99% 4.33M 1s
>     1484750K .......... .......... .......... .......... .......... 99% 3.58M 1s
>     1484800K .......... .......... .......... .......... .......... 99%  142M 1s
>     1484850K .......... .......... .......... .......... .......... 99% 3.95M 1s
>     1484900K .......... .......... .......... .......... .......... 99% 4.10M 1s
>     1484950K .......... .......... .......... .......... .......... 99% 3.15M 1s
>     1485000K .......... .......... .......... .......... .......... 99%  128M 1s
>     1485050K .......... .......... .......... .......... .......... 99% 4.71M 1s
>     1485100K .......... .......... .......... .......... .......... 99% 3.68M 1s
>     1485150K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1485200K .......... .......... .......... .......... .......... 99% 2.82M 1s
>     1485250K .......... .......... .......... .......... .......... 99%  133M 1s
>     1485300K .......... .......... .......... .......... .......... 99% 2.92M 1s
>     1485350K .......... .......... .......... .......... .......... 99% 2.75M 1s
>     1485400K .......... .......... .......... .......... .......... 99% 2.80M 1s
>     1485450K .......... .......... .......... .......... .......... 99%  156M 1s
>     1485500K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1485550K .......... .......... .......... .......... .......... 99% 3.49M 1s
>     1485600K .......... .......... .......... .......... .......... 99% 2.66M 1s
>     1485650K .......... .......... .......... .......... .......... 99% 3.72M 1s
>     1485700K .......... .......... .......... .......... .......... 99%  142M 1s
>     1485750K .......... .......... .......... .......... .......... 99% 4.77M 1s
>     1485800K .......... .......... .......... .......... .......... 99% 3.85M 1s
>     1485850K .......... .......... .......... .......... .......... 99% 3.88M 1s
>     1485900K .......... .......... .......... .......... .......... 99%  150M 1s
>     1485950K .......... .......... .......... .......... .......... 99% 2.85M 1s
>     1486000K .......... .......... .......... .......... .......... 99% 3.84M 1s
>     1486050K .......... .......... .......... .......... .......... 99% 3.78M 1s
>     1486100K .......... .......... .......... .......... .......... 99% 3.81M 1s
>     1486150K .......... .......... .......... .......... .......... 99%  136M 1s
>     1486200K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1486250K .......... .......... .......... .......... .......... 99% 2.79M 1s
>     1486300K .......... .......... .......... .......... .......... 99% 3.68M 1s
>     1486350K .......... .......... .......... .......... .......... 99% 3.85M 1s
>     1486400K .......... .......... .......... .......... .......... 99%  119M 1s
>     1486450K .......... .......... .......... .......... .......... 99% 2.22M 1s
>     1486500K .......... .......... .......... .......... .......... 99% 3.81M 1s
>     1486550K .......... .......... .......... .......... .......... 99% 3.67M 1s
>     1486600K .......... .......... .......... .......... .......... 99%  133M 1s
>     1486650K .......... .......... .......... .......... .......... 99% 3.52M 1s
>     1486700K .......... .......... .......... .......... .......... 99% 3.78M 1s
>     1486750K .......... .......... .......... .......... .......... 99% 4.63M 1s
>     1486800K .......... .......... .......... .......... .......... 99% 3.46M 1s
>     1486850K .......... .......... .......... .......... .......... 99%  195M 1s
>     1486900K .......... .......... .......... .......... .......... 99% 2.67M 1s
>     1486950K .......... .......... .......... .......... .......... 99% 3.46M 1s
>     1487000K .......... .......... .......... .......... .......... 99% 2.87M 1s
>     1487050K .......... .......... .......... .......... .......... 99%  191M 1s
>     1487100K .......... .......... .......... .......... .......... 99% 1.45M 1s
>     1487150K .......... .......... .......... .......... .......... 99% 2.74M 1s
>     1487200K .......... .......... .......... .......... .......... 99% 2.76M 1s
>     1487250K .......... .......... .......... .......... .......... 99% 2.95M 1s
>     1487300K .......... .......... .......... .......... .......... 99%  194M 1s
>     1487350K .......... .......... .......... .......... .......... 99% 2.55M 1s
>     1487400K .......... .......... .......... .......... .......... 99% 2.77M 1s
>     1487450K .......... .......... .......... .......... .......... 99% 2.86M 1s
>     1487500K .......... .......... .......... .......... .......... 99%  220M 1s
>     1487550K .......... .......... .......... .......... .......... 99% 3.23M 1s
>     1487600K .......... .......... .......... .......... .......... 99% 2.77M 1s
>     1487650K .......... .......... .......... .......... .......... 99% 2.73M 1s
>     1487700K .......... .......... .......... .......... .......... 99% 2.87M 1s
>     1487750K .......... .......... .......... .......... .......... 99%  133M 1s
>     1487800K .......... .......... .......... .......... .......... 99% 2.82M 1s
>     1487850K .......... .......... .......... .......... .......... 99% 2.93M 1s
>     1487900K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1487950K .......... .......... .......... .......... .......... 99% 3.66M 0s
>     1488000K .......... .......... .......... .......... .......... 99%  144M 0s
>     1488050K .......... .......... .......... .......... .......... 99% 3.04M 0s
>     1488100K .......... .......... .......... .......... .......... 99% 3.02M 0s
>     1488150K .......... .......... .......... .......... .......... 99% 2.92M 0s
>     1488200K .......... .......... .......... .......... .......... 99%  174M 0s
>     1488250K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1488300K .......... .......... .......... .......... .......... 99% 3.82M 0s
>     1488350K .......... .......... .......... .......... .......... 99% 2.41M 0s
>     1488400K .......... .......... .......... .......... .......... 99% 3.82M 0s
>     1488450K .......... .......... .......... .......... .......... 99%  135M 0s
>     1488500K .......... .......... .......... .......... .......... 99% 3.01M 0s
>     1488550K .......... .......... .......... .......... .......... 99% 3.92M 0s
>     1488600K .......... .......... .......... .......... .......... 99% 3.94M 0s
>     1488650K .......... .......... .......... .......... .......... 99%  152M 0s
>     1488700K .......... .......... .......... .......... .......... 99% 4.03M 0s
>     1488750K .......... .......... .......... .......... .......... 99% 2.93M 0s
>     1488800K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1488850K .......... .......... .......... .......... .......... 99% 3.55M 0s
>     1488900K .......... .......... .......... .......... .......... 99%  138M 0s
>     1488950K .......... .......... .......... .......... .......... 99% 2.89M 0s
>     1489000K .......... .......... .......... .......... .......... 99% 4.48M 0s
>     1489050K .......... .......... .......... .......... .......... 99% 3.68M 0s
>     1489100K .......... .......... .......... .......... .......... 99%  174M 0s
>     1489150K .......... .......... .......... .......... .......... 99% 4.37M 0s
>     1489200K .......... .......... .......... .......... .......... 99% 3.65M 0s
>     1489250K .......... .......... .......... .......... .......... 99% 3.78M 0s
>     1489300K .......... .......... .......... .......... .......... 99% 3.01M 0s
>     1489350K .......... .......... .......... .......... .......... 99%  119M 0s
>     1489400K .......... .......... .......... .......... .......... 99% 2.86M 0s
>     1489450K .......... .......... .......... .......... .......... 99% 2.98M 0s
>     1489500K .......... .......... .......... .......... .......... 99% 3.00M 0s
>     1489550K .......... .......... .......... .......... .......... 99% 2.97M 0s
>     1489600K .......... .......... .......... .......... .......... 99%  151M 0s
>     1489650K .......... .......... .......... .......... .......... 99% 2.00M 0s
>     1489700K .......... .......... .......... .......... .......... 99% 4.04M 0s
>     1489750K .......... .......... .......... .......... .......... 99% 2.82M 0s
>     1489800K .......... .......... .......... .......... .......... 99%  161M 0s
>     1489850K .......... .......... .......... .......... .......... 99% 3.14M 0s
>     1489900K .......... .......... .......... .......... .......... 99% 3.00M 0s
>     1489950K .......... .......... .......... .......... .......... 99% 3.03M 0s
>     1490000K .......... .......... .......... .......... .......... 99% 2.93M 0s
>     1490050K .......... .......... .......... .......... .......... 99%  135M 0s
>     1490100K .......... .......... .......... .........            100% 2.35M=5m33s
>
>     2020-11-30 09:22:07 (4.36 MB/s) - ‘svwiki-20201120-pages-articles.xml.bz2’ saved [1525903265/1525903265]

  

Extract the .xml-file

In [ ]:
ls
bzip2 -dk svwiki-20201120-pages-articles.xml.bz2

  

>     conf
>     derby.log
>     eventlogs
>     ganglia
>     logs
>     svwiki-20201120-pages-articles.xml.bz2

  

Copy file to DBFS cluster

In [ ]:

val localpath="file:/databricks/driver/svwiki-20201120-pages-articles.xml"

dbutils.fs.mkdirs("dbfs:/datasets/")

dbutils.fs.cp(localpath, "dbfs:/datasets/")

display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

  

[TABLE]

  

Load data
=========

In [ ]:
// Verify Dataset is in cluster
display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

  

[TABLE]

  

We now have a wikipedia dump ready to be processed

This page has some resources that can be used for Wikipedia parsing:
https://mindfulmachines.io/blog/2018/3/18/wikipedia-data-in-spark-and-scala-updated

In [ ]:
//---------------spark_import
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SQLContext

//----------------xml_loader_import
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{ LongWritable, Text }
import com.databricks.spark.xml._

import org.apache.hadoop.conf.Configuration

  

>     import org.apache.spark.SparkContext
>     import org.apache.spark.SparkConf
>     import org.apache.spark.sql.SQLContext
>     import org.apache.hadoop.io.LongWritable
>     import org.apache.hadoop.io.Text
>     import org.apache.hadoop.conf.Configuration
>     import org.apache.hadoop.io.{LongWritable, Text}
>     import com.databricks.spark.xml._
>     import org.apache.hadoop.conf.Configuration

In [ ]:

def readWikiDump(sc: SparkContext, file: String) : RDD[(Long, String)] = {
  val conf = new Configuration()
  conf.set(XmlInputFormat.START_TAG_KEY, "<page>")
  conf.set(XmlInputFormat.END_TAG_KEY, "</page>")
  val rdd = sc.newAPIHadoopFile(file, 
                                classOf[XmlInputFormat], 
                                classOf[LongWritable], 
                                classOf[Text], 
                                conf)
  rdd.map{case (k,v) => (k.get(), new String(v.copyBytes()))}
}

  

>     readWikiDump: (sc: org.apache.spark.SparkContext, file: String)org.apache.spark.rdd.RDD[(Long, String)]

In [ ]:
// Read the wiki dump

val dump = readWikiDump(sc, "dbfs:/datasets/svwiki-20201120-pages-articles.xml")

//dump.collect() This throws a memory error

dump.take(5)

  

>     dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[10] at map at command-738986526041623:11
>     res0: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>), (52187,<page>
>         <title>Afrika</title>
>         <ns>0</ns>
>         <id>2</id>
>         <revision>
>           <id>48459142</id>
>           <parentid>48374249</parentid>
>           <timestamp>2020-11-18T06:30:03Z</timestamp>
>           <contributor>
>             <username>LarskeBot</username>
>             <id>310602</id>
>           </contributor>
>           <minor />
>           <comment>/* Geografi */mellan X - Y → mellan X och Y med [[Project:AWB|AWB]]</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="37806" xml:space="preserve">{{templateStyles-tagg Faktamall}}
>     {| class=&quot;infobox&quot; style=&quot;font-size:90%;&quot; width=&quot;300&quot;
>     |+ &lt;big&gt;'''Afrika'''&lt;/big&gt;
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Africa (orthographic projection).svg|280px|Afrika i världen.]]
>     ----
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Afrika politisk.png|280px|Politisk karta.]]
>     ----
>     |-
>     | colspan=&quot;2&quot; |
>     |-
>     | '''Area:''' &lt;br /&gt; '''Folkmängd:''' &lt;br /&gt; '''Antal stater:''' &lt;br /&gt; '''Autonoma regioner:''' &lt;br /&gt; '''Högsta punkt:''' &lt;br /&gt; '''Lägsta punkt:''' || 30&amp;nbsp;221&amp;nbsp;532&amp;nbsp;km² &lt;br /&gt; 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;000&lt;ref name=&quot;esa.un.org&quot;/&gt; (2010) &lt;br /&gt; 54 &lt;br /&gt; 8 &lt;br /&gt; [[Kilimanjaro]]&amp;nbsp;5&amp;nbsp;895&amp;nbsp;[[m ö.h.|m&amp;nbsp;ö.h.]] &lt;br /&gt; [[Assalsjön]]&amp;nbsp;156&amp;nbsp;[[Meter över havet|m&amp;nbsp;u.h.]]
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     |-
>     | colspan=&quot;2&quot; | '''Stater'''
>     |-
>     | {{flaggbild2|Algeriet}} &lt;br /&gt; {{flaggbild2|Angola}} &lt;br /&gt; {{flaggbild2|Benin}} &lt;br /&gt; {{flaggbild2|Botswana}} &lt;br /&gt; {{flaggbild2|Burkina Faso}} &lt;br /&gt; {{flaggbild2|Burundi}} &lt;br /&gt; {{flaggbild2|Centralafrikanska republiken}} &lt;br /&gt; {{flaggbild2|Djibouti}} &lt;br /&gt; {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Ekvatorialguinea}} &lt;br /&gt; {{flaggbild2|Elfenbenskusten}} &lt;br /&gt; {{flaggbild2|Eritrea}} &lt;br /&gt; {{flaggbild2|Etiopien}} &lt;br /&gt; {{flaggbild2|Gabon}} &lt;br /&gt; {{flaggbild2|Gambia}} &lt;br /&gt; {{flaggbild2|Ghana}} &lt;br /&gt; {{flaggbild2|Guinea}} &lt;br /&gt; {{flaggbild2|Guinea-Bissau}} &lt;br /&gt; {{flaggbild2|Kamerun}} &lt;br /&gt; {{flaggbild2|Kap Verde}} &lt;br /&gt; {{flaggbild2|Kenya}} &lt;br /&gt; {{flaggbild2|Komorerna}} &lt;br /&gt; {{flaggbild2|Kongo-Brazzaville}} &lt;br /&gt; {{flaggbild2|Kongo-Kinshasa}} &lt;br /&gt; {{flaggbild2|Lesotho}} &lt;br /&gt; {{flaggbild2|Liberia}} &lt;br /&gt;{{flaggbild2|Libyen}} ||{{flaggbild2|Madagaskar}} &lt;br /&gt; {{flaggbild2|Malawi}} &lt;br /&gt; {{flaggbild2|Mali}} &lt;br /&gt; {{flaggbild2|Marocko}} &lt;br /&gt; {{flaggbild2|Mauretanien}} &lt;br /&gt; {{flaggbild2|Mauritius}} &lt;br /&gt; {{flaggbild2|Moçambique}} &lt;br /&gt; {{flaggbild2|Namibia}} &lt;br /&gt; {{flaggbild2|Niger}} &lt;br /&gt; {{flaggbild2|Nigeria}} &lt;br /&gt; {{flaggbild2|Rwanda}} &lt;br /&gt; {{flaggbild2|São Tomé och Príncipe}} &lt;br /&gt; {{flaggbild2|Senegal}} &lt;br /&gt; {{flaggbild2|Seychellerna}} &lt;br /&gt; {{flaggbild2|Sierra Leone}} &lt;br /&gt; {{flaggbild2|Somalia}} &lt;br /&gt; {{flaggbild2|Sudan}} &lt;br /&gt; {{flaggbild2|Sydafrika}} &lt;br /&gt; {{flaggbild2|Sydsudan}} &lt;br /&gt; {{flaggbild2|Swaziland}} &lt;br /&gt; {{flaggbild2|Tanzania}} &lt;br /&gt; {{flaggbild2|Tchad}} &lt;br /&gt; {{flaggbild2|Togo}} &lt;br /&gt; {{flaggbild2|Tunisien}} &lt;br /&gt; {{flaggbild2|Uganda}} &lt;br /&gt; {{flaggbild2|Zambia}} &lt;br /&gt; {{flaggbild2|Zimbabwe}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Stater som har delar av ytan i Afrika'''
>     |-
>     | {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Frankrike}} &lt;br /&gt; {{flaggbild2|Jemen}} &lt;br /&gt; {{flaggbild2|Portugal}} &lt;br /&gt; {{flaggbild2|Spanien}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Särskilda områden'''
>     &lt;br /&gt; {{flaggbild|Ceuta}} [[Ceuta]] ([[Spanien]])
>
>     {{flaggbild|Kanarieöarna}} [[Kanarieöarna]] ([[Spanien]])
>
>     {{flaggbild|Madeira}} [[Madeira]] ([[Portugal]])
>
>     {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>
>     {{flaggbild|Melilla}} [[Melilla]] ([[Spanien]])
>
>     {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>
>     {{flaggbild|Sankta Helena}} [[Sankta Helena]] ([[Storbritannien]])
>
>     [[Fil:Flag of Somaliland.svg|22px|Somaliland]] [[Somaliland]]
>
>     {{flaggbild2|Västsahara}} ([[Marocko]])
>     ----
>     '''Största städer (över 10 miljoner invånare med förorter):'''
>     &lt;br /&gt;{{flaggbild|Egypten}} [[Kairo]]
>     &lt;br /&gt;{{flaggbild|Kongo-Kinshasa}} [[Kinshasa]]
>     &lt;br /&gt;{{flaggbild|Nigeria}} [[Lagos]]
>     ----
>     [[Fil:Africa satellite.jpg|280px|Satellitfoto]]
>     |}
>     '''Afrika''' är [[jorden]]s näst största [[kontinent]] (efter [[Eurasien]]) och även jordens näst största världsdel efter [[Asien]], både vad gäller [[areal]] och [[folkmängd]]. Med världsdelens öar inräknade mäter Afrika 30 244 050&amp;nbsp;km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. Omkring 22 miljoner km² av dessa ligger i [[tropikerna]] vilket gör den afrikanska kontinenten till världens varmaste kontinent. I Afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. Dess längd i nordlig-sydlig riktning är omkring 8 000&amp;nbsp;km och dess största bredd omkring 7&amp;nbsp;800&amp;nbsp;km.
>
>     == Etymologi ==
>     Ordet &quot;Afrika&quot;s [[etymologi]]ska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika [[epok]]er.
>
>     I äldre tider kallades allt land väster om [[Egypten]] för &quot;[[Libyen]]&quot;. Enligt den [[Grekland|grek]]iska historieskrivaren [[Herodotos]] berättelse seglade [[fenicisk]]a sjömän på uppdrag av den egyptiska kungen Neko runt Libyen, det vill säga [[kontinent]]en Afrika redan omkring 600 f.Kr.
>
>     När [[Romerska riket|romarna]] först använde termen ''[[Africa terra]]'' avsåg de före [[andra puniska kriget]] endast trakten kring [[Karthago]], det vill säga motsvarande ungefär dagens Tunisien där själva staden Karthago låg samt stora delar av Libyen och Algeriet. Romarna uppfattade först [[Egypten]] som en del av [[Asien]], och det var först geografen [[Klaudios Ptolemaios|Ptolemaios]] som drog gränsen vid [[Suez]]. Under kejsartiden kom ''Africa'' att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. Namnet ska ha avsett &quot;landet där ''afri''&quot; (singularis: ''afer'') bodde. ''Afer'' avsåg namnet på ett folk som kan motsvara dagens [[berber]]. [[Viking]]arna under 800- och 900-talen kallade norra Afrika för &quot;Blåland&quot;.
>
>     == Geografi ==
>     {{Fördjupning|Afrikas geografi}}
>     Geologiskt är Afrika en mycket gammal kontinent, i huvudsak uppbyggd av [[urberg]]. Under jordens forntid var den en del av jättekontinenten [[Gondwana]]land. Denna gled isär under [[Jura (period)|juraperioden]] och [[Krita (geologi)|kritaperioden]] för 100-150 miljoner år sedan, och så fick Afrika sin nuvarande form.
>
>     Inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050&amp;nbsp;km² inklusive öarna. Afrika åtskiljs från [[Europa]] av [[Medelhavet]] och förenas med [[Asien]] i sitt nordöstra hörn genom [[näs]]et vid [[Suez]], omkring 130&amp;nbsp;km i bredd. Omkring 8 000&amp;nbsp;km skiljer den nordligaste punkten, [[Ras ben Sakka]] i [[Tunisien]], från dess sydligaste, [[Kap Agulhas]] i [[Sydafrika]]. Från [[Kap Verde]], dess västligaste punkt, till [[Ras Hafun]] i [[Somalia]] är det omkring 7 400&amp;nbsp;km. Afrikas kuster mäter omkring 28 000&amp;nbsp;km (1&amp;nbsp;km kust på 1 057&amp;nbsp;km²) varav [[Medelhavet]] står för 5 400&amp;nbsp;km, [[Atlanten]] 11 000&amp;nbsp;km, [[Indiska oceanen]] 8 700&amp;nbsp;km och [[Röda havet]] 2 900&amp;nbsp;km, och avsaknaden av djupare [[bukt]]er och betydande [[halvö]]ar blir uppenbar om man jämför med Europas 9 700 000&amp;nbsp;km² som omgärdas av 32 000&amp;nbsp;km kust (1&amp;nbsp;km kust på 278&amp;nbsp;km² land), eller för det [[fjord]]rika [[Norge]], där motsvarande siffra 385 000&amp;nbsp;km² land och en kuststräcka på 20 000&amp;nbsp;km (1&amp;nbsp;km kust per 19&amp;nbsp;km² land).
>
>     Afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. Jämfört med de andra kontinenterna utmärker sig Afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. Landområdena under 180 [[Meter över havet|m ö.h.]] upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i Asien, utgörs av isolerade berg och bergskedjor. Generellt ligger de högre [[platå]]erna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid [[Röda havet]]s mitt till [[Kongofloden]]s mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. Man kan således dela upp Afrika i fyra övergripande landområden:
>     # Slättlanden vid kusterna – ofta med [[Mangrove|mangroveträsk]] insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. Slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # [[Atlasbergen]] i norr som ligger isolerat av [[Sahara]]öknen som bitvis ligger under havsnivån.
>     # De södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # De nordliga och västra slättlanden, inklusive Sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     Skog täcker cirka en femtedel av Afrikas yta.
>
>     Hela Afrika har ett [[Tropiskt klimat|tropiskt]] eller [[Subtropiskt klimat]].
>
>     === Fauna ===
>
>     {{Fördjupning|Afrikas fauna}}
>
>     Afrika är huvudsakligen känt för den öppna [[savann]]en med väldiga hjordar av hovdjur som [[afrikansk buffel]], gnuer, [[antiloper]], [[giraff]]er och [[zebror]]. Dessa jagas av stora rovdjur som [[lejon]], [[hyenor]] och [[Gepard|geparder]]. Den afrikanska djungeln befolkas av [[ormar]] och [[primater]] samt av vattenlevande djur som [[krokodiler]] och [[groddjur]]. Afrika har även det största antalet arter som kan räknas till [[megafauna]]n, bland annat [[afrikanska elefanter]], [[trubbnoshörning]] och [[flodhäst]], på grund av att kontinenten bara i mindre utsträckning var påverkad av det [[massutdöende]] som skedde under [[pleistocen]].
>
>     == Historia ==
>     {{Fördjupning|Afrikas historia}}
>     Den första mänskliga migrationen, från kontinenten men även inom Afrikas kuster, kan spåras [[lingvistisk]]t, kulturellt och genom datoranalyser av [[genetik|genetiskt]] material. Någonstans i [[Östafrika]] tror dagens forskare att de första [[hominider]] uppkom. I [[Etiopien]] har man hittat ett två miljoner år gammalt skelett som uppkallades [[Australopithecus afarensis|Lucy]], den första kända mänskliga [[primater|primaten]].
>
>     === Tidigare civilisationer och handel ===
>     [[Fil:African-civilizations-map-pre-colonial.svg|miniatyr|höger|Några av de mest kända afrikanska civilisationerna. Civilisationer från olika tidsepoker visas parallellt.]]Lingvistiska spår tyder på att [[Bantu]]folk emigrerade från [[Västafrika]] in i [[khoisan]]folkets (de som tidigare kallades [[bushmän]] och [[hottentott]]er) område och undanträngde dem. Bantufolken använde en uppsättning grödor, bland annat [[kassava]] och [[jams]], som var mer lämpade för jordbruk i det [[tropik|tropiska]] Afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. Bantufolkens traditionella bosättningsområde sträcker sig från [[Sahara]]öknen till de [[Tempererat klimat|tempererade regionerna]] i söder. Historiskt har deras vapen varit spjut, sköld, pil och båge. Det för [[khoisanspråk]]ens karaktäristiska klickljuden återfinns i [[nguni (språk)|nguni]]- och [[bantu]]språk som till exempel [[xhosa]] och [[zulu]]. Khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra Afrika i länderna kring [[Kalahariöknen]].
>
>     Afrika söder om Sahara har bestått olika stater och riken, mindre stadsstater samt [[stam (samhällsvetenskap)|stammar]] och [[klan]]er. Det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska [[civilisation]]erna. Det fanns även statsbildningar i [[Nubien]], [[Ghana]], [[Timbuktu]], [[Kanem-Bornu]] och i nuvarande [[Zimbabwe]].
>
>     [[Nordafrika]] kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds [[Islam]] och den arabiska kulturen genom Sahara och vidare söderut. Arabiska och persiska handelsmän spred likaså Islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. Detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till Java, därmed den etniska, kulturella och språkliga mångfalden på Östafrikas kust, [[Madagaskar]] och [[Komorerna]].
>
>     === Arabvärlden och Europa utforskar kontinenten ===
>     Det arabiska intresset för Afrika går långt tillbaka, men handel med slavar från centra som Lamu och Zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.Kr. De miljoner slavar som exporterades till arabvärlden och österut under den [[arabiska slavhandeln]] finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. Slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel Sudan.&lt;ref&gt;[http://www.hrw.org/backgrounder/africa/sudanupdate.htm Human Rights Watch] [http://news.bbc.co.uk/2/hi/programmes/this_world/6458377.stm BBC World]&lt;/ref&gt;
>
>     Under [[1300-talet]] började vissa europeiska handelsexpeditioner få kontakt med Nord- och Västafrika.
>
>     Från 1500-talets mitt fram till förbudet mot slavhandel i Storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst Väst- och Centralafrika. Flera miljoner afrikaner såldes på detta vis som slavar till främst de Europeiska kolonierna i Amerika.
>
>     Tidiga kolonisatörer var [[araber]], [[Portugal|portugiser]] och [[Spanien|spanjorer]]. Därefter koloniserades kontinenten av framförallt av [[Storbritannien]], [[Frankrike]], [[Spanien]], [[Italien]], [[Belgien]], [[Nederländerna]], [[Tyskland]] och [[Portugal]].
>
>     === Koloniseringen ===
>     {{Fördjupning|Kapplöpningen om Afrika}}
>     [[Fil:Map_of_Africa_from_Encyclopaedia_Britannica_1890.jpg|miniatyr|Politisk karta över Afrika från 1890.]] Från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. Holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av Afrika. Portugiserna grundade flera kolonier i västra Afrika och vid nuvarande Moçambique. I mitten av [[1800-talet]] leddes flera expeditioner in i de centrala delarna av Afrika, bland annat av Livingstone och Speke, men också den svenske upptäcktsresanden Andersson. Denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. [[Leopold II av Belgien|Kung Leopolds]] rovdrift på sin privata koloni [[Kongostaten]] är ökänd och kostade miljontals afrikanska liv.
>
>     I Afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket Egypten, republiken Liberia och konungariket Abessinien. Tanger bildade med sitt område en internationell zon. För övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;{{runeberg.org|nfcn|0245.html Supplement Afrika}}&lt;/ref&gt;
>
>     Flertalet kolonier erhöll [[självständighet]] under slutet av 1950-talet och början av 1960-talet. Avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i Rhodesia (nuvarande Zimbabwe), Rwanda, Uganda, Nigeria m.fl. stater.
>
>     == Politik ==
>     {{Fördjupning|Afrikas politik}}
>     [[Fil:Electoral_Democracies_in_Africa,_Freedom_House_2007.png|miniatyr|De blå länderna är demokratier enligt tankesmedjan Freedom House.]]
>     Det postkoloniala Afrikas politik har i hög grad präglats av diktatur, statskupper och brist på [[mänskliga rättigheter]].
>
>     [[Nation]]erna i Afrika har ofta visat sig vara instabila [[stat]]er som efter självständigheten hemsökts av korruption, våldsamma [[statskupp]]er, [[despotism]] och [[dikatur|militärdiktaturer]]. Under perioden [[1960]]-[[1989]] genomfördes över 70 kupper och 13 [[mord]] på [[president]]er. Under det [[kalla kriget]] kämpade [[USA]] och [[Sovjetunionen]] om makten i Afrika, och tillsammans med länder som [[Frankrike]] och [[Kuba]], stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. Enligt en spridd uppfattning beror många [[militär]]a konflikter om [[territorium|territorier]] och [[gräns]]er under Afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter [[Kongokonferensen]], men detta kan inte visas eftersom något jämförelseområde inte finns. Konflikter i Afrika har ofta drivits av tillgången till kontinentens rika [[råvara|råvarutillgångar]] – [[diamant]]er och [[metall]]er som [[guld]], [[platina]], [[uran]] och [[koppar]].
>
>     Korruption och havererade politiska program har lett till utbredd [[undernäring]] och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. Sanitetsproblemen har också orsakat en akut spridning av bland annat [[HIV]]-viruset som leder till [[sjukdom]]en [[AIDS]].
>
>     Trots nöd och umbäranden finns vissa positiva tecken – [[demokrati]]skt valda [[regering]]ar växer i antal även om de inte utgör någon majoritet ännu. Genom påtryckningar från till exempel [[Internationella valutafonden]] (IMF) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. Om man beaktar IMF:s katastrofala misslyckanden i länder som [[Argentina]] och [[Filippinerna]] är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad Afrika faktiskt behöver.
>
>     De afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. Hittills har dock detta inte lyckats särskilt väl. Under [[inbördeskrig]]et i [[Kongo-Kinshasa|Kongo]] var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. Antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     De politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. Organisationen för afrikansk enighet (OAU 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället ''[[Afrikanska unionen]]'', som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. Hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i Darfur-provinsen i Sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. En avgörande skillnad mellan såväl OAU som AU å ena sidan och EU å den andra, är att EU startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. OAU och AU har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. På så sätt liknar AU mer FN än EU.
>
>     == Ekonomi ==
>     {{Fördjupning|Afrikas ekonomi}}
>     [[Fil:Bakweri cocoyam farmer from Cameroon.jpg|250px|miniatyr|Jordbrukare på en [[jams]]odling i Kamerun]]
>     Afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater ([[Juli 2005]]). Afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. På [[FN]]:s Human Development Report från [[2003]] (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i Afrika.
>
>     Det finns dock länder som lyckas bra ekonomiskt, bland andra [[Botswana]], [[Ghana]] och [[Sydafrika]]. [[Etiopien]] har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. Sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat [[guld]] och [[diamanter]] och ett väl fungerande rättsväsen. Det finns i Sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. Det finns även andra afrikanska länder, exempelvis [[Egypten]], som länge haft en väl fungerande handelssektor och ekonomi.
>
>     Kontinentens kust mot [[Atlanten]] återfinns stora fyndigheter av [[Petroleum|olja]]. I [[Nigeria]] finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     Afrikas jordbruk sysselsätter drygt hälften av Afrikas befolkning och är den viktigaste näringen. Cirka 6 % (ungefär 1 814 600&amp;nbsp;km²) av världsdelens yta är uppodlad, mest i savannområdena. I stora delar av de brukningsbara områdena är jordens bördighet ganska låg. De bästa jordarna återfinns i begränsade höglandsområden som i till exempel [[Kamerun]] och [[Östafrika]].
>
>     == Demografi ==
>     {{Fördjupning|Afrikas demografi}}
>     ===Befolkning===
>     Människan, ''[[Homo sapiens|Homo sapiens sapiens]]'', antas härstamma från Afrika och spred sig härifrån till de omgivande kontinenterna. Den svarta befolkningen söder om Sahara tillhör den ursprungliga befolkningen som hela tiden levat i Afrika.
>
>     Norr om Sahara präglar samröret med övriga [[Medelhavet|Medelhavsområdet]] och inte minst sydvästra [[Asien]] både befolkning, språk och kultur.
>
>     På ön [[Madagaskar]], som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från [[Indonesien]]. Under kolonialtiden skedde inflyttning från [[Europa]] och [[Asien]] till södra Afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     Afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga [[födelsetal]] i kombination med sjunkande [[Mortalitet (demografi)|dödstal]] med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. Även om befolkningstätheten varierar kraftigt mellan olika regioner är Afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     Afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. Även här är variationerna stora, och i norra och östra Afrika finns en lång urban tradition.
>
>     === Språk ===
>     {{Fördjupning|Afrikas språk}}
>     [[Fil:African languages SV.png|miniatyr|300px|Språkfamiljer i Afrika]]
>     Uppskattningsvis finns det ungefär 1800 talade [[språk]] i Afrika idag. Vissa av dessa, till exempel [[Swahili]], [[Hausa]], och [[Yoruba (språk)|Yoruba]], talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. Afrika har även många olika [[teckenspråk]]. De flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna [[afroasiatiska språk]], [[nilo-sahariska språk]], [[Niger-Kongospråk]] eller [[khoisanspråk]].
>     Med några få nämnvärda undantag i östra Afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. Till exempel används i ett stort antal afrikanska länder [[franska]] eller [[engelska]] som administrativt språk för stat, handel, utbildning och media. [[Arabiska]], [[portugisiska]], [[afrikaans]] och [[malagassiska]] är andra exempel på språk med ursprung utanför Afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === Religion ===
>     {{Fördjupning|Afrikas religion}}{{Se även|Kristendomen i Afrika}}
>     [[Kristendom]] och [[islam]] är de största religionerna i Afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. Resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. Ett mindre antal afrikaner är [[Hinduism|hinduer]], eller har tro från [[Judendom|judisk]] tradition. Exempel på [[Afrikanska judar|afrikanska folk som tillhör judendomen]] är [[Beta Israel]], [[Lemba]]folk och [[Abayudaya]] i östra [[Uganda]]. Sydafrika och Angola har över 55% kristna katoliker.
>
>     == Kultur ==
>     {{Fördjupning|Afrikas kultur}}
>     Afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. Den mest konventionella distinktionen är den mellan området söder om Sahara och området norr om öknen. De nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. En av de största kulturella identiteterna är den som innefattar alla folk som talar något [[bantuspråk]].
>
>     Det går också att göra en indelning i gamla franska Västafrika å ena sidan, och resten av Afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra Afrika) å andra sidan. En annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. Det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     [[Afrikansk konst]] och [[afrikansk arkitektur]] reflekterar de afrikanska kulturernas diversitet. De äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av [[nassarius]]skal, dessa hittades i [[Blombosgrottan]]. [[Pyramiderna i Giza]] var världens högsta byggnader under 4 000 år. [[Klippkyrkorna i Lalibela]] i [[Etiopien]], bland vilka ''[[Bete Giyorgis]]'' (&quot;St. Göranskyrkan&quot;) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == Regioner och territorier ==
>     [[Fil:Africa-regions.png|miniatyr|Regioner i Afrika:
>     {{legend|#0000ff|[[Nordafrika]]}}
>     {{legend|#00ff00|[[Västafrika]]}}
>     {{legend|#ff00ff|[[Centralafrika]]}}
>     {{legend|#ffa500|[[Östafrika]]}}
>     {{legend|#ff0000|[[Södra Afrika]]}}]]
>     {| class=&quot;wikitable sortable&quot;
>     |+
>     |- bgcolor=&quot;#ECECEC&quot;
>     ! Regioners namn&lt;ref&gt;Enligt [[:Fil:United Nations geographical subregions.png|FN:s regionindelning]] av världen.&lt;br /&gt;&lt;/ref&gt; och&lt;br /&gt; territorier, med flagga
>     ! data-sort-type=number | [[Area]]&lt;br /&gt;(km²)
>     ! data-sort-type=number |[[Folkmängd]]&lt;br /&gt;(2010)&lt;ref name=&quot;esa.un.org1&quot;/&gt;
>     ! data-sort-type=number |[[Befolkningstäthet]]&lt;br /&gt;(per km²)
>     ! [[Huvudstad]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Centralafrika]]''':
>     |-
>     | {{flaggbild2|Angola}}
>     | align=&quot;right&quot; | 1 246 700
>     | align=&quot;right&quot; | 19&amp;nbsp;081&amp;nbsp;912
>     | align=&quot;right&quot; | 15,3
>     | [[Luanda]]
>     |-
>     | {{flaggbild2|Kamerun}}
>     | align=&quot;right&quot; | 475 440
>     | align=&quot;right&quot; | 19 598 889
>     | align=&quot;right&quot; | 41,2
>     | [[Yaoundé]]
>     |-
>     | {{flaggbild2|Centralafrikanska republiken}}
>     | align=&quot;right&quot; | 622 984
>     | align=&quot;right&quot; | 4 401 051
>     | align=&quot;right&quot; | 7,1
>     | [[Bangui]]
>     |-
>     | {{flaggbild2|Tchad}}
>     | align=&quot;right&quot; | 1 284 000
>     | align=&quot;right&quot; | 11 227 208
>     | align=&quot;right&quot; | 8,7
>     | [[N'Djamena]]
>     |-
>     | {{flaggbild2|Kongo-Brazzaville}}
>     | align=&quot;right&quot; | 342 000
>     | align=&quot;right&quot; | 4 042 899
>     | align=&quot;right&quot; | 12,9
>     | [[Brazzaville]]
>     |-
>     | {{flaggbild2|Kongo-Kinshasa}}
>     | align=&quot;right&quot; | 2 345 410
>     | align=&quot;right&quot; | 65 965 795
>     | align=&quot;right&quot; | 28,1
>     | [[Kinshasa]]
>     |-
>     | {{flaggbild2|Ekvatorialguinea}}
>     | align=&quot;right&quot; | 28 051
>     | align=&quot;right&quot; | 700 401
>     | align=&quot;right&quot; | 24,9
>     | [[Malabo]]
>     |-
>     | {{flaggbild2|Gabon}}
>     | align=&quot;right&quot; | 267 667
>     | align=&quot;right&quot; | 1 505 463
>     | align=&quot;right&quot; | 5,6
>     | [[Libreville]]
>     |-
>     | {{flaggbild2|São Tomé och Príncipe}}
>     | align=&quot;right&quot; | 1 001
>     | align=&quot;right&quot; | 165 397
>     | align=&quot;right&quot; | 165,0
>     | [[São Tomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Nordafrika]]''':
>     |-
>     | {{flaggbild2|Algeriet}}
>     | align=&quot;right&quot; | 2 381 740
>     | align=&quot;right&quot; | 35 468 000
>     | align=&quot;right&quot; | 14,9
>     | [[Alger (stad)|Alger]]
>     |-
>     | {{flaggbild2|Egypten}}&lt;ref&gt;[[Egypten]] brukar anses ligga i två världsdelar, Nordafrika (enligt FN:s regionindelning) och Västasien; befolknings- och arealuppgifterna avser endast den afrikanska delen, väster om [[Suezkanalen]].&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 1 001 450
>     | align=&quot;right&quot; | 81 121 000
>     | align=&quot;right&quot; | 81,0
>     | [[Kairo]]
>     |-
>     | {{flaggbild2|Libyen}}
>     | align=&quot;right&quot; | 1 759 540
>     | align=&quot;right&quot; | 6 355 112
>     | align=&quot;right&quot; | 3,6
>     | [[Tripoli]]
>     |-
>     | {{flaggbild2|Marocko}}
>     | align=&quot;right&quot; | 446 550
>     | align=&quot;right&quot; | 31 951 412
>     | align=&quot;right&quot; | 71,5
>     | [[Rabat]]
>     |-
>     | {{flaggbild2|Sudan}}
>     | align=&quot;right&quot; | 1 886 068
>     | align=&quot;right&quot; | 33 551 941
>     | align=&quot;right&quot; | 17,8
>     | [[Khartoum]]
>     |-
>     | {{flaggbild2|Tunisien}}
>     | align=&quot;right&quot; | 163 610
>     | align=&quot;right&quot; | 10 480 934
>     | align=&quot;right&quot; | 64,1
>     | [[Tunis]]
>     |-
>     | {{flaggbild2|Västsahara}} ([[Marocko]])&lt;ref&gt;[[Västsahara]] är ockuperat av [[Marocko]], som också har anspråk på området.&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 266 000
>     | align=&quot;right&quot; | 530 500
>     | align=&quot;right&quot; | 2,0
>     | [[El Aaiún]]
>     |-
>     | colspan=5 | ''Områden som är likställda delar av europeiska stater'':
>     |-
>     | [[Fil:Flag of the Canary Islands.svg|22x20px|border|Kanarieöarna]] [[Kanarieöarna]] ([[Spanien]])&lt;ref&gt;De till [[Spanien]] hörande [[Kanarieöarna]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]] och [[Västsahara]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 7 492
>     | align=&quot;right&quot; | 2 117 519
>     | align=&quot;right&quot; | 282,6
>     | [[Las Palmas de Gran Canaria]],&lt;br /&gt;[[Santa Cruz de Tenerife]]
>     |-
>     | [[Fil:Flag Ceuta.svg|22x20px|border|Ceuta]] [[Ceuta]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Ceuta]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 20
>     | align=&quot;right&quot; | 78 674
>     | align=&quot;right&quot; | 3933,7
>     | –
>     |-
>     | [[Fil:Flag of Madeira.svg|22x20px|Madeira]] [[Madeira]] ([[Portugal]])&lt;ref&gt;Det till [[Portugal]] hörande [[Madeira]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 797
>     | align=&quot;right&quot; | 267 302
>     | align=&quot;right&quot; | 335,4
>     | [[Funchal]]
>     |-
>     | [[Fil:Flag of Melilla.svg|22x20px|Melilla]] [[Melilla]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Melilla]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 12
>     | align=&quot;right&quot; | 73 460
>     | align=&quot;right&quot; | 6121,6
>     | –
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Södra Afrika]]''':
>     |-
>     | {{flaggbild2|Botswana}}
>     | align=&quot;right&quot; | 600 370
>     | align=&quot;right&quot; | 2 006 945
>     | align=&quot;right&quot; | 3,3
>     | [[Gaborone]]
>     |-
>     | {{flaggbild2|Lesotho}}
>     | align=&quot;right&quot; | 30 355
>     | align=&quot;right&quot; | 2 171 318
>     | align=&quot;right&quot; | 71,5
>     | [[Maseru]]
>     |-
>     | {{flaggbild2|Namibia}}
>     | align=&quot;right&quot; | 825 418
>     | align=&quot;right&quot; | 2 283 289
>     | align=&quot;right&quot; | 2,7
>     | [[Windhoek]]
>     |-
>     | {{flaggbild2|Sydafrika}}
>     | align=&quot;right&quot; | 1 219 912
>     | align=&quot;right&quot; | 50 132 817
>     | align=&quot;right&quot; | 41,1
>     | [[Bloemfontein]], [[Kapstaden]], [[Pretoria]] (Tshwane)&lt;ref&gt;[[Bloemfontein]] är [[Sydafrika]]s judiciella huvudstad, medan [[Kapstaden]] är säte för den lagstiftande församlingen och [[Pretoria]] är administrativ huvudstad. Staden Pretoria har inlett en process för att eventuellt byta namn till Tshwane.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Swaziland}}
>     | align=&quot;right&quot; | 17 363
>     | align=&quot;right&quot; | 1 186 056
>     | align=&quot;right&quot; | 68,3
>     | [[Mbabane]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Västafrika]]''':
>     |-
>     | {{flaggbild2|Benin}}
>     | align=&quot;right&quot; | 112 620
>     | align=&quot;right&quot; | 8 849 892
>     | align=&quot;right&quot; | 78,6
>     | [[Porto-Novo]]
>     |-
>     | {{flaggbild2|Burkina Faso}}
>     | align=&quot;right&quot; | 274 200
>     | align=&quot;right&quot; | 16 468 714
>     | align=&quot;right&quot; | 60,1
>     | [[Ouagadougou]]
>     |-
>     | {{flaggbild2|Kap Verde}}
>     | align=&quot;right&quot; | 4 033
>     | align=&quot;right&quot; | 495 999
>     | align=&quot;right&quot; | 123,0
>     | [[Praia]]
>     |-
>     | {{flaggbild2|Elfenbenskusten}}
>     | align=&quot;right&quot; | 322 460
>     | align=&quot;right&quot; | 19 737 800
>     | align=&quot;right&quot; | 61,2
>     | [[Abidjan]], [[Yamoussoukro]]&lt;ref&gt;[[Yamoussoukro]] är [[Elfenbenskusten]]s officiella huvudstad, men [[Abidjan]] fungerar ''[[de facto]]'' som huvudstad.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Gambia}}
>     | align=&quot;right&quot; | 11 300
>     | align=&quot;right&quot; | 1 728 394
>     | align=&quot;right&quot; | 152,9
>     | [[Banjul]]
>     |-
>     | {{flaggbild2|Ghana}}
>     | align=&quot;right&quot; | 239 460
>     | align=&quot;right&quot; | 24 391 823
>     | align=&quot;right&quot; | 101,9
>     | [[Accra]]
>     |-
>     | {{flaggbild2|Guinea}}
>     | align=&quot;right&quot; | 245 857
>     | align=&quot;right&quot; | 9 981 590
>     | align=&quot;right&quot; | 40,6
>     | [[Conakry]]
>     |-
>     | {{flaggbild2|Guinea-Bissau}}
>     | align=&quot;right&quot; | 36 120
>     | align=&quot;right&quot; | 1 515 224
>     | align=&quot;right&quot; | 41,9
>     | [[Bissau]]
>     |-
>     | {{flaggbild2|Liberia}}
>     | align=&quot;right&quot; | 111 370
>     | align=&quot;right&quot; | 3 994 122
>     | align=&quot;right&quot; | 35,9
>     | [[Monrovia]]
>     |-
>     | {{flaggbild2|Mali}}
>     | align=&quot;right&quot; | 1 240 000
>     | align=&quot;right&quot; | 15 369 809
>     | align=&quot;right&quot; | 9,1
>     | [[Bamako]]
>     |-
>     | {{flaggbild2|Mauretanien}}
>     | align=&quot;right&quot; | 1 030 700
>     | align=&quot;right&quot; | 3 459 773
>     | align=&quot;right&quot; | 3,4
>     | [[Nouakchott]]
>     |-
>     | {{flaggbild2|Niger}}
>     | align=&quot;right&quot; | 1 267 000
>     | align=&quot;right&quot; | 15 511 953
>     | align=&quot;right&quot; | 12,2
>     | [[Niamey]]
>     |-
>     | {{flaggbild2|Nigeria}}
>     | align=&quot;right&quot; | 923 768
>     | align=&quot;right&quot; | 158 423 182
>     | align=&quot;right&quot; | 171,5
>     | [[Abuja]]
>     |-
>     | [[Fil:Flag of Saint Helena.svg|22x20px|Sankta Helena]] [[Sankta Helena (ö)|Sankta Helena]] ([[Storbritannien]])
>     | align=&quot;right&quot; | 410
>     | align=&quot;right&quot; | 4 118
>     | align=&quot;right&quot; | 10,0
>     | [[Jamestown, Sankta Helena|Jamestown]]
>     |-
>     | {{flaggbild2|Senegal}}
>     | align=&quot;right&quot; | 196 190
>     | align=&quot;right&quot; | 12 433 728
>     | align=&quot;right&quot; | 63,4
>     | [[Dakar]]
>     |-
>     | {{flaggbild2|Sierra Leone}}
>     | align=&quot;right&quot; | 71 740
>     | align=&quot;right&quot; | 5 867 536
>     | align=&quot;right&quot; | 81,8
>     | [[Freetown]]
>     |-
>     | {{flaggbild2|Togo}}
>     | align=&quot;right&quot; | 56 785
>     | align=&quot;right&quot; | 6 027 798
>     | align=&quot;right&quot; | 106,1
>     | [[Lomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Östafrika]]''':
>     |-
>     | {{flaggbild2|Burundi}}
>     | align=&quot;right&quot; | 27 830
>     | align=&quot;right&quot; | 8 382 849
>     | align=&quot;right&quot; | 301,2
>     | [[Bujumbura]]
>     |-
>     | {{flaggbild2|Komorerna}}
>     | align=&quot;right&quot; | 2 170
>     | align=&quot;right&quot; | 734 750
>     | align=&quot;right&quot; | 338,6
>     | [[Moroni]]
>     |-
>     | {{flaggbild2|Djibouti}}
>     | align=&quot;right&quot; | 23 000
>     | align=&quot;right&quot; | 888 716
>     | align=&quot;right&quot; | 38,6
>     | [[Djibouti (stad)|Djibouti]]
>     |-
>     | {{flaggbild2|Eritrea}}
>     | align=&quot;right&quot; | 121 320
>     | align=&quot;right&quot; | 5 253 676
>     | align=&quot;right&quot; | 43,3
>     | [[Asmara]]
>     |-
>     | {{flaggbild2|Etiopien}}
>     | align=&quot;right&quot; | 1 127 127
>     | align=&quot;right&quot; | 82 949 541
>     | align=&quot;right&quot; | 73,6
>     | [[Addis Abeba]]
>     |-
>     | {{flaggbild2|Kenya}}
>     | align=&quot;right&quot; | 582 650
>     | align=&quot;right&quot; | 40 512 682
>     | align=&quot;right&quot; | 69,5
>     | [[Nairobi]]
>     |-
>     |{{flaggbild2|Madagaskar}}
>     | align=&quot;right&quot; | 587 040
>     | align=&quot;right&quot; | 20 713 819
>     | align=&quot;right&quot; | 35,3
>     | [[Antananarivo]]
>     |-
>     | {{flaggbild2|Malawi}}
>     | align=&quot;right&quot; | 118 480
>     | align=&quot;right&quot; | 14 900 841
>     | align=&quot;right&quot; | 90,3
>     | [[Lilongwe]]
>     |-
>     | {{flaggbild2|Mauritius}}
>     | align=&quot;right&quot; | 2 040
>     | align=&quot;right&quot; | 1 299 172
>     | align=&quot;right&quot; | 636,8
>     | [[Port Louis]]
>     |-
>     | {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>     | align=&quot;right&quot; | 374
>     | align=&quot;right&quot; | 204 114
>     | align=&quot;right&quot; | 545,8
>     | [[Mamoudzou]]
>     |-
>     | {{flaggbild|Moçambique}} [[Moçambique]]
>     | align=&quot;right&quot; | 801 590
>     | align=&quot;right&quot; | 23 390 765
>     | align=&quot;right&quot; | 29,2
>     | [[Maputo]]
>     |-
>     | {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>     | align=&quot;right&quot; | 2 512
>     | align=&quot;right&quot; | 846 068
>     | align=&quot;right&quot; | 336,8
>     | [[Saint-Denis (Réunion)|Saint-Denis]]
>     |-
>     | {{flaggbild2|Rwanda}}
>     | align=&quot;right&quot; | 26 338
>     | align=&quot;right&quot; | 10 624 005
>     | align=&quot;right&quot; | 403,4
>     | [[Kigali]]
>     |-
>     | {{flaggbild2|Seychellerna}}
>     | align=&quot;right&quot; | 455
>     | align=&quot;right&quot; | 86 518
>     | align=&quot;right&quot; | 190,1
>     | [[Victoria, Seychellerna|Victoria]]
>     |-
>     | {{flaggbild2|Somalia}}
>     | align=&quot;right&quot; | 637 657
>     | align=&quot;right&quot; | 9 330 872
>     | align=&quot;right&quot; | 14,6
>     | [[Mogadishu]]
>     |-
>     | {{flaggbild2|Sydsudan}}
>     | align=&quot;right&quot; | 633 783
>     | align=&quot;right&quot; | 8 207 607
>     | align=&quot;right&quot; | 13
>     | [[Juba]]
>     |-
>     | {{flaggbild2|Tanzania}}
>     | align=&quot;right&quot; | 945 087
>     | align=&quot;right&quot; | 44 841 226
>     | align=&quot;right&quot; | 47,4
>     | [[Dodoma]]
>     |-
>     | {{flaggbild2|Uganda}}
>     | align=&quot;right&quot; | 236 040
>     | align=&quot;right&quot; | 33 424 683
>     | align=&quot;right&quot; | 141,6
>     | [[Kampala]]
>     |-
>     | {{flaggbild2|Zambia}}
>     | align=&quot;right&quot; | 752 614
>     | align=&quot;right&quot; | 13 088 570
>     | align=&quot;right&quot; | 17,4
>     | [[Lusaka]]
>     |-
>     | {{flaggbild2|Zimbabwe}}
>     | align=&quot;right&quot; | 390 580
>     | align=&quot;right&quot; | 12 571 454
>     | align=&quot;right&quot; | 32,2
>     | [[Harare]]
>     |- style=&quot; font-weight:bold; &quot;
>     | Total
>     | align=&quot;right&quot; | 30&amp;nbsp;305&amp;nbsp;053
>     | align=&quot;right&quot; | 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;400
>     | align=&quot;right&quot; | 33,7
>     |}
>
>     == Se även ==
>     * [[Nordafrika]]
>     * [[Västafrika]]
>     * [[Östafrika]]
>     * [[Centralafrika]]
>     * [[Södra Afrika]]
>     * [[Afrikas klimat]]
>
>     == Referenser ==
>
>     === Noter ===
>     &lt;references&gt;
>     &lt;ref name=&quot;esa.un.org&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[Förenta nationerna|United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;ref name=&quot;esa.un.org1&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;/references&gt;
>
>     === Källor ===
>     {{enwp|url=https://en.wikipedia.org/w/index.php?title=Africa&amp;oldid=81702686|artikel=Africa}}
>     * [https://www.courses.psu.edu/test/test100_hkr/AFIM/Main_HTML/Intro_1.html Pennsylvania State University – ''African Ethnic Groups'' och andra kartor]
>     * [http://www.pygmies.info/ Pygméerna i Afrika] Kultur och musik av de första urinvånarna i Afrika
>     * Afrika i skolböckerna, M. Palmberg 1987
>
>     == Externa länkar ==
>     {{commonscat|Africa}}
>     {{Wikinews|Afrika}}
>     {{wikiatlas|Africa}}
>
>     {{Afrika}}
>     {{Världens regioner}}
>     {{auktoritetsdata}}
>
>     [[Kategori:Afrika| ]]
>     [[Kategori:Wikipedia:Basartiklar]]</text>
>           <sha1>1ueywtz2sm89v81ujno96unvkkmxpoa</sha1>
>         </revision>
>       </page>), (55447,<page>
>         <title>Amerika</title>
>         <ns>0</ns>
>         <id>3</id>
>         <revision>
>           <id>48343172</id>
>           <parentid>48187017</parentid>
>           <timestamp>2020-10-11T13:36:09Z</timestamp>
>           <contributor>
>             <username>Olympicbandy</username>
>             <id>564328</id>
>           </contributor>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="2728" xml:space="preserve">{{för|Amerika som kortnamn på '''Amerikas förenta stater'''|USA|andra betydelser|Amerika (olika betydelser)}}
>     [[Fil:Americas (orthographic projection).svg|miniatyr|250px|Karta som visar Nord- och Sydamerika, tillsammans benämnda Amerika]]
>     '''Amerika''' (engelska: ''the Americas'') består av landmassorna mellan [[Atlanten]] och [[Stilla havet]] på [[västra halvklotet]], namngivna efter den italienske upptäcktsresanden [[Amerigo Vespucci]]. Amerika uppfattas vanligen idag som två världsdelar, [[Nordamerika]] och [[Sydamerika]].&lt;ref&gt;[https://snl.no/Amerika Store norske leksikon/ Amerika]&lt;/ref&gt; Amerika indelas geografiskt i Nordamerika (med [[Centralamerika]]) och Sydamerika. [[Grönland]] ingår geografiskt i Nordamerika, men räknas politiskt och kulturellt till Europa. Kulturgeografiskt avgränsas också ofta [[Latinamerika]], då inklusive både Central- och ...

  

Keep only articles
------------------

In [ ]:
import scala.util.matching.UnanchoredRegex
var regex = s"<ns>0</ns>".r
val articles_dump = dump.filter({ case (id, source) => regex.findFirstIn(source.toString()) != None})
articles_dump.take(1)

  

>     import scala.util.matching.UnanchoredRegex
>     regex: scala.util.matching.Regex = <ns>0</ns>
>     articles_dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[11] at filter at command-2158711750341777:3
>     res1: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>))

In [ ]:
import scala.util.matching.UnanchoredRegex
var regex = s"<redirect title".r
val filtered_dump = articles_dump.filter({ case (id, source) => regex.findFirstIn(source.toString()) == None})
filtered_dump.take(1)

  

>     import scala.util.matching.UnanchoredRegex
>     regex: scala.util.matching.Regex = <redirect title
>     filtered_dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[12] at filter at command-2158711750341794:3
>     res2: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>))

In [ ]:
val corpus_dump = filtered_dump

  

>     corpus_dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[12] at filter at command-2158711750341794:3

  

Filtering
=========

In [ ]:
def filterSource(source: String) : String = {
  
  // There is an &amp;nbsp; where the &amp; provides the & in &nbsp;
  // I should therefore replace &amp; first
  val noAmp = raw"&amp;".r.replaceAllIn(source, "&")
  
  // Loop to remove some specific sections
  val sections = Seq("Se även", "Referenser", "Noter", "Källor", "Externa länkar")
  var current = noAmp
  for (section <- sections) {
    val regexp = s"[=]+ *$section *[=]+".r
    if (regexp.findAllIn(current).hasNext)
    {
      val a = regexp.findAllIn(current).start
      val pre = current.substring(0, a)
      var post = current.substring(a, current.length)
      post = regexp.replaceAllIn(post, "")
      if ("^=".r.findAllIn(post).hasNext) {
        val b = "^=".r.findAllIn(post).start
        post = post.substring(b, post.length)
      }
      else {
        post = ""        
      }
      current = pre + post

    }
  }

  val lowerCase = current.toLowerCase  
  
  // Loop to remove some specific tags
  val tags = Seq("id", "ns", "parentid", "timestamp", "contributor", "comment", "model", "format", "sha1")
  current = lowerCase
  for (tag <- tags) {
    val start = s"<$tag>".r
    val end = s"</$tag>".r
    while (start.findAllIn(current).hasNext) {
      val a = start.findAllIn(current).start
      val b = end.findAllIn(current).start
      val len = current.length
      current = current.substring(0, a) + current.substring(b+tag.length+3, current.length)
      assert (current.length < len)
    }

  }
  val removedSpecificTags = current

  val noTags = raw"<[^<>]*>".r.replaceAllIn(removedSpecificTags, "")
  
  // A loop to remove innermost curly braces, then outer such things.
  val curly = raw"\{[^\{\}]*\}".r
  current = noTags
  while (curly.findAllIn(current).hasNext)
  {
     current = curly.replaceAllIn(current, "")
  }
  val noCurly = current
  
  // val noNumbers = raw"\d".r.replaceAllIn(noCurly, "")
  
  // Unnamed internal links appear in figure captions, confusing later matching
  val noUnnamedInternalLinks = raw"\[\[([^\|\]]*)\]\]".r.replaceAllIn(noCurly, m => m.group(1))

  // Loop to remove internal links and their properties
  current = noUnnamedInternalLinks
  val regexp = raw"\[\[[^\|\[]*\|([^\[\]]*)\]\]".r 
  while (regexp.findAllIn(current).hasNext) {
    val len = current.length
    current = regexp.replaceAllIn(current, m => "[[" + m.group(1) + "]]")
    assert (current.length < len)
  }
  val noNamedInternalLinks = current
  
  val noInternalLinks = raw"\[\[([^\]]*)\]\]".r.replaceAllIn(noNamedInternalLinks, m => m.group(1))
  val noNamedExternalLinks = raw"\[[^ \[]*\ ([^\]]*)\]".r.replaceAllIn(noInternalLinks, m => m.group(1))
  val noExternalLinks = raw"\[([^\]]*)\]".r.replaceAllIn(noNamedExternalLinks, m => m.group(1))
  
  val nbsp = raw"&nbsp;".r.replaceAllIn(noExternalLinks, " ")
  val noQuotes = raw"&quot;".r.replaceAllIn(nbsp, "")
  current = noQuotes
  /*
  val noQuoteChars = raw"'".r.replaceAllIn(noQuotes, "")
  // Loop to remove double whitespace characters
  current = noQuoteChars
  val doubleWhitespace = raw"\s(\s)".r
  while (doubleWhitespace.findAllIn(current).hasNext) {
    current = doubleWhitespace.replaceAllIn(current, m => m.group(1))
  }
  */
  val noDoubleWhitespace = current
  noDoubleWhitespace
}

  

>     filterSource: (source: String)String

In [ ]:
import java.util.regex.Matcher;
import java.io.{PrintWriter, StringWriter}

def getStackTraceAsString(t: Throwable) = {
    val sw = new StringWriter
    t.printStackTrace(new PrintWriter(sw))
    sw.toString
}

/**
  * Represents a parsed Wikipedia page from the Wikipedia XML dump
  *
  * https://en.wikipedia.org/wiki/Wikipedia:Database_download
  * https://meta.wikimedia.org/wiki/Data_dump_torrents#enwiki
  *
  * @param title Title of the current page
  * @param text Text of the current page including markup
  * @param isCategory Is the page a category page, not perfectly accurate
  * @param isFile Is the page a file page, not perfectly accurate
  * @param isTemplate Is the page a template page, not perfectly accurate
  */
case class Page(title: String, text: String, status: String, space: String) // isCategory: Boolean , isFile: Boolean, isTemplate: Boolean)

def filterSource(source: String) : Page = {
  var current = source
  var regex = "none".r
  var out = ""
  var title = ""
  var status = ""
  var space = ""
  try {
    var tag = "title"
    regex = s"<$tag[^>]*>((.*:)?.*)</$tag>".r
    var m = regex.findAllIn(current)
    title = m.group(1)
    space = m.group(2)

    tag = "text"
    regex = s"(?s)<$tag[^>]*>(.*?)</$tag>".r
    current = regex.findAllIn(current).group(1)
    val noXML = current
    
    // There is an &amp;nbsp; where the &amp; provides the & in &nbsp;
    // I should therefore replace &amp; first
    val noAmp = raw"&amp;".r.replaceAllIn(current, "&")
    current = noAmp

    // Loop to remove some specific sections
    val sections = Seq("Se även", "Referenser", "Noter", "Källor", "Externa länkar")
    for (section <- sections) {
      val regexp = s"[=]+ *$section *[=]+".r
      if (regexp.findAllIn(current).hasNext)
      {
        val a = regexp.findAllIn(current).start
        val pre = current.substring(0, a)
        var post = current.substring(a, current.length)
        post = regexp.replaceAllIn(post, "")
        if ("^=".r.findAllIn(post).hasNext) {
          val b = "^=".r.findAllIn(post).start
          post = post.substring(b, post.length)
        }
        else {
          post = ""        
        }
        current = pre + post
      }
    }

    val lowerCase = current.toLowerCase  

    val noTags = raw"<[^<>]*>".r.replaceAllIn(lowerCase, "")

    // A loop to remove innermost curly braces, then outer such things.
    val curly = raw"\{[^\{\}]*\}".r
    current = noTags
    while (curly.findAllIn(current).hasNext)
    {
       current = curly.replaceAllIn(current, "")
    }
    val noCurly = current

    // val noNumbers = raw"\d".r.replaceAllIn(noCurly, "")

    // Unnamed internal links appear in figure captions, confusing later matching
    val noUnnamedInternalLinks = raw"\[\[([^\|\]]*)\]\]".r.replaceAllIn(noCurly, m => Matcher.quoteReplacement(m.group(1)))

    // Loop to remove internal links and their properties
    current = noUnnamedInternalLinks
    val regexp = raw"\[\[[^\]\[]*\|([^\[\]]*)\]\]".r 
    while (regexp.findAllIn(current).hasNext) {
      val len = current.length
      current = regexp.replaceAllIn(current, m => Matcher.quoteReplacement(m.group(1)))
      assert (current.length < len)
    }
    val noNamedInternalLinks = current

    val noInternalLinks = raw"\[\[([^\]]*)\]\]".r.replaceAllIn(noNamedInternalLinks, m => Matcher.quoteReplacement(m.group(1)))
    current = noInternalLinks
    val noNamedExternalLinks = raw"\[[^ \[]*\ ([^\]]*)\]".r.replaceAllIn(noInternalLinks, m => Matcher.quoteReplacement(m.group(1)))
    current = noNamedExternalLinks
    val noExternalLinks = raw"\[([^\]]*)\]".r.replaceAllIn(noNamedExternalLinks, m => Matcher.quoteReplacement(m.group(1)))
    current = noExternalLinks

    val nbsp = raw"&nbsp;".r.replaceAllIn(noExternalLinks, " ")
    val noQuotes = raw"&quot;".r.replaceAllIn(nbsp, "")
    current = noQuotes
    val noQuoteChars = raw"''+".r.replaceAllIn(noQuotes, "")
    // Loop to remove double whitespace characters
    current = noQuoteChars
    /*
    val doubleWhitespace = raw"\s(\s)".r
    while (doubleWhitespace.findAllIn(current).hasNext) {
      current = doubleWhitespace.replaceAllIn(current, m => Matcher.quoteReplacement(m.group(1)))
    }
    */
    val noDoubleWhitespace = current
  }
  catch {
    case anyException: Throwable => {out += s"**EXCEPTION**\n$regex\n"; status += getStackTraceAsString(anyException) + "\n"}
  }
  finally {
    out += current
  }
  return Page(title,out,status,space)
}


  

>     import java.util.regex.Matcher
>     import java.io.{PrintWriter, StringWriter}
>     getStackTraceAsString: (t: Throwable)String
>     defined class Page
>     filterSource: (source: String)Page

In [ ]:
val corpus = corpus_dump.map({ case (id, source) => (id, filterSource(source))}).cache() // let's cache


  

>     corpus: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[13] at map at command-738986526041628:1

In [ ]:
corpus.take(5)

  

>     res6: Array[(Long, Page)] =
>     Array((10671,Page(Amager,
>
>
>
>     amager är en dansk ö i öresund. öns norra och västra delar tillhör köpenhamn, medan övriga delar upptas av tårnby kommun och dragørs kommun.
>     amager har en yta på 96,29 km² och befolkningen uppgår till 196 047 personer (1/1 2018). på den östra delen av ön finns kastrups flygplats. amager har också flera militära anläggningar (t.ex. kastrup fort och dragör fort), strandparksområdena amager strandpark och kastrup strandpark, flera restauranger i världsklass (t.ex. noma), ett stort naturområde (amager fælled), ett av nordens största köpcentra (field's), och mycket annat. amager förkortas ofta ama'r för att återspegla det danska uttalet.
>
>     amager är delvis en konstgjord ö, delvis en naturlig sådan. t.ex. så anlades en konstgjord ö i anslutning till det redan existerande amager strandpark, när denna rustades upp väsentligen 2005. ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området kalvebod fælled (vestamager). ön började uppodlas i större skala under 1500-talets första hälft då kung kristian ii bjöd in nederländska bönder till att odla upp ön för kronans räkning. flertalet slog sig ned i fiskeläget dragør där många än idag har namn med nederländskt ursprung som neels och tønnes.
>
>     den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i sundbyöster ut mot kusten. under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i vestamager som idag innefattar ett stort naturområde endast några kilometer från köpenhamns centrum. under senare delen av 1900-talet har flygplatsen kastrup som ligger på ön expanderat kraftigt och i samband med byggandet av öresundsförbindelsen började även ett stort område byggas som kallas för ørestad.
>
>     ön, som är förbunden med övriga köpenhamn genom flera broar, järnväg och sedan 2017 även metro, har under de senaste 10-20 åren förändrats mycket. dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena örestad, islands brygge och amager strand, och dels så har amagers närhet till köpenhamn raderat dess tidigare rykte som en industriell förstad till köpenhamn. idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. amager strand har förändrat bilden av ön. idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     ön återfick i samband med öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. en motorväg byggdes samtidigt över öns mellersta del. köpenhamns metro går till kastrup och vestamager med ett antal stationer på ön. kring metro- och regionaltågsstationen ørestad har ett stort kontors-, handels- och bostadsområde uppförts under 00-talet. detta sträcker sig från bella center i norr till metrostationen vestamager i söder. även om området inte ligger vid kusten har ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. vid islands brygge finns en äkta sjöstad.
>
>     före öresundsförbindelsens uppförande hade småstaden dragør en färjeförbindelse med limhamn i södra malmö. på grund av denna fanns i dragør tidigare en betydande gränshandel. denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     öns västra del, som utvanns från öresund under 1920- och 30-talen, är med undantag för bella center, ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     längst i söder finns den flerhundraåriga skogen kongelunden. på öns östra del, vid öresundskusten, finns amager strandpark med en populär sandstrand. området har omgestaltats, med en konstgjord ö och en lagun innanför. nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     amager är den tätast befolkade ön i danmark. den näst mest tätbefolkade är thurø.
>
>     ,,null)), (52187,Page(Afrika,
>
>     afrika är jordens näst största kontinent (efter eurasien) och även jordens näst största världsdel efter asien, både vad gäller areal och folkmängd. med världsdelens öar inräknade mäter afrika 30 244 050 km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. omkring 22 miljoner km² av dessa ligger i tropikerna vilket gör den afrikanska kontinenten till världens varmaste kontinent. i afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. dess längd i nordlig-sydlig riktning är omkring 8 000 km och dess största bredd omkring 7 800 km.
>
>     == etymologi ==
>     ordet afrikas etymologiska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika epoker.
>
>     i äldre tider kallades allt land väster om egypten för libyen. enligt den grekiska historieskrivaren herodotos berättelse seglade feniciska sjömän på uppdrag av den egyptiska kungen neko runt libyen, det vill säga kontinenten afrika redan omkring 600 f.kr.
>
>     när romarna först använde termen africa terra avsåg de före andra puniska kriget endast trakten kring karthago, det vill säga motsvarande ungefär dagens tunisien där själva staden karthago låg samt stora delar av libyen och algeriet. romarna uppfattade först egypten som en del av asien, och det var först geografen ptolemaios som drog gränsen vid suez. under kejsartiden kom africa att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. namnet ska ha avsett landet där afri (singularis: afer) bodde. afer avsåg namnet på ett folk som kan motsvara dagens berber. vikingarna under 800- och 900-talen kallade norra afrika för blåland.
>
>     == geografi ==
>
>     geologiskt är afrika en mycket gammal kontinent, i huvudsak uppbyggd av urberg. under jordens forntid var den en del av jättekontinenten gondwanaland. denna gled isär under juraperioden och kritaperioden för 100-150 miljoner år sedan, och så fick afrika sin nuvarande form.
>
>     inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050 km² inklusive öarna. afrika åtskiljs från europa av medelhavet och förenas med asien i sitt nordöstra hörn genom näset vid suez, omkring 130 km i bredd. omkring 8 000 km skiljer den nordligaste punkten, ras ben sakka i tunisien, från dess sydligaste, kap agulhas i sydafrika. från kap verde, dess västligaste punkt, till ras hafun i somalia är det omkring 7 400 km. afrikas kuster mäter omkring 28 000 km (1 km kust på 1 057 km²) varav medelhavet står för 5 400 km, atlanten 11 000 km, indiska oceanen 8 700 km och röda havet 2 900 km, och avsaknaden av djupare bukter och betydande halvöar blir uppenbar om man jämför med europas 9 700 000 km² som omgärdas av 32 000 km kust (1 km kust på 278 km² land), eller för det fjordrika norge, där motsvarande siffra 385 000 km² land och en kuststräcka på 20 000 km (1 km kust per 19 km² land).
>
>     afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. jämfört med de andra kontinenterna utmärker sig afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. landområdena under 180 m ö.h. upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i asien, utgörs av isolerade berg och bergskedjor. generellt ligger de högre platåerna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid röda havets mitt till kongoflodens mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. man kan således dela upp afrika i fyra övergripande landområden:
>     # slättlanden vid kusterna – ofta med mangroveträsk insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # atlasbergen i norr som ligger isolerat av saharaöknen som bitvis ligger under havsnivån.
>     # de södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # de nordliga och västra slättlanden, inklusive sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     skog täcker cirka en femtedel av afrikas yta.
>
>     hela afrika har ett tropiskt eller subtropiskt klimat.
>
>     === fauna ===
>
>
>
>     afrika är huvudsakligen känt för den öppna savannen med väldiga hjordar av hovdjur som afrikansk buffel, gnuer, antiloper, giraffer och zebror. dessa jagas av stora rovdjur som lejon, hyenor och geparder. den afrikanska djungeln befolkas av ormar och primater samt av vattenlevande djur som krokodiler och groddjur. afrika har även det största antalet arter som kan räknas till megafaunan, bland annat afrikanska elefanter, trubbnoshörning och flodhäst, på grund av att kontinenten bara i mindre utsträckning var påverkad av det massutdöende som skedde under pleistocen.
>
>     == historia ==
>
>     den första mänskliga migrationen, från kontinenten men även inom afrikas kuster, kan spåras lingvistiskt, kulturellt och genom datoranalyser av genetiskt material. någonstans i östafrika tror dagens forskare att de första hominider uppkom. i etiopien har man hittat ett två miljoner år gammalt skelett som uppkallades lucy, den första kända mänskliga primaten.
>
>     === tidigare civilisationer och handel ===
>     några av de mest kända afrikanska civilisationerna. civilisationer från olika tidsepoker visas parallellt.lingvistiska spår tyder på att bantufolk emigrerade från västafrika in i khoisanfolkets (de som tidigare kallades bushmän och hottentotter) område och undanträngde dem. bantufolken använde en uppsättning grödor, bland annat kassava och jams, som var mer lämpade för jordbruk i det tropiska afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. bantufolkens traditionella bosättningsområde sträcker sig från saharaöknen till de tempererade regionerna i söder. historiskt har deras vapen varit spjut, sköld, pil och båge. det för khoisanspråkens karaktäristiska klickljuden återfinns i nguni- och bantuspråk som till exempel xhosa och zulu. khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra afrika i länderna kring kalahariöknen.
>
>     afrika söder om sahara har bestått olika stater och riken, mindre stadsstater samt stammar och klaner. det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska civilisationerna. det fanns även statsbildningar i nubien, ghana, timbuktu, kanem-bornu och i nuvarande zimbabwe.
>
>     nordafrika kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds islam och den arabiska kulturen genom sahara och vidare söderut. arabiska och persiska handelsmän spred likaså islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till java, därmed den etniska, kulturella och språkliga mångfalden på östafrikas kust, madagaskar och komorerna.
>
>     === arabvärlden och europa utforskar kontinenten ===
>     det arabiska intresset för afrika går långt tillbaka, men handel med slavar från centra som lamu och zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.kr. de miljoner slavar som exporterades till arabvärlden och österut under den arabiska slavhandeln finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel sudan.&lt;ref&gt;human rights watch bbc world&lt;/ref&gt;
>
>     under 1300-talet började vissa europeiska handelsexpeditioner få kontakt med nord- och västafrika.
>
>     från 1500-talets mitt fram till förbudet mot slavhandel i storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst väst- och centralafrika. flera miljoner afrikaner såldes på detta vis som slavar till främst de europeiska kolonierna i amerika.
>
>     tidiga kolonisatörer var araber, portugiser och spanjorer. därefter koloniserades kontinenten av framförallt av storbritannien, frankrike, spanien, italien, belgien, nederländerna, tyskland och portugal.
>
>     === koloniseringen ===
>
>     politisk karta över afrika från 1890. från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av afrika. portugiserna grundade flera kolonier i västra afrika och vid nuvarande moçambique. i mitten av 1800-talet leddes flera expeditioner in i de centrala delarna av afrika, bland annat av livingstone och speke, men också den svenske upptäcktsresanden andersson. denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. kung leopolds rovdrift på sin privata koloni kongostaten är ökänd och kostade miljontals afrikanska liv.
>
>     i afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket egypten, republiken liberia och konungariket abessinien. tanger bildade med sitt område en internationell zon. för övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;&lt;/ref&gt;
>
>     flertalet kolonier erhöll självständighet under slutet av 1950-talet och början av 1960-talet. avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i rhodesia (nuvarande zimbabwe), rwanda, uganda, nigeria m.fl. stater.
>
>     == politik ==
>
>     de blå länderna är demokratier enligt tankesmedjan freedom house.
>     det postkoloniala afrikas politik har i hög grad präglats av diktatur, statskupper och brist på mänskliga rättigheter.
>
>     nationerna i afrika har ofta visat sig vara instabila stater som efter självständigheten hemsökts av korruption, våldsamma statskupper, despotism och militärdiktaturer. under perioden 1960-1989 genomfördes över 70 kupper och 13 mord på presidenter. under det kalla kriget kämpade usa och sovjetunionen om makten i afrika, och tillsammans med länder som frankrike och kuba, stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. enligt en spridd uppfattning beror många militära konflikter om territorier och gränser under afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter kongokonferensen, men detta kan inte visas eftersom något jämförelseområde inte finns. konflikter i afrika har ofta drivits av tillgången till kontinentens rika råvarutillgångar – diamanter och metaller som guld, platina, uran och koppar.
>
>     korruption och havererade politiska program har lett till utbredd undernäring och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. sanitetsproblemen har också orsakat en akut spridning av bland annat hiv-viruset som leder till sjukdomen aids.
>
>     trots nöd och umbäranden finns vissa positiva tecken – demokratiskt valda regeringar växer i antal även om de inte utgör någon majoritet ännu. genom påtryckningar från till exempel internationella valutafonden (imf) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. om man beaktar imf:s katastrofala misslyckanden i länder som argentina och filippinerna är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad afrika faktiskt behöver.
>
>     de afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. hittills har dock detta inte lyckats särskilt väl. under inbördeskriget i kongo var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     de politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. organisationen för afrikansk enighet (oau 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället afrikanska unionen, som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i darfur-provinsen i sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. en avgörande skillnad mellan såväl oau som au å ena sidan och eu å den andra, är att eu startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. oau och au har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. på så sätt liknar au mer fn än eu.
>
>     == ekonomi ==
>
>     jordbrukare på en jamsodling i kamerun
>     afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater (juli 2005). afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. på fn:s human development report från 2003 (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i afrika.
>
>     det finns dock länder som lyckas bra ekonomiskt, bland andra botswana, ghana och sydafrika. etiopien har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat guld och diamanter och ett väl fungerande rättsväsen. det finns i sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. det finns även andra afrikanska länder, exempelvis egypten, som länge haft en väl fungerande handelssektor och ekonomi.
>
>     kontinentens kust mot atlanten återfinns stora fyndigheter av olja. i nigeria finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     afrikas jordbruk sysselsätter drygt hälften av afrikas befolkning och är den viktigaste näringen. cirka 6 % (ungefär 1 814 600 km²) av världsdelens yta är uppodlad, mest i savannområdena. i stora delar av de brukningsbara områdena är jordens bördighet ganska låg. de bästa jordarna återfinns i begränsade höglandsområden som i till exempel kamerun och östafrika.
>
>     == demografi ==
>
>     ===befolkning===
>     människan, homo sapiens sapiens, antas härstamma från afrika och spred sig härifrån till de omgivande kontinenterna. den svarta befolkningen söder om sahara tillhör den ursprungliga befolkningen som hela tiden levat i afrika.
>
>     norr om sahara präglar samröret med övriga medelhavsområdet och inte minst sydvästra asien både befolkning, språk och kultur.
>
>     på ön madagaskar, som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från indonesien. under kolonialtiden skedde inflyttning från europa och asien till södra afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga födelsetal i kombination med sjunkande dödstal med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. även om befolkningstätheten varierar kraftigt mellan olika regioner är afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. även här är variationerna stora, och i norra och östra afrika finns en lång urban tradition.
>
>     === språk ===
>
>     språkfamiljer i afrika
>     uppskattningsvis finns det ungefär 1800 talade språk i afrika idag. vissa av dessa, till exempel swahili, hausa, och yoruba, talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. afrika har även många olika teckenspråk. de flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna afroasiatiska språk, nilo-sahariska språk, niger-kongospråk eller khoisanspråk.
>     med några få nämnvärda undantag i östra afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. till exempel används i ett stort antal afrikanska länder franska eller engelska som administrativt språk för stat, handel, utbildning och media. arabiska, portugisiska, afrikaans och malagassiska är andra exempel på språk med ursprung utanför afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === religion ===
>
>     kristendom och islam är de största religionerna i afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. ett mindre antal afrikaner är hinduer, eller har tro från judisk tradition. exempel på afrikanska folk som tillhör judendomen är beta israel, lembafolk och abayudaya i östra uganda. sydafrika och angola har över 55% kristna katoliker.
>
>     == kultur ==
>
>     afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. den mest konventionella distinktionen är den mellan området söder om sahara och området norr om öknen. de nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. en av de största kulturella identiteterna är den som innefattar alla folk som talar något bantuspråk.
>
>     det går också att göra en indelning i gamla franska västafrika å ena sidan, och resten av afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra afrika) å andra sidan. en annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     afrikansk konst och afrikansk arkitektur reflekterar de afrikanska kulturernas diversitet. de äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av nassariusskal, dessa hittades i blombosgrottan. pyramiderna i giza var världens högsta byggnader under 4 000 år. klippkyrkorna i lalibela i etiopien, bland vilka bete giyorgis (st. göranskyrkan) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == regioner och territorier ==
>     regioner i afrika:
>
>
>
>
>
>
>
>     ,,null)), (55447,Page(Amerika,
>     karta som visar nord- och sydamerika, tillsammans benämnda amerika
>     amerika (engelska: the americas) består av landmassorna mellan atlanten och stilla havet på västra halvklotet, namngivna efter den italienske upptäcktsresanden amerigo vespucci. amerika uppfattas vanligen idag som två världsdelar, nordamerika och sydamerika.&lt;ref&gt;store norske leksikon/ amerika&lt;/ref&gt; amerika indelas geografiskt i nordamerika (med centralamerika) och sydamerika. grönland ingår geografiskt i nordamerika, men räknas politiskt och kulturellt till europa. kulturgeografiskt avgränsas också ofta latinamerika, då inklusive både central- och sydamerika samt mexiko, och angloamerika, som är usa och kanada. områdets totala landyta är ungefär 42 miljoner kvadratkilometer och är således något mindre än asiens.
>
>     ofta används amerika och amerikansk för enbart landet usa, amerikas förenta stater, vilket medför en förväxlingsrisk med dubbelkontinenten som helhet men är jämförbart med hur andra landsnamn förkortas i dagligt tal (exempelvis mexiko, vars fullständiga namn översatt till svenska är mexikos förenta stater). på engelska, portugisiska och spanska kommer man ofta runt problemet genom att de två världsdelarna kallas the americas respektive las américas (två amerikorna, ett uttryck som dock aldrig har etablerats på svenska).
>
>     == namnet ==
>     amerika namngavs efter amerigo vespucci, som var den första som insåg att amerika var en kontinent som nyligen kommit till kännedom för européer, som inte var östindien. namnet används 1507 för första gången i en världskarta som skapades av den tyske geografen martin waldseemüller och beskriver där sydamerika. 1511 omnämnes det i ett engelskt skådespel och 1516 används det av leonardo da vinci på hans världskarta och vann alltså snabb utbredning. spanjorerna brukade i början ha andra namn. de kallade landet för den nya världen eller las indias occidentales (västra indien). för sydamerika behöll de under hela 1500-talet namnen peruana och brasilia. först på 1600-talet blev namnet amerika allmänt antaget.
>
>     == historia ==
>     * se amerikas historia
>
>     ,,null)), (58373,Page(Abba (olika betydelser),
>     abba kan avse:
>
>     * abba – en tidigare svensk popgrupp
>     ** abba (musikalbum) – ett album av gruppen
>     * abba seafood – ett fiskkonservföretag
>     * abba (teologi) – ett arameiskt ord
>     * abba (hederstitel) – en hederstitel och förtroligt tilltal
>
>     ==geografi==
>
>     &lt;!--notera: om platser läggs till, tas bort eller ordningen på platserna ändras, bör också mallen kartposition under samma rubrik korrigeras för att kartan ska förbli rättvisande.--&gt;
>
>     === australien ===
>
>     * abba river
>
>     === centralafrikanska republiken ===
>
>     # abba (subprefektur), préfecture de la nana-mambéré, &lt;!--geonames id 9143482--&gt;
>
>     === etiopien ===
>
>     # abba (periodiskt vattendrag), amhara, &lt;!--geonames id 345835--&gt;
>     # ābaya hāyk', sjö, &lt;!--geonames id 345844--&gt;
>     # abba (skog), oromia, &lt;!--geonames id 6642287--&gt;
>
>
>     kategori:robotskapade förgreningar 2016-01
>     kategori:robotskapade centralafrikanska republikenförgreningar
>     kategori:robotskapade etiopienförgreningar,,null)), (64397,Page(Abbekås,&lt;!-- slut på mall:ortsfakta sverige --&gt;
>
>     abbekås (uttalas abbekå-s)&lt;ref&gt;runeberg.org: svenska ortnamn med uttalsuppgifter sida 4&lt;/ref&gt; är en tätort i skivarps distrikt (skivarps socken) på söderslätt i skurups kommun.
>
>     == historia ==
>     abbekås var från början ett fiskeläge på skånes sydkust mellan ystad och trelleborg, omtalad redan på medeltiden. onsdagen den 13 november 1872 förstördes stora delar av byn av en stormflod kallad backafloden, framkallad av en orkan som drog in över sydkusten från nordost. 34 hus förstördes, alla fiskeredskap, båtar och befolkningens vinterförråd. vattenståndet ökade till 3,6 meter över det normala, vilket fortfarande är svenskt rekord. redan året därpå inleddes med hjälp av statsbidrag arbetet med en riktig hamn, och fisket och kusthandeln kunde åter komma igång. en ny storhetstid för fisket inträffade under andra världskriget.
>
>     2011 renoverades pirar och kajer, och inloppet gjordes om för att öka säkerheten i yttre hamnbassängen.
>
>     === befolkningsutveckling ===
>
>
>     ==namnet==
>     namnet skrevs 1536 abbekassz. efterleden är kås, 'båtplats, mindre hamn'. förleden innehåller förmodligen mansnamnet abbi.&lt;ref&gt;&lt;/ref&gt;.
>
>     ==samhället==
>     abbekås hamn består av en yttre och en inre hamnbassäng, där den inre används av fiskarna och den yttre är till för fritidsseglare och gästbåtar. här finns såväl hamnkrog och hotell som ett modernt tvätt/dusch/toaletthus för besökare. längre upp i byn finns ytterligare en restaurang och en livsmedelsaffär, ett konstgalleri och två bed&breakfast-ställen. strax öster om byn ligger abbekås golfklubb.
>
>     ==abbekås i musiken==
>     abbekås är omsjunget av edvard persson i sången abbekåsagåsen joakim.
>
>     == kända profiler ==
>     *lorens brolin, spelman
>
>     == bilder ==
>     &lt;gallery&gt;
>     abbekås 2.jpg|abbekås hamn
>     hamnen abbekås.jpg|abbekås hamn
>     magasinet abbekås.jpg|magasinet vid abbekås hamn
>     coast line abbekås.jpg|strandlinjen väster om abbekås hamn
>     abbekås 3.jpg|abbekåsagåsen joakim
>     &lt;/gallery&gt;
>
>     ==omgivande orter==
>
>
>     ,,null)))

  

Count the number of articles

Swedish Wikipedia has approximately 3 million articles, so not sure why
the count says 6 million

In [ ]:
// Looks like valida articles can have : in name. They are only few, so no strucural problem hiding there.
val articles = corpus.filter({ case (id, page) => page.space != null})
articles.take(10)

In [ ]:
articles.count()

In [ ]:
corpus.count()

  

Find problem with filtering
---------------------------

In [ ]:
// Some random test cases
println(filterSource("""test <title>titles</title> mer <text bytes="7485" xml:space="preserve">info
</text>"""))
println(filterSource("""test <title>space:nospace</title> mer <text bytes="7485" xml:space="preserve">info
</text>"""))
println(filterSource("test"))
println(filterSource("test [[| asf|]] [[nada|]] [[asf|show]] "))
println(filterSource("fail [[] {"))
println(filterSource("fail [[|]] [[]] {}"))
println(filterSource("fail [[dollar|$]]"))


  

>     Page(titles,info
>     ,,null)
>     Page(space:nospace,info
>     ,,space:)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     test,java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d55.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:6)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:78)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:80)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:82)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:84)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:86)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:88)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw.<init>(command-738986526041635:90)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw.<init>(command-738986526041635:92)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw.<init>(command-738986526041635:94)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read.<init>(command-738986526041635:96)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<init>(command-738986526041635:100)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<clinit>(command-738986526041635)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print$lzycompute(<notebook>:7)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print(<notebook>:6)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:233)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:233)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:223)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     test [[| asf|]] [[nada|]] [[asf|show]] ,java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d55.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:7)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:78)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:80)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:82)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:84)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:86)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:88)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw.<init>(command-738986526041635:90)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw.<init>(command-738986526041635:92)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw.<init>(command-738986526041635:94)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read.<init>(command-738986526041635:96)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<init>(command-738986526041635:100)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<clinit>(command-738986526041635)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print$lzycompute(<notebook>:7)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print(<notebook>:6)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:233)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:233)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:223)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     fail [[] {,java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d55.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:8)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:78)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:80)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:82)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:84)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:86)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:88)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw.<init>(command-738986526041635:90)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw.<init>(command-738986526041635:92)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw.<init>(command-738986526041635:94)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read.<init>(command-738986526041635:96)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<init>(command-738986526041635:100)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<clinit>(command-738986526041635)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print$lzycompute(<notebook>:7)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print(<notebook>:6)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:233)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:233)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:223)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     fail [[|]] [[]] {},java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d55.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:9)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:78)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:80)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:82)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:84)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:86)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:88)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw.<init>(command-738986526041635:90)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw.<init>(command-738986526041635:92)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw.<init>(command-738986526041635:94)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read.<init>(command-738986526041635:96)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<init>(command-738986526041635:100)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<clinit>(command-738986526041635)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print$lzycompute(<notebook>:7)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print(<notebook>:6)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:233)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:233)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:223)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     fail [[dollar|$]],java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d55.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:10)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:78)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:80)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:82)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:84)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:86)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:88)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw$$iw.<init>(command-738986526041635:90)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw$$iw.<init>(command-738986526041635:92)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$$iw.<init>(command-738986526041635:94)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read.<init>(command-738986526041635:96)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<init>(command-738986526041635:100)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$read$.<clinit>(command-738986526041635)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print$lzycompute(<notebook>:7)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval$.$print(<notebook>:6)
>     	at linef5fdecc3f5fa4fc389af4b3d2e18c16d63.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:233)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:233)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:223)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)

In [ ]:
val failed = corpus.filter({ case (id, source) => source.status != ""})
failed.take(10)

In [ ]:
failed.count()

In [ ]:
// Are the empty articles mistakes or reasonable? Looks reasonable.
val failed_empty = corpus.filter({ case (id, page) => page.text.length() < 20})
failed_empty.take(10)

  

>     failed_empty: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[1333] at filter at command-738986526041638:2
>     res9: Array[(Long, Page)] =
>     Array((239722707,Page(Lista över nationaldagar,
>
>
>
>     ,,null)), (371889681,Page(Lista över USA:s delstater och territorier efter yta,
>
>     ,,Lista över USA:)), (583039664,Page(Lista över fjäll på Färöarna,
>
>     färöarnas berg,,null)), (679739723,Page(Lista över öar efter folkmängd,
>
>
>     ,,null)), (739536420,Page(Lista över Wimbledonmästare i damsingel,
>     ==lista==
>
>
>     ,,null)), (752944879,Page(Lista över nationalsånger,
>
>
>     ,,null)), (783397284,Page(Lista över köpingar i Sverige,
>
>
>     ,,null)), (969318751,Page(Lista över Tysklands delstater efter yta,
>
>     delstater,,null)), (1188511201,Page(Lista över Greklands utrikesministrar,
>
>     ,,null)), (1198242590,Page(Lista över Schleswig-Holsteins ministerpresidenter,
>
>       ,,null)))

In [ ]:
val example = dump.filter({ case (id, source) => id == 2577428})
val fail_source = example.take(1)(0)._2

In [ ]:
val fail_state = filterSource(fail_source)

In [ ]:
import java.util.regex.Matcher;

val regexp = raw"\[\[[^\]\[]*\|([^\[\]]*)\]\]".r 
var current = fail_source
var count = 0
current = regexp.replaceAllIn(current, m => {println(m.group(0) + " ==> " + m.group(1)); Matcher.quoteReplacement(m.group(1))}) // )


  

Convert to DataFrame

In [ ]:
// Let's discard the smaller articles 
val recorpus = corpus.filter({ case (id, page) => page.text.length() >= 1000})

  

>     recorpus: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[15] at filter at command-2158711750343073:2

In [ ]:
// Convert RDD to DF with ID for every document 
val corpus_df = recorpus.map({case (wpid,item) => (item.title,item.text,wpid)}).toDF("title","corpus", "id")

  

>     corpus_df: org.apache.spark.sql.DataFrame = [title: string, corpus: string ... 1 more field]

  

Tokenization
============

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer

// Set params for RegexTokenizer
val tokenizer = new RegexTokenizer()
//.setPattern("[\\W_]+") // break at non-word character(s). \\W is [^A-Za-z0-9_]
.setPattern("[^a-zåäö]+") // break at non-word character(s). \\W is [^A-Za-z0-9_]
.setMinTokenLength(4) // Filter away tokens with length < 4
.setInputCol("corpus") // name of the input column
.setOutputCol("tokens") // name of the output column

// Tokenize document
val tokenized_df = tokenizer.transform(corpus_df)

  

>     import org.apache.spark.ml.feature.RegexTokenizer
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_fce97196568c, minTokenLength=4, gaps=true, pattern=[^a-zåäö]+, toLowercase=true
>     tokenized_df: org.apache.spark.sql.DataFrame = [title: string, corpus: string ... 2 more fields]

In [ ]:
display(tokenized_df.sample(0.001).select("tokens"))

  

Remove Stopwords
================

In [ ]:
val stopwords = sc.textFile(stopword_path).collect()

  

>     stopwords: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)

  

add custom stopwords

In [ ]:
val add_stopwords = Array("label", "note", "area", "unit", "type", "mark", "long", "right", "kartposition", "quot", "text", "title", "page", "timestamp", "revision", "name", "username", "sha1", "format", "coord", "left", "center", "align", "region", "nasa", "source", "mouth", "species", "highest", "style", "kategori", "http", "wikipedia", "referenser", "källor", "noter")

  

>     add_stopwords: Array[String] = Array(label, note, area, unit, type, mark, long, right, kartposition, quot, text, title, page, timestamp, revision, name, username, sha1, format, coord, left, center, align, region, nasa, source, mouth, species, highest, style, kategori, http, wikipedia, referenser, källor, noter)

In [ ]:
// Combine newly identified stopwords to our exising list of stopwords
val new_stopwords = stopwords.union(add_stopwords)

  

>     new_stopwords: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre, label, note, area, unit, type, mark, long, right, kartposition, quot, text, title, page, timestamp, revision, name, username, sha1, format, coord, left, center, align, region, nasa, source, mouth, species, highest, style, kategori, http, wikipedia, referenser, källor, noter)

  

Remove stopwords

In [ ]:
import org.apache.spark.ml.feature.StopWordsRemover

// Set params for StopWordsRemover
val remover = new StopWordsRemover()
.setStopWords(new_stopwords) // This parameter is optional
.setInputCol("tokens")
.setOutputCol("filtered")

// Create new DF with Stopwords removed
val filtered_df = remover.transform(tokenized_df)

  

>     import org.apache.spark.ml.feature.StopWordsRemover
>     remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_7d9ef67b1bb7, numStopWords=366, locale=en, caseSensitive=false
>     filtered_df: org.apache.spark.sql.DataFrame = [title: string, corpus: string ... 3 more fields]

  

Vector of token counts
======================

In [ ]:
import org.apache.spark.ml.feature.CountVectorizer

// Set params for CountVectorizer
val vectorizer = new CountVectorizer()
.setInputCol("filtered")
.setOutputCol("features")
.setVocabSize(10000) 
.setMinDF(5) // the minimum number of different documents a term must appear in to be included in the vocabulary.
.fit(filtered_df)

  

>     import org.apache.spark.ml.feature.CountVectorizer
>     vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVec_2af0b7f4b0d7, vocabularySize=10000

In [ ]:
// Create vector of token counts
val countVectors = vectorizer.transform(filtered_df).select("id", "features")

  

>     countVectors: org.apache.spark.sql.DataFrame = [id: bigint, features: vector]

In [ ]:
// see the first countVectors
countVectors.take(2)

  

>     res25: Array[org.apache.spark.sql.Row] = Array([10671,(10000,[32,46,50,56,58,67,85,86,88,94,103,104,106,107,111,119,122,123,127,129,148,153,158,159,162,164,173,177,186,187,190,198,200,228,233,247,254,272,276,279,280,299,314,375,377,411,465,476,519,531,565,591,593,594,632,648,714,734,755,801,837,855,883,886,888,917,949,1019,1055,1077,1097,1098,1124,1221,1240,1249,1295,1309,1399,1464,1484,1522,1555,1562,1744,1798,1821,1840,1848,1866,2034,2092,2130,2136,2197,2442,2451,2528,2598,2809,2937,3044,3328,3487,3571,3582,3626,3669,3689,3867,3871,3967,4277,4291,4608,4686,4794,4841,4850,5199,5327,5389,5431,5467,5483,5899,5948,5960,6158,6250,6258,6379,6399,6419,6454,6825,7455,8361,8750,8806,8810,9213,9645,9943],[1.0,1.0,7.0,2.0,1.0,4.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,2.0,4.0,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])], [52187,(10000,[0,1,3,6,9,14,17,22,28,29,32,37,41,42,45,50,56,57,58,60,61,63,67,73,75,77,85,87,88,89,91,94,102,103,104,106,107,111,113,116,119,123,124,126,127,129,134,136,138,139,142,147,148,151,155,157,158,159,162,164,165,169,171,175,179,180,181,186,189,190,198,199,200,202,205,208,211,212,217,218,219,225,231,233,234,248,249,256,263,265,267,274,276,278,280,286,287,290,294,295,296,297,299,303,304,314,319,326,328,329,334,338,340,344,346,351,354,375,378,383,396,397,401,403,406,407,408,410,416,421,423,427,429,433,435,439,441,454,455,456,459,475,476,482,489,495,497,508,519,550,565,578,591,593,595,597,598,601,611,612,617,632,639,642,644,650,659,661,688,689,690,693,695,700,706,713,714,722,724,727,731,732,735,746,757,762,764,766,768,769,771,775,777,781,784,790,803,808,813,817,822,823,830,837,840,842,846,858,869,871,874,875,880,890,906,911,923,925,953,958,976,979,982,986,995,1025,1028,1036,1050,1055,1058,1061,1062,1067,1072,1077,1083,1088,1092,1098,1103,1106,1110,1124,1128,1142,1143,1149,1153,1169,1172,1173,1178,1182,1191,1221,1232,1233,1234,1243,1249,1252,1253,1269,1284,1289,1295,1301,1307,1309,1329,1335,1348,1353,1355,1376,1387,1399,1415,1421,1424,1446,1467,1494,1496,1514,1515,1517,1527,1544,1549,1561,1563,1584,1587,1608,1620,1641,1646,1651,1741,1753,1755,1757,1761,1773,1774,1824,1828,1842,1847,1849,1871,1887,1912,1929,1949,1976,1979,1980,1986,1992,1994,2000,2002,2010,2022,2034,2069,2096,2111,2127,2129,2131,2136,2170,2187,2197,2201,2225,2236,2259,2276,2306,2324,2336,2384,2398,2415,2417,2424,2438,2472,2488,2499,2509,2541,2599,2601,2644,2657,2663,2715,2726,2728,2746,2753,2766,2784,2809,2820,2842,2894,2914,2918,2937,3000,3012,3027,3043,3044,3045,3091,3125,3155,3162,3189,3299,3345,3350,3369,3380,3451,3490,3493,3500,3501,3509,3520,3557,3569,3580,3626,3635,3661,3693,3715,3743,3749,3790,3799,3800,3865,3881,3892,3902,3915,3933,3935,3956,3977,3990,3992,4014,4049,4056,4082,4120,4137,4146,4184,4193,4203,4245,4314,4330,4341,4359,4369,4392,4429,4459,4470,4476,4513,4536,4550,4576,4580,4600,4611,4621,4623,4650,4711,4713,4776,4832,4896,4925,4929,4978,5001,5111,5142,5173,5192,5213,5249,5271,5294,5341,5373,5396,5409,5410,5470,5484,5489,5574,5581,5642,5661,5665,5713,5735,5738,5912,5958,6062,6072,6151,6216,6332,6333,6389,6444,6496,6528,6536,6539,6606,6609,6732,6799,6841,6846,7035,7038,7065,7069,7146,7210,7219,7228,7325,7403,7462,7560,7593,7623,7658,7677,7692,7697,7794,7815,7819,7829,7964,8021,8089,8114,8294,8323,8371,8406,8421,8454,8581,8587,8651,8697,8727,8732,8787,8817,8825,8903,8916,8922,8971,9003,9009,9070,9084,9086,9108,9118,9279,9317,9358,9365,9440,9449,9452,9468,9527,9529,9578,9696,9718,9733,9754,9838,9889,9940,9970,9983],[1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,8.0,6.0,1.0,5.0,1.0,1.0,2.0,3.0,1.0,3.0,1.0,2.0,2.0,6.0,1.0,11.0,2.0,1.0,2.0,7.0,5.0,6.0,3.0,1.0,1.0,5.0,2.0,4.0,2.0,3.0,1.0,2.0,12.0,1.0,2.0,1.0,1.0,6.0,2.0,1.0,1.0,1.0,2.0,3.0,2.0,2.0,4.0,1.0,1.0,4.0,3.0,1.0,5.0,2.0,4.0,4.0,1.0,3.0,1.0,1.0,2.0,11.0,1.0,1.0,1.0,1.0,1.0,5.0,2.0,1.0,1.0,4.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,4.0,4.0,3.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,10.0,1.0,3.0,1.0,1.0,1.0,4.0,1.0,1.0,3.0,3.0,6.0,6.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,10.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,5.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,44.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,6.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,7.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,16.0,3.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,6.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,10.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,6.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])])

In [ ]:
// Convert DF to RDD
import org.apache.spark.ml.linalg.Vector

val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }

  

>     import org.apache.spark.ml.linalg.Vector
>     lda_countVector: org.apache.spark.sql.Dataset[(Long, org.apache.spark.ml.linalg.Vector)] = [_1: bigint, _2: vector]

In [ ]:
// format: Array(id, (VocabSize, Array(indexedTokens), Array(Token Frequency)))
lda_countVector.take(1)

  

>     res27: Array[(Long, org.apache.spark.ml.linalg.Vector)] = Array((10671,(10000,[32,46,50,56,58,67,85,86,88,94,103,104,106,107,111,119,122,123,127,129,148,153,158,159,162,164,173,177,186,187,190,198,200,228,233,247,254,272,276,279,280,299,314,375,377,411,465,476,519,531,565,591,593,594,632,648,714,734,755,801,837,855,883,886,888,917,949,1019,1055,1077,1097,1098,1124,1221,1240,1249,1295,1309,1399,1464,1484,1522,1555,1562,1744,1798,1821,1840,1848,1866,2034,2092,2130,2136,2197,2442,2451,2528,2598,2809,2937,3044,3328,3487,3571,3582,3626,3669,3689,3867,3871,3967,4277,4291,4608,4686,4794,4841,4850,5199,5327,5389,5431,5467,5483,5899,5948,5960,6158,6250,6258,6379,6399,6419,6454,6825,7455,8361,8750,8806,8810,9213,9645,9943],[1.0,1.0,7.0,2.0,1.0,4.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,2.0,4.0,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])))

  

LDA
===

In [ ]:
val numTopics = 50 // how to set this?
val maxIterations = 20 // this probably has to be increased (from 3)
val maxTermsPerTopic = 5 // only to display?

  

>     numTopics: Int = 50
>     maxIterations: Int = 20
>     maxTermsPerTopic: Int = 5

In [ ]:
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}

// Set LDA params
val lda = new LDA()
.setOptimizer(new OnlineLDAOptimizer().setMiniBatchFraction(0.8))
.setK(numTopics)
.setMaxIterations(maxIterations)
.setDocConcentration(-1) // use default values
.setTopicConcentration(-1) // use default values

  

>     import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
>     lda: org.apache.spark.mllib.clustering.LDA = org.apache.spark.mllib.clustering.LDA@1be6eced

In [ ]:
// convert ML vectors into MLlib vectors
val lda_countVector_mllib = lda_countVector.map { case (id, vector) => (id, org.apache.spark.mllib.linalg.Vectors.fromML(vector)) }.rdd

// save this countvector for later
val lda_countVector_file = s"lda_countvector_${lang}_${freeze_date}"
println(lda_countVector_file)

lda_countVector_mllib.saveAsTextFile(lda_countVector_file);

  

>     lda_countvector_sv_20201208
>     lda_countVector_mllib: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[49] at rdd at command-738986526041666:2
>     lda_countVector_file: String = lda_countvector_sv_20201208

In [ ]:
// Run LDA
val ldaModel = lda.run(lda_countVector_mllib)

In [ ]:
val lda_file = s"lda_${lang}_${freeze_date}.model"
println(lda_file)

  

>     lda_sv_20201208.model
>     lda_file: String = lda_sv_20201208.model

In [ ]:
ldaModel.save(sc, dir + lda_file)  // sc is spark_context

In [ ]:
dbutils.fs.ls(dir)

  

>     res41: Seq[com.databricks.backend.daemon.dbutils.FileInfo] = WrappedArray(FileInfo(dbfs:/datasets/05_lda/enwiki-20201120-pages-articles.xml, enwiki-20201120-pages-articles.xml, 80561829658), FileInfo(dbfs:/datasets/05_lda/lda_sv_20201120.model/, lda_sv_20201120.model/, 0), FileInfo(dbfs:/datasets/05_lda/lda_sv_20201208.model/, lda_sv_20201208.model/, 0), FileInfo(dbfs:/datasets/05_lda/stoppord.csv, stoppord.csv, 1936))

  

Review topics
=============

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic)

  

>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(312, 146, 196, 905, 697),Array(0.03584527137263872, 0.02491996116865994, 0.02490628738653775, 0.022825068088324012, 0.017879608346786705)), (Array(0, 2, 3, 5, 4),Array(0.03226894586119219, 0.024409431158202783, 0.022132711657064968, 0.02148558030874581, 0.021356313771342332)), (Array(1204, 1064, 1999, 1606, 1398),Array(0.03223323621121513, 0.032140352784615744, 0.028628035554835788, 0.02572213421059921, 0.022714847469118508)), (Array(143, 183, 72, 694, 266),Array(0.04043182297527037, 0.036426713096318626, 0.03335972843491673, 0.03294952066003985, 0.0329078354521061)), (Array(300, 301, 336, 330, 404),Array(0.009358444119624937, 0.009218605994143599, 0.008569286231897809, 0.008494384319722029, 0.006593890980715514)), (Array(2, 645, 1041, 1513, 429),Array(0.034014980004080625, 0.03299031954197243, 0.01740713855328453, 0.016493036980131597, 0.016473530616266375)), (Array(85, 372, 346, 69, 532),Array(0.005202912736908954, 0.004980332650975352, 0.004956359036949585, 0.004107078219445283, 0.003994412233925799)), (Array(901, 510, 1279, 2568, 841),Array(0.02865565696236134, 0.01497731749999038, 0.012856303027619758, 0.011020796630618354, 0.01088948153793848)), (Array(0, 1, 3, 2, 4),Array(0.04217375386317798, 0.03443350635893168, 0.029432163834692823, 0.029164515781816613, 0.028263430039365416)), (Array(291, 165, 263, 216, 292),Array(0.021849080720731134, 0.02104582186322425, 0.019143573459580128, 0.01906342816207476, 0.017466187171276955)), (Array(323, 559, 250, 454, 2262),Array(0.10161487730061466, 0.04696683184763887, 0.03283890738943248, 0.01767809259682316, 0.016690717240719372)), (Array(31, 95, 71, 117, 44),Array(0.16173888357413663, 0.04772361521855838, 0.045209691062612106, 0.04064929862689341, 0.03671487993757405)), (Array(352, 204, 225, 557, 125),Array(0.02369681037340463, 0.020619560775043934, 0.01805647612510781, 0.015933941106456802, 0.013941192536265194)), (Array(166, 309, 534, 0, 337),Array(0.11064594620845476, 0.05967252584628175, 0.022062432102082228, 0.02161165367091075, 0.01971175642758329)), (Array(121, 51, 128, 95, 31),Array(0.12269798734719728, 0.11548170564697531, 0.1080413631122436, 0.05567056593344491, 0.03005264844658276)), (Array(213, 204, 173, 403, 507),Array(0.013686188564871101, 0.010069959578121875, 0.009834209338061728, 0.009684304135010991, 0.006769956157331183)), (Array(280, 1797, 313, 824, 1574),Array(0.03260243910309328, 0.013986789615136346, 0.0121153524586439, 0.011625755878276552, 0.009249631189674428)), (Array(999, 0, 786, 3834, 2),Array(0.07027290860872792, 0.01886574550707347, 0.016179163575668384, 0.015799019016393998, 0.014780885004736585)), (Array(245, 7, 1700, 1479, 3),Array(0.034502823564636326, 0.02864792614705575, 0.025037979653382458, 0.024953856343253075, 0.024907385096831488)), (Array(1, 44, 0, 4, 2),Array(0.04096386788951671, 0.0345224866846443, 0.02963580174185564, 0.028115313448807144, 0.027858463237621884)), (Array(503, 196, 1193, 11, 101),Array(0.025398611367203856, 0.02289216003761088, 0.01846838158518249, 0.0176316418497597, 0.013980852803650965)), (Array(147, 50, 528, 151, 6),Array(0.023384488081661554, 0.014684568820705514, 0.008041407554453981, 0.005899803204835226, 0.005315004050335149)), (Array(31, 51, 92, 243, 117),Array(0.11557194603913014, 0.10396838795255336, 0.07664488602777711, 0.044735247147267086, 0.032041160696272615)), (Array(394, 125, 178, 251, 144),Array(0.015555705383864719, 0.014550019239920886, 0.008732132798153132, 0.007788712779924475, 0.006967143392245979)), (Array(910, 3004, 2203, 6220, 5258),Array(0.12364353938874159, 0.041319606928897926, 0.027513514818681497, 0.02290853181360536, 0.02141349760145409)), (Array(86, 146, 759, 1047, 1682),Array(0.10290042230605351, 0.05728034134114776, 0.01811165620130087, 0.010530347777947284, 0.007339475945269198)), (Array(1078, 1665, 259, 1473, 2021),Array(0.0200670569291999, 0.016284072132314146, 0.0135712033444977, 0.013479451859270629, 0.01336898859104554)), (Array(477, 564, 433, 1543, 2353),Array(0.09684045533161303, 0.09188909300387012, 0.03486418022216705, 0.023982054573107807, 0.016686082802654822)), (Array(50, 2, 111, 382, 104),Array(0.012694136012504064, 0.010889970029104822, 0.00882690394783239, 0.008722786999135347, 0.006863758992484095)), (Array(1255, 0, 1, 4331, 2),Array(0.04844274548131795, 0.019001409457080374, 0.01675423618341787, 0.014601997139814487, 0.014327415704946335)), (Array(84, 70, 69, 79, 131),Array(0.015135856576968138, 0.01214761100144125, 0.011952592249192522, 0.010682496369850301, 0.006188853427958379)), (Array(1248, 0, 1518, 1594, 2426),Array(0.03048136070857475, 0.022898886518763237, 0.020878561387803588, 0.020243918240258764, 0.018246897035616844)), (Array(185, 310, 284, 264, 178),Array(0.020781490320475385, 0.007085303531532155, 0.0064215730197610225, 0.006262473771143063, 0.006106569562524211)), (Array(271, 302, 629, 0, 712),Array(0.03607418632541323, 0.035660992757308906, 0.025960037172071338, 0.024025464515943364, 0.02315327816207366)), (Array(91, 136, 50, 127, 107),Array(0.009401336618340807, 0.0074417216581975035, 0.006309347400350304, 0.00599903713081023, 0.0059618095032171435)), (Array(386, 909, 2958, 0, 365),Array(0.152053730570257, 0.04342233664608969, 0.02784126696431022, 0.017052063582336168, 0.016573201370297795)), (Array(221, 92, 0, 133, 156),Array(0.029949250854912395, 0.02401769656994233, 0.023552175762361592, 0.023491880302950876, 0.023446760999741703)), (Array(152, 191, 327, 161, 335),Array(0.018247050041365145, 0.01699606742987092, 0.010585508000319446, 0.010541376807987448, 0.010174683683362207)), (Array(84, 389, 695, 254, 816),Array(0.023463309330417906, 0.020588688992590618, 0.01531361753508411, 0.013601805832605934, 0.01339290590857053)), (Array(916, 0, 1166, 1, 981),Array(0.029847680557851572, 0.02478222302763145, 0.022600156012697558, 0.02176298711203436, 0.020395277368909562)), (Array(0, 1, 2, 3, 4),Array(0.04674932020852935, 0.0357544398258386, 0.03514437047605392, 0.03080664018465887, 0.030751659585869125)), (Array(50, 197, 188, 143, 180),Array(0.020927168859939324, 0.016978453949576977, 0.016398363668156845, 0.009234697415750651, 0.006700990866659774)), (Array(660, 469, 709, 266, 456),Array(0.009200084245357574, 0.006781072683076032, 0.006751636661360212, 0.006591367471565553, 0.006211111760647785)), (Array(487, 499, 847, 542, 1603),Array(0.017632507535482355, 0.016531267519608497, 0.011765155277036494, 0.009193469609380736, 0.008789598592035592)), (Array(79, 70, 144, 206, 242),Array(0.027812732588726418, 0.020404036898973906, 0.016893938539467617, 0.015654966665244758, 0.010136789647076259)), (Array(70, 85, 260, 113, 195),Array(0.012110483884254163, 0.0064196545583964214, 0.0046644671122361885, 0.004522535383402723, 0.003858167880370399)), (Array(269, 1444, 2157, 2592, 2702),Array(0.08655099647021003, 0.039588701471860624, 0.027362529200185913, 0.023957239335077818, 0.022674789610265166)), (Array(271, 302, 234, 866, 1858),Array(0.06494948578534294, 0.05479658672484445, 0.03202701607086499, 0.03185579878936539, 0.020373304237372863)), (Array(82, 105, 98, 130, 172),Array(0.08411532601710255, 0.06861519273333161, 0.0645509153944095, 0.046919681737576234, 0.03393173559526991)), (Array(39, 42, 83, 289, 606),Array(0.2675005896664047, 0.09592849535029228, 0.04349781576096616, 0.0196392508406257, 0.01294402564510321)))

In [ ]:
val vocabList = vectorizer.vocabulary

  

>     vocabList: Array[String] = Array(runt, trakten, meter, havet, månaden, nederbörd, invånare, kvadratkilometer, terrängen, landet, närheten, januari, huvudstaden, medeltemperaturen, varmaste, kallaste, årsmedeltemperaturen, högsta, kommentarer, nasalandcover, nasapop, millimeter, genomsnitt, genomsnittlig, torraste, årsnederbörd, regnigaste, nasarain, punkten, juli, koppen, geonames, större, omgivningarna, kuperad, augusti, råder, huvudsak, närmaste, cerro, samhälle, provinsen, berg, toppen, lake, ganska, kilometer, februari, juni, öster, talet, vattendrag, nordväst, nordost, december, platt, söder, väster, norr, glesbefolkat, växer, sydväst, september, huvudsakligen, tätbefolkat, sydost, november, mycket, mars, svenska, född, kanada, ingår, består, nära, mindre, april, centrala, sverige, svensk, oktober, samma, from, kulle, stockholm, senare, kommun, antalet, största, omgivande, basen, olika, mexiko, bredden, norra, county, sydöstra, bergig, small, klimatzonen, jordbruksmark, kommunen, riktning, tidigare, stora, shift, södra, många, regionen, obefolkad, blandskog, området, följande, tillsammans, ottawa, växtlighet, lite, robotskapade, hemiboreala, östra, delstaten, creek, namn, västra, enligt, läst, namnet, ofta, math, började, value, sveriges, öppet, city, stor, countyt, exempel, stäppklimat, nord, början, busklandskap, kallt, plats, gallery, amerikansk, iran, socken, staden, mest, gunung, teheran, historia, vann, personer, gräsmarker, persiska, mexico, därefter, sträcker, grund, arean, spelade, endast, platser, medan, frankrike, cheshmeh, lövskog, carl, cirka, västlig, sydlig, linegrey, förekommer, norrut, övrigt, cobar, åren, john, först, gamla, söderut, klimatet, familjen, ökenklimat, filmen, idag, naturlig, kyrka, används, vissa, säsongen, mosaik, nordvästra, johan, barn, församling, kyrkan, antal, österut, delar, västerut, tropiskt, författare, iucn, bort, skådespelare, medelhavsklimat, slutet, göteborg, samhällen, omkring, nordöstra, arten, städsegrön, stockholms, saint, tiden, tyskland, indonesien, ofruktbar, loma, karl, tyska, savannklimat, world, distriktet, skrev, kallas, känd, band, främst, erik, stort, bästa, gången, finland, departementet, ontario, width, gray, sluttar, politiker, periodiskt, uppsala, sjön, istället, samtidigt, amerikanska, nuvarande, serien, york, ordningen, timeline, malmö, london, engelska, plotdata, gift, gånger, biografi, universitet, egen, franska, regi, tillbaka, file, trots, version, ryssland, island, bosnien, byggdes, peter, äldre, black, kring, santa, form, samband, kung, hett, deltog, sång, musik, maria, dessutom, därmed, slut, chile, europa, paris, jean, ytterligare, storbritannien, kallade, lång, land, sida, kända, album, albumet, hercegovina, sidan, miljoner, enda, medlemmar, verk, gruppen, jabal, film, period, översättning, england, övriga, jakarta, dotter, måste, gustaf, sydvästra, ledde, inlandsklimat, ordförande, mountain, anders, längre, norge, matcher, spanien, fortfarande, låten, savannskog, stad, flyttade, världen, spelare, bandet, källa, särskilt, lars, grupp, david, numera, professor, aldrig, brasilien, kriget, anna, varierad, bildades, visar, länder, birds, australien, världskriget, läggs, karriär, district, fredrik, provincia, tall, tysk, totalt, marocko, kort, pass, syfta, ledamot, park, tempererat, indien, urval, ryska, nytt, poäng, tillhör, notera, danmark, exempelvis, kartan, göteborgs, ändras, mount, internationella, arbetade, användes, puerto, stod, rabat, museum, släpptes, kallad, platserna, innehåller, university, robert, varav, människor, start, martin, color, nästan, förbli, arter, gitarr, såsom, dagens, höjd, världens, bergstopp, ungefär, liksom, rubrik, österrike, studerade, boreala, italien, korrigeras, mallen, rättvisande, ursprungligen, brittiska, paul, egna, slutade, säsong, kvinnor, mitten, klubben, guld, fortsatte, thomas, fuktigt, bild, boken, subtropiskt, lämnade, division, ville, andersson, djebel, språk, gifte, roll, kungliga, year, lika, förgreningar, huset, laget, medlem, population, spela, oftast, serie, liten, politiska, nils, arbete, högre, gustav, långa, förutom, afghanistan, procent, verksamhet, kina, william, james, regeringen, bergspass, spelades, kantonen, berlin, studier, själva, områden, punta, president, lyckades, british, brittisk, argentina, colombia, alto, bror, befolkningsutveckling, företaget, michael, vatten, love, lever, historik, lund, gora, europeiska, regnskogsklimat, republiken, chef, airport, sjöarna, spelar, varmt, landskommun, varandra, utbredning, innebär, släktet, respektive, rätt, grand, uppfördes, officiella, anses, sven, årets, använda, förlorade, skolan, område, league, henrik, quebrada, direkt, wien, columbia, santiago, japan, spelet, grundades, vita, dels, slaget, late, hill, titeln, george, little, referens, hand, kabul, böcker, international, familj, fransk, johansson, fontsize, startade, olof, increment, modern, radio, lilla, spng, https, gridcolor, stöd, clements, slott, avsnitt, trummor, användas, bottom, magnus, udde, minst, liknande, height, ghar, colors, vertical, använder, orientation, structure, ålder, föddes, svenskt, enwiki, längs, nilsson, imagesize, timeaxis, plotarea, scalemajor, dateformat, helsingfors, alignbars, yyyy, tillhörde, textcolor, bilder, talets, monsunklimat, långt, danska, uppdrag, volution, äldsta, svart, avled, arrondissementet, kungen, visade, tidningen, tidigt, sommaren, grande, insee, chiffres, togs, hösten, alltid, vanligen, madrid, problem, börjar, samtliga, länge, bestod, kronor, system, sankt, richard, charles, spel, låtar, gavs, uppe, flertal, pasir, troligen, south, moderna, åter, musiker, jawa, bibliografi, förlag, byggnaden, delas, axel, enbart, utgör, lagen, gammal, olympiska, ingick, företag, centrum, utveckling, omfattande, laguna, eget, samhället, växte, rock, person, sett, nådde, betydelse, partiet, kallades, north, grekland, nere, gunnar, gjordes, villa, strax, life, viss, live, resultat, stadens, kalifornien, utseende, medverkade, filmer, ordning, jebel, bukit, ledare, betyder, status, marie, handling, silver, sångare, ursprungliga, geografi, såväl, vägen, snabbt, röda, släkte, konst, gård, uppgick, with, ihop, vapen, återvände, administrativ, brdo, bättre, washington, skedde, adolf, underarter, valet, larsson, valde, sarajevo, utbildning, ursprung, national, födda, records, historiska, månader, norska, platsen, ordet, listan, algeriet, general, örebro, höga, verksam, finalen, koudiat, spår, program, kustklimat, ägde, arabiska, august, schweiz, polen, bygga, museet, final, lärare, beroende, handlar, tekniska, byggnader, dåvarande, allra, utmärkelser, roman, delhi, saguenay, match, starkt, delvis, daniel, livet, länet, lappland, press, finska, djur, sten, gränsen, fotboll, följd, sker, samarbete, ekonomiska, föreningen, räknas, programmet, arkitekt, sonora, stället, phou, bakgrund, yngre, stil, härad, använde, främsta, bengt, bildade, cruz, juan, just, konstnär, wilhelm, svarta, emellertid, våren, unga, west, alger, colorado, singel, alexander, river, tjänst, dansk, litteratur, innebar, louis, reservoar, smith, music, landets, pris, mont, skapa, brukar, skillnad, viktig, nordiska, målare, lokala, emot, lista, däremot, mängd, bodde, kultur, drottning, hustru, lades, bruk, spanska, entiteten, känt, priset, allmänna, förste, christian, ledning, chihuahua, befolkningen, krig, belägen, material, star, tidiga, följde, francisco, vanliga, grupper, piano, georg, medeltida, därpå, information, möjligt, tävlingen, arroyo, köpenhamn, aires, forskning, riket, buenos, johannes, italienska, vanligt, sydkorea, ungern, oscar, kroatien, peru, hölls, leder, henry, relativt, nyheter, albert, inklusive, egypten, beslut, ansåg, flyttades, spelat, grekiska, kompositör, jorden, höll, petit, försökte, slog, nova, militär, försök, nederländerna, guvernementet, fotbollsspelare, premiär, teater, valdes, samman, wood, barrskog, diskografi, verksamheten, ford, bong, förening, koudyet, socknen, städer, eriksson, denne, group, länsväg, omfattar, skola, utsågs, kalmar, plan, morro, senaste, produktion, olsson, norsk, gården, exemplar, sovjetunionen, arbetet, grundade, kyrkans, hall, kvinna, texas, tour, federationen, björn, vanlig, hålla, matchen, venezuela, orgel, utnämndes, american, röst, ersattes, staten, församlingen, skåne, distrikt, nationella, beskrevs, jesus, marken, skrivit, dennes, bytte, öppna, umeå, debuterade, skriven, antingen, folk, invigdes, turn, resa, svårt, kontrakt, massa, byggde, förslag, engelsk, kvinnliga, mode, savann, läns, bygger, ansågs, skriva, aktiv, lägre, eric, statens, såldes, tagit, monte, befolkning, tilldelades, skrevs, färg, perioden, levande, egenskaper, royal, kritik, arbeta, lämna, sullivan, stift, inre, sändes, journalist, barcelona, bulu)

In [ ]:
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}

  

>     topics: Array[Array[(String, Double)]] = Array(Array((översättning,0.03584527137263872), (socken,0.02491996116865994), (församling,0.02490628738653775), (socknen,0.022825068088324012), (uppgick,0.017879608346786705)), Array((runt,0.03226894586119219), (meter,0.024409431158202783), (havet,0.022132711657064968), (nederbörd,0.02148558030874581), (månaden,0.021356313771342332)), Array((klassen,0.03223323621121513), (orden,0.032140352784615744), (kommendör,0.028628035554835788), (riddare,0.02572213421059921), (senast,0.022714847469118508)), Array((gallery,0.04043182297527037), (familjen,0.036426713096318626), (ingår,0.03335972843491673), (släkte,0.03294952066003985), (file,0.0329078354521061)), Array((album,0.009358444119624937), (albumet,0.009218605994143599), (bandet,0.008569286231897809), (låten,0.008494384319722029), (gitarr,0.006593890980715514)), Array((meter,0.034014980004080625), (olympiska,0.03299031954197243), (herrar,0.01740713855328453), (sommarspelen,0.016493036980131597), (guld,0.016473530616266375)), Array((senare,0.005202912736908954), (ryska,0.004980332650975352), (kriget,0.004956359036949585), (svenska,0.004107078219445283), (slaget,0.003994412233925799)), Array((ford,0.02865565696236134), (grand,0.01497731749999038), (avsnittet,0.012856303027619758), (ferrari,0.011020796630618354), (star,0.01088948153793848)), Array((runt,0.04217375386317798), (trakten,0.03443350635893168), (havet,0.029432163834692823), (meter,0.029164515781816613), (månaden,0.028263430039365416)), Array((paris,0.021849080720731134), (frankrike,0.02104582186322425), (franska,0.019143573459580128), (saint,0.01906342816207476), (jean,0.017466187171276955)), Array((mountain,0.10161487730061466), (avsnitt,0.04696683184763887), (serien,0.03283890738943248), (serie,0.01767809259682316), (volvo,0.016690717240719372)), Array((geonames,0.16173888357413663), (county,0.04772361521855838), (kanada,0.045209691062612106), (robotskapade,0.04064929862689341), (lake,0.03671487993757405)), Array((birds,0.02369681037340463), (iucn,0.020619560775043934), (world,0.01805647612510781), (clements,0.015933941106456802), (läst,0.013941192536265194)), Array((cheshmeh,0.11064594620845476), (jabal,0.05967252584628175), (hill,0.022062432102082228), (runt,0.02161165367091075), (källa,0.01971175642758329)), Array((creek,0.12269798734719728), (vattendrag,0.11548170564697531), (math,0.1080413631122436), (county,0.05567056593344491), (geonames,0.03005264844658276)), Array((arten,0.013686188564871101), (iucn,0.010069959578121875), (förekommer,0.009834209338061728), (arter,0.009684304135010991), (släktet,0.006769956157331183)), Array((kung,0.03260243910309328), (hamilton,0.013986789615136346), (england,0.0121153524586439), (drottning,0.011625755878276552), (skottland,0.009249631189674428)), Array((bulu,0.07027290860872792), (runt,0.01886574550707347), (juan,0.016179163575668384), (viejo,0.015799019016393998), (meter,0.014780885004736585)), Array((sjön,0.034502823564636326), (kvadratkilometer,0.02864792614705575), (lago,0.025037979653382458), (maja,0.024953856343253075), (havet,0.024907385096831488)), Array((trakten,0.04096386788951671), (lake,0.0345224866846443), (runt,0.02963580174185564), (månaden,0.028115313448807144), (meter,0.027858463237621884)), Array((landskommun,0.025398611367203856), (församling,0.02289216003761088), (stmi,0.01846838158518249), (januari,0.0176316418497597), (kommunen,0.013980852803650965)), Array((staden,0.023384488081661554), (talet,0.014684568820705514), (spelet,0.008041407554453981), (historia,0.005899803204835226), (invånare,0.005315004050335149)), Array((geonames,0.11557194603913014), (vattendrag,0.10396838795255336), (mexiko,0.07664488602777711), (periodiskt,0.044735247147267086), (robotskapade,0.032041160696272615)), Array((university,0.015555705383864719), (läst,0.014550019239920886), (john,0.008732132798153132), (york,0.007788712779924475), (amerikansk,0.006967143392245979)), Array((länsväg,0.12364353938874159), (gradina,0.041319606928897926), (makedonien,0.027513514818681497), (eupithecia,0.02290853181360536), (font,0.02141349760145409)), Array((kommun,0.10290042230605351), (socken,0.05728034134114776), (lappland,0.01811165620130087), (småland,0.010530347777947284), (värmland,0.007339475945269198)), Array((solen,0.0200670569291999), (bibcode,0.016284072132314146), (gånger,0.0135712033444977), (stjärna,0.013479451859270629), (magnitud,0.01336898859104554)), Array((punta,0.09684045533161303), (udde,0.09188909300387012), (bild,0.03486418022216705), (point,0.023982054573107807), (road,0.016686082802654822)), Array((talet,0.012694136012504064), (meter,0.010889970029104822), (området,0.00882690394783239), (mount,0.008722786999135347), (stora,0.006863758992484095)), Array((buttu,0.04844274548131795), (runt,0.019001409457080374), (trakten,0.01675423618341787), (qimmat,0.014601997139814487), (meter,0.014327415704946335)), Array((stockholm,0.015135856576968138), (född,0.01214761100144125), (svenska,0.011952592249192522), (svensk,0.010682496369850301), (sveriges,0.006188853427958379)), Array((tang,0.03048136070857475), (runt,0.022898886518763237), (morne,0.020878561387803588), (castle,0.020243918240258764), (dahr,0.018246897035616844)), Array((filmen,0.020781490320475385), (film,0.007085303531532155), (musik,0.0064215730197610225), (regi,0.006262473771143063), (john,0.006106569562524211)), Array((bosnien,0.03607418632541323), (hercegovina,0.035660992757308906), (pasir,0.025960037172071338), (runt,0.024025464515943364), (sarajevo,0.02315327816207366)), Array((olika,0.009401336618340807), (exempel,0.0074417216581975035), (talet,0.006309347400350304), (ofta,0.00599903713081023), (många,0.0059618095032171435)), Array((puerto,0.152053730570257), (group,0.04342233664608969), (drae,0.02784126696431022), (runt,0.017052063582336168), (pass,0.016573201370297795)), Array((loma,0.029949250854912395), (mexiko,0.02401769656994233), (runt,0.023552175762361592), (city,0.023491880302950876), (mexico,0.023446760999741703)), Array((vann,0.018247050041365145), (säsongen,0.01699606742987092), (matcher,0.010585508000319446), (spelade,0.010541376807987448), (spelare,0.010174683683362207)), Array((stockholm,0.023463309330417906), (museum,0.020588688992590618), (konst,0.01531361753508411), (malmö,0.013601805832605934), (målare,0.01339290590857053)), Array((morro,0.029847680557851572), (runt,0.02478222302763145), (sang,0.022600156012697558), (trakten,0.02176298711203436), (monte,0.020395277368909562)), Array((runt,0.04674932020852935), (trakten,0.0357544398258386), (meter,0.03514437047605392), (havet,0.03080664018465887), (månaden,0.030751659585869125)), Array((talet,0.020927168859939324), (kyrkan,0.016978453949576977), (kyrka,0.016398363668156845), (gallery,0.009234697415750651), (gamla,0.006700990866659774)), Array((partiet,0.009200084245357574), (regeringen,0.006781072683076032), (valet,0.006751636661360212), (file,0.006591367471565553), (politiska,0.006211111760647785)), Array((företaget,0.017632507535482355), (airport,0.016531267519608497), (piano,0.011765155277036494), (international,0.009193469609380736), (orkester,0.008789598592035592)), Array((svensk,0.027812732588726418), (född,0.020404036898973906), (amerikansk,0.016893938539467617), (skådespelare,0.015654966665244758), (politiker,0.010136789647076259)), Array((född,0.012110483884254163), (senare,0.0064196545583964214), (biografi,0.0046644671122361885), (tillsammans,0.004522535383402723), (barn,0.003858167880370399)), Array((ryssland,0.08655099647021003), (oblast,0.039588701471860624), (tubu,0.027362529200185913), (buntu,0.023957239335077818), (ozero,0.022674789610265166)), Array((bosnien,0.06494948578534294), (hercegovina,0.05479658672484445), (bästa,0.03202701607086499), (kroatien,0.03185579878936539), (serbien,0.020373304237372863)), Array((from,0.08411532601710255), (shift,0.06861519273333161), (small,0.0645509153944095), (value,0.046919681737576234), (linegrey,0.03393173559526991)), Array((cerro,0.2675005896664047), (berg,0.09592849535029228), (kulle,0.04349781576096616), (chile,0.0196392508406257), (grande,0.01294402564510321)))

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic)
val vocabList = vectorizer.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     50 topics:
>     TOPIC 0
>     översättning	0.03584527137263872
>     socken	0.02491996116865994
>     församling	0.02490628738653775
>     socknen	0.022825068088324012
>     uppgick	0.017879608346786705
>     ==========
>     TOPIC 1
>     runt	0.03226894586119219
>     meter	0.024409431158202783
>     havet	0.022132711657064968
>     nederbörd	0.02148558030874581
>     månaden	0.021356313771342332
>     ==========
>     TOPIC 2
>     klassen	0.03223323621121513
>     orden	0.032140352784615744
>     kommendör	0.028628035554835788
>     riddare	0.02572213421059921
>     senast	0.022714847469118508
>     ==========
>     TOPIC 3
>     gallery	0.04043182297527037
>     familjen	0.036426713096318626
>     ingår	0.03335972843491673
>     släkte	0.03294952066003985
>     file	0.0329078354521061
>     ==========
>     TOPIC 4
>     album	0.009358444119624937
>     albumet	0.009218605994143599
>     bandet	0.008569286231897809
>     låten	0.008494384319722029
>     gitarr	0.006593890980715514
>     ==========
>     TOPIC 5
>     meter	0.034014980004080625
>     olympiska	0.03299031954197243
>     herrar	0.01740713855328453
>     sommarspelen	0.016493036980131597
>     guld	0.016473530616266375
>     ==========
>     TOPIC 6
>     senare	0.005202912736908954
>     ryska	0.004980332650975352
>     kriget	0.004956359036949585
>     svenska	0.004107078219445283
>     slaget	0.003994412233925799
>     ==========
>     TOPIC 7
>     ford	0.02865565696236134
>     grand	0.01497731749999038
>     avsnittet	0.012856303027619758
>     ferrari	0.011020796630618354
>     star	0.01088948153793848
>     ==========
>     TOPIC 8
>     runt	0.04217375386317798
>     trakten	0.03443350635893168
>     havet	0.029432163834692823
>     meter	0.029164515781816613
>     månaden	0.028263430039365416
>     ==========
>     TOPIC 9
>     paris	0.021849080720731134
>     frankrike	0.02104582186322425
>     franska	0.019143573459580128
>     saint	0.01906342816207476
>     jean	0.017466187171276955
>     ==========
>     TOPIC 10
>     mountain	0.10161487730061466
>     avsnitt	0.04696683184763887
>     serien	0.03283890738943248
>     serie	0.01767809259682316
>     volvo	0.016690717240719372
>     ==========
>     TOPIC 11
>     geonames	0.16173888357413663
>     county	0.04772361521855838
>     kanada	0.045209691062612106
>     robotskapade	0.04064929862689341
>     lake	0.03671487993757405
>     ==========
>     TOPIC 12
>     birds	0.02369681037340463
>     iucn	0.020619560775043934
>     world	0.01805647612510781
>     clements	0.015933941106456802
>     läst	0.013941192536265194
>     ==========
>     TOPIC 13
>     cheshmeh	0.11064594620845476
>     jabal	0.05967252584628175
>     hill	0.022062432102082228
>     runt	0.02161165367091075
>     källa	0.01971175642758329
>     ==========
>     TOPIC 14
>     creek	0.12269798734719728
>     vattendrag	0.11548170564697531
>     math	0.1080413631122436
>     county	0.05567056593344491
>     geonames	0.03005264844658276
>     ==========
>     TOPIC 15
>     arten	0.013686188564871101
>     iucn	0.010069959578121875
>     förekommer	0.009834209338061728
>     arter	0.009684304135010991
>     släktet	0.006769956157331183
>     ==========
>     TOPIC 16
>     kung	0.03260243910309328
>     hamilton	0.013986789615136346
>     england	0.0121153524586439
>     drottning	0.011625755878276552
>     skottland	0.009249631189674428
>     ==========
>     TOPIC 17
>     bulu	0.07027290860872792
>     runt	0.01886574550707347
>     juan	0.016179163575668384
>     viejo	0.015799019016393998
>     meter	0.014780885004736585
>     ==========
>     TOPIC 18
>     sjön	0.034502823564636326
>     kvadratkilometer	0.02864792614705575
>     lago	0.025037979653382458
>     maja	0.024953856343253075
>     havet	0.024907385096831488
>     ==========
>     TOPIC 19
>     trakten	0.04096386788951671
>     lake	0.0345224866846443
>     runt	0.02963580174185564
>     månaden	0.028115313448807144
>     meter	0.027858463237621884
>     ==========
>     TOPIC 20
>     landskommun	0.025398611367203856
>     församling	0.02289216003761088
>     stmi	0.01846838158518249
>     januari	0.0176316418497597
>     kommunen	0.013980852803650965
>     ==========
>     TOPIC 21
>     staden	0.023384488081661554
>     talet	0.014684568820705514
>     spelet	0.008041407554453981
>     historia	0.005899803204835226
>     invånare	0.005315004050335149
>     ==========
>     TOPIC 22
>     geonames	0.11557194603913014
>     vattendrag	0.10396838795255336
>     mexiko	0.07664488602777711
>     periodiskt	0.044735247147267086
>     robotskapade	0.032041160696272615
>     ==========
>     TOPIC 23
>     university	0.015555705383864719
>     läst	0.014550019239920886
>     john	0.008732132798153132
>     york	0.007788712779924475
>     amerikansk	0.006967143392245979
>     ==========
>     TOPIC 24
>     länsväg	0.12364353938874159
>     gradina	0.041319606928897926
>     makedonien	0.027513514818681497
>     eupithecia	0.02290853181360536
>     font	0.02141349760145409
>     ==========
>     TOPIC 25
>     kommun	0.10290042230605351
>     socken	0.05728034134114776
>     lappland	0.01811165620130087
>     småland	0.010530347777947284
>     värmland	0.007339475945269198
>     ==========
>     TOPIC 26
>     solen	0.0200670569291999
>     bibcode	0.016284072132314146
>     gånger	0.0135712033444977
>     stjärna	0.013479451859270629
>     magnitud	0.01336898859104554
>     ==========
>     TOPIC 27
>     punta	0.09684045533161303
>     udde	0.09188909300387012
>     bild	0.03486418022216705
>     point	0.023982054573107807
>     road	0.016686082802654822
>     ==========
>     TOPIC 28
>     talet	0.012694136012504064
>     meter	0.010889970029104822
>     området	0.00882690394783239
>     mount	0.008722786999135347
>     stora	0.006863758992484095
>     ==========
>     TOPIC 29
>     buttu	0.04844274548131795
>     runt	0.019001409457080374
>     trakten	0.01675423618341787
>     qimmat	0.014601997139814487
>     meter	0.014327415704946335
>     ==========
>     TOPIC 30
>     stockholm	0.015135856576968138
>     född	0.01214761100144125
>     svenska	0.011952592249192522
>     svensk	0.010682496369850301
>     sveriges	0.006188853427958379
>     ==========
>     TOPIC 31
>     tang	0.03048136070857475
>     runt	0.022898886518763237
>     morne	0.020878561387803588
>     castle	0.020243918240258764
>     dahr	0.018246897035616844
>     ==========
>     TOPIC 32
>     filmen	0.020781490320475385
>     film	0.007085303531532155
>     musik	0.0064215730197610225
>     regi	0.006262473771143063
>     john	0.006106569562524211
>     ==========
>     TOPIC 33
>     bosnien	0.03607418632541323
>     hercegovina	0.035660992757308906
>     pasir	0.025960037172071338
>     runt	0.024025464515943364
>     sarajevo	0.02315327816207366
>     ==========
>     TOPIC 34
>     olika	0.009401336618340807
>     exempel	0.0074417216581975035
>     talet	0.006309347400350304
>     ofta	0.00599903713081023
>     många	0.0059618095032171435
>     ==========
>     TOPIC 35
>     puerto	0.152053730570257
>     group	0.04342233664608969
>     drae	0.02784126696431022
>     runt	0.017052063582336168
>     pass	0.016573201370297795
>     ==========
>     TOPIC 36
>     loma	0.029949250854912395
>     mexiko	0.02401769656994233
>     runt	0.023552175762361592
>     city	0.023491880302950876
>     mexico	0.023446760999741703
>     ==========
>     TOPIC 37
>     vann	0.018247050041365145
>     säsongen	0.01699606742987092
>     matcher	0.010585508000319446
>     spelade	0.010541376807987448
>     spelare	0.010174683683362207
>     ==========
>     TOPIC 38
>     stockholm	0.023463309330417906
>     museum	0.020588688992590618
>     konst	0.01531361753508411
>     malmö	0.013601805832605934
>     målare	0.01339290590857053
>     ==========
>     TOPIC 39
>     morro	0.029847680557851572
>     runt	0.02478222302763145
>     sang	0.022600156012697558
>     trakten	0.02176298711203436
>     monte	0.020395277368909562
>     ==========
>     TOPIC 40
>     runt	0.04674932020852935
>     trakten	0.0357544398258386
>     meter	0.03514437047605392
>     havet	0.03080664018465887
>     månaden	0.030751659585869125
>     ==========
>     TOPIC 41
>     talet	0.020927168859939324
>     kyrkan	0.016978453949576977
>     kyrka	0.016398363668156845
>     gallery	0.009234697415750651
>     gamla	0.006700990866659774
>     ==========
>     TOPIC 42
>     partiet	0.009200084245357574
>     regeringen	0.006781072683076032
>     valet	0.006751636661360212
>     file	0.006591367471565553
>     politiska	0.006211111760647785
>     ==========
>     TOPIC 43
>     företaget	0.017632507535482355
>     airport	0.016531267519608497
>     piano	0.011765155277036494
>     international	0.009193469609380736
>     orkester	0.008789598592035592
>     ==========
>     TOPIC 44
>     svensk	0.027812732588726418
>     född	0.020404036898973906
>     amerikansk	0.016893938539467617
>     skådespelare	0.015654966665244758
>     politiker	0.010136789647076259
>     ==========
>     TOPIC 45
>     född	0.012110483884254163
>     senare	0.0064196545583964214
>     biografi	0.0046644671122361885
>     tillsammans	0.004522535383402723
>     barn	0.003858167880370399
>     ==========
>     TOPIC 46
>     ryssland	0.08655099647021003
>     oblast	0.039588701471860624
>     tubu	0.027362529200185913
>     buntu	0.023957239335077818
>     ozero	0.022674789610265166
>     ==========
>     TOPIC 47
>     bosnien	0.06494948578534294
>     hercegovina	0.05479658672484445
>     bästa	0.03202701607086499
>     kroatien	0.03185579878936539
>     serbien	0.020373304237372863
>     ==========
>     TOPIC 48
>     from	0.08411532601710255
>     shift	0.06861519273333161
>     small	0.0645509153944095
>     value	0.046919681737576234
>     linegrey	0.03393173559526991
>     ==========
>     TOPIC 49
>     cerro	0.2675005896664047
>     berg	0.09592849535029228
>     kulle	0.04349781576096616
>     chile	0.0196392508406257
>     grande	0.01294402564510321
>     ==========
>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(312, 146, 196, 905, 697),Array(0.03584527137263872, 0.02491996116865994, 0.02490628738653775, 0.022825068088324012, 0.017879608346786705)), (Array(0, 2, 3, 5, 4),Array(0.03226894586119219, 0.024409431158202783, 0.022132711657064968, 0.02148558030874581, 0.021356313771342332)), (Array(1204, 1064, 1999, 1606, 1398),Array(0.03223323621121513, 0.032140352784615744, 0.028628035554835788, 0.02572213421059921, 0.022714847469118508)), (Array(143, 183, 72, 694, 266),Array(0.04043182297527037, 0.036426713096318626, 0.03335972843491673, 0.03294952066003985, 0.0329078354521061)), (Array(300, 301, 336, 330, 404),Array(0.009358444119624937, 0.009218605994143599, 0.008569286231897809, 0.008494384319722029, 0.006593890980715514)), (Array(2, 645, 1041, 1513, 429),Array(0.034014980004080625, 0.03299031954197243, 0.01740713855328453, 0.016493036980131597, 0.016473530616266375)), (Array(85, 372, 346, 69, 532),Array(0.005202912736908954, 0.004980332650975352, 0.004956359036949585, 0.004107078219445283, 0.003994412233925799)), (Array(901, 510, 1279, 2568, 841),Array(0.02865565696236134, 0.01497731749999038, 0.012856303027619758, 0.011020796630618354, 0.01088948153793848)), (Array(0, 1, 3, 2, 4),Array(0.04217375386317798, 0.03443350635893168, 0.029432163834692823, 0.029164515781816613, 0.028263430039365416)), (Array(291, 165, 263, 216, 292),Array(0.021849080720731134, 0.02104582186322425, 0.019143573459580128, 0.01906342816207476, 0.017466187171276955)), (Array(323, 559, 250, 454, 2262),Array(0.10161487730061466, 0.04696683184763887, 0.03283890738943248, 0.01767809259682316, 0.016690717240719372)), (Array(31, 95, 71, 117, 44),Array(0.16173888357413663, 0.04772361521855838, 0.045209691062612106, 0.04064929862689341, 0.03671487993757405)), (Array(352, 204, 225, 557, 125),Array(0.02369681037340463, 0.020619560775043934, 0.01805647612510781, 0.015933941106456802, 0.013941192536265194)), (Array(166, 309, 534, 0, 337),Array(0.11064594620845476, 0.05967252584628175, 0.022062432102082228, 0.02161165367091075, 0.01971175642758329)), (Array(121, 51, 128, 95, 31),Array(0.12269798734719728, 0.11548170564697531, 0.1080413631122436, 0.05567056593344491, 0.03005264844658276)), (Array(213, 204, 173, 403, 507),Array(0.013686188564871101, 0.010069959578121875, 0.009834209338061728, 0.009684304135010991, 0.006769956157331183)), (Array(280, 1797, 313, 824, 1574),Array(0.03260243910309328, 0.013986789615136346, 0.0121153524586439, 0.011625755878276552, 0.009249631189674428)), (Array(999, 0, 786, 3834, 2),Array(0.07027290860872792, 0.01886574550707347, 0.016179163575668384, 0.015799019016393998, 0.014780885004736585)), (Array(245, 7, 1700, 1479, 3),Array(0.034502823564636326, 0.02864792614705575, 0.025037979653382458, 0.024953856343253075, 0.024907385096831488)), (Array(1, 44, 0, 4, 2),Array(0.04096386788951671, 0.0345224866846443, 0.02963580174185564, 0.028115313448807144, 0.027858463237621884)), (Array(503, 196, 1193, 11, 101),Array(0.025398611367203856, 0.02289216003761088, 0.01846838158518249, 0.0176316418497597, 0.013980852803650965)), (Array(147, 50, 528, 151, 6),Array(0.023384488081661554, 0.014684568820705514, 0.008041407554453981, 0.005899803204835226, 0.005315004050335149)), (Array(31, 51, 92, 243, 117),Array(0.11557194603913014, 0.10396838795255336, 0.07664488602777711, 0.044735247147267086, 0.032041160696272615)), (Array(394, 125, 178, 251, 144),Array(0.015555705383864719, 0.014550019239920886, 0.008732132798153132, 0.007788712779924475, 0.006967143392245979)), (Array(910, 3004, 2203, 6220, 5258),Array(0.12364353938874159, 0.041319606928897926, 0.027513514818681497, 0.02290853181360536, 0.02141349760145409)), (Array(86, 146, 759, 1047, 1682),Array(0.10290042230605351, 0.05728034134114776, 0.01811165620130087, 0.010530347777947284, 0.007339475945269198)), (Array(1078, 1665, 259, 1473, 2021),Array(0.0200670569291999, 0.016284072132314146, 0.0135712033444977, 0.013479451859270629, 0.01336898859104554)), (Array(477, 564, 433, 1543, 2353),Array(0.09684045533161303, 0.09188909300387012, 0.03486418022216705, 0.023982054573107807, 0.016686082802654822)), (Array(50, 2, 111, 382, 104),Array(0.012694136012504064, 0.010889970029104822, 0.00882690394783239, 0.008722786999135347, 0.006863758992484095)), (Array(1255, 0, 1, 4331, 2),Array(0.04844274548131795, 0.019001409457080374, 0.01675423618341787, 0.014601997139814487, 0.014327415704946335)), (Array(84, 70, 69, 79, 131),Array(0.015135856576968138, 0.01214761100144125, 0.011952592249192522, 0.010682496369850301, 0.006188853427958379)), (Array(1248, 0, 1518, 1594, 2426),Array(0.03048136070857475, 0.022898886518763237, 0.020878561387803588, 0.020243918240258764, 0.018246897035616844)), (Array(185, 310, 284, 264, 178),Array(0.020781490320475385, 0.007085303531532155, 0.0064215730197610225, 0.006262473771143063, 0.006106569562524211)), (Array(271, 302, 629, 0, 712),Array(0.03607418632541323, 0.035660992757308906, 0.025960037172071338, 0.024025464515943364, 0.02315327816207366)), (Array(91, 136, 50, 127, 107),Array(0.009401336618340807, 0.0074417216581975035, 0.006309347400350304, 0.00599903713081023, 0.0059618095032171435)), (Array(386, 909, 2958, 0, 365),Array(0.152053730570257, 0.04342233664608969, 0.02784126696431022, 0.017052063582336168, 0.016573201370297795)), (Array(221, 92, 0, 133, 156),Array(0.029949250854912395, 0.02401769656994233, 0.023552175762361592, 0.023491880302950876, 0.023446760999741703)), (Array(152, 191, 327, 161, 335),Array(0.018247050041365145, 0.01699606742987092, 0.010585508000319446, 0.010541376807987448, 0.010174683683362207)), (Array(84, 389, 695, 254, 816),Array(0.023463309330417906, 0.020588688992590618, 0.01531361753508411, 0.013601805832605934, 0.01339290590857053)), (Array(916, 0, 1166, 1, 981),Array(0.029847680557851572, 0.02478222302763145, 0.022600156012697558, 0.02176298711203436, 0.020395277368909562)), (Array(0, 1, 2, 3, 4),Array(0.04674932020852935, 0.0357544398258386, 0.03514437047605392, 0.03080664018465887, 0.030751659585869125)), (Array(50, 197, 188, 143, 180),Array(0.020927168859939324, 0.016978453949576977, 0.016398363668156845, 0.009234697415750651, 0.006700990866659774)), (Array(660, 469, 709, 266, 456),Array(0.009200084245357574, 0.006781072683076032, 0.006751636661360212, 0.006591367471565553, 0.006211111760647785)), (Array(487, 499, 847, 542, 1603),Array(0.017632507535482355, 0.016531267519608497, 0.011765155277036494, 0.009193469609380736, 0.008789598592035592)), (Array(79, 70, 144, 206, 242),Array(0.027812732588726418, 0.020404036898973906, 0.016893938539467617, 0.015654966665244758, 0.010136789647076259)), (Array(70, 85, 260, 113, 195),Array(0.012110483884254163, 0.0064196545583964214, 0.0046644671122361885, 0.004522535383402723, 0.003858167880370399)), (Array(269, 1444, 2157, 2592, 2702),Array(0.08655099647021003, 0.039588701471860624, 0.027362529200185913, 0.023957239335077818, 0.022674789610265166)), (Array(271, 302, 234, 866, 1858),Array(0.06494948578534294, 0.05479658672484445, 0.03202701607086499, 0.03185579878936539, 0.020373304237372863)), (Array(82, 105, 98, 130, 172),Array(0.08411532601710255, 0.06861519273333161, 0.0645509153944095, 0.046919681737576234, 0.03393173559526991)), (Array(39, 42, 83, 289, 606),Array(0.2675005896664047, 0.09592849535029228, 0.04349781576096616, 0.0196392508406257, 0.01294402564510321)))
>     vocabList: Array[String] = Array(runt, trakten, meter, havet, månaden, nederbörd, invånare, kvadratkilometer, terrängen, landet, närheten, januari, huvudstaden, medeltemperaturen, varmaste, kallaste, årsmedeltemperaturen, högsta, kommentarer, nasalandcover, nasapop, millimeter, genomsnitt, genomsnittlig, torraste, årsnederbörd, regnigaste, nasarain, punkten, juli, koppen, geonames, större, omgivningarna, kuperad, augusti, råder, huvudsak, närmaste, cerro, samhälle, provinsen, berg, toppen, lake, ganska, kilometer, februari, juni, öster, talet, vattendrag, nordväst, nordost, december, platt, söder, väster, norr, glesbefolkat, växer, sydväst, september, huvudsakligen, tätbefolkat, sydost, november, mycket, mars, svenska, född, kanada, ingår, består, nära, mindre, april, centrala, sverige, svensk, oktober, samma, from, kulle, stockholm, senare, kommun, antalet, största, omgivande, basen, olika, mexiko, bredden, norra, county, sydöstra, bergig, small, klimatzonen, jordbruksmark, kommunen, riktning, tidigare, stora, shift, södra, många, regionen, obefolkad, blandskog, området, följande, tillsammans, ottawa, växtlighet, lite, robotskapade, hemiboreala, östra, delstaten, creek, namn, västra, enligt, läst, namnet, ofta, math, började, value, sveriges, öppet, city, stor, countyt, exempel, stäppklimat, nord, början, busklandskap, kallt, plats, gallery, amerikansk, iran, socken, staden, mest, gunung, teheran, historia, vann, personer, gräsmarker, persiska, mexico, därefter, sträcker, grund, arean, spelade, endast, platser, medan, frankrike, cheshmeh, lövskog, carl, cirka, västlig, sydlig, linegrey, förekommer, norrut, övrigt, cobar, åren, john, först, gamla, söderut, klimatet, familjen, ökenklimat, filmen, idag, naturlig, kyrka, används, vissa, säsongen, mosaik, nordvästra, johan, barn, församling, kyrkan, antal, österut, delar, västerut, tropiskt, författare, iucn, bort, skådespelare, medelhavsklimat, slutet, göteborg, samhällen, omkring, nordöstra, arten, städsegrön, stockholms, saint, tiden, tyskland, indonesien, ofruktbar, loma, karl, tyska, savannklimat, world, distriktet, skrev, kallas, känd, band, främst, erik, stort, bästa, gången, finland, departementet, ontario, width, gray, sluttar, politiker, periodiskt, uppsala, sjön, istället, samtidigt, amerikanska, nuvarande, serien, york, ordningen, timeline, malmö, london, engelska, plotdata, gift, gånger, biografi, universitet, egen, franska, regi, tillbaka, file, trots, version, ryssland, island, bosnien, byggdes, peter, äldre, black, kring, santa, form, samband, kung, hett, deltog, sång, musik, maria, dessutom, därmed, slut, chile, europa, paris, jean, ytterligare, storbritannien, kallade, lång, land, sida, kända, album, albumet, hercegovina, sidan, miljoner, enda, medlemmar, verk, gruppen, jabal, film, period, översättning, england, övriga, jakarta, dotter, måste, gustaf, sydvästra, ledde, inlandsklimat, ordförande, mountain, anders, längre, norge, matcher, spanien, fortfarande, låten, savannskog, stad, flyttade, världen, spelare, bandet, källa, särskilt, lars, grupp, david, numera, professor, aldrig, brasilien, kriget, anna, varierad, bildades, visar, länder, birds, australien, världskriget, läggs, karriär, district, fredrik, provincia, tall, tysk, totalt, marocko, kort, pass, syfta, ledamot, park, tempererat, indien, urval, ryska, nytt, poäng, tillhör, notera, danmark, exempelvis, kartan, göteborgs, ändras, mount, internationella, arbetade, användes, puerto, stod, rabat, museum, släpptes, kallad, platserna, innehåller, university, robert, varav, människor, start, martin, color, nästan, förbli, arter, gitarr, såsom, dagens, höjd, världens, bergstopp, ungefär, liksom, rubrik, österrike, studerade, boreala, italien, korrigeras, mallen, rättvisande, ursprungligen, brittiska, paul, egna, slutade, säsong, kvinnor, mitten, klubben, guld, fortsatte, thomas, fuktigt, bild, boken, subtropiskt, lämnade, division, ville, andersson, djebel, språk, gifte, roll, kungliga, year, lika, förgreningar, huset, laget, medlem, population, spela, oftast, serie, liten, politiska, nils, arbete, högre, gustav, långa, förutom, afghanistan, procent, verksamhet, kina, william, james, regeringen, bergspass, spelades, kantonen, berlin, studier, själva, områden, punta, president, lyckades, british, brittisk, argentina, colombia, alto, bror, befolkningsutveckling, företaget, michael, vatten, love, lever, historik, lund, gora, europeiska, regnskogsklimat, republiken, chef, airport, sjöarna, spelar, varmt, landskommun, varandra, utbredning, innebär, släktet, respektive, rätt, grand, uppfördes, officiella, anses, sven, årets, använda, förlorade, skolan, område, league, henrik, quebrada, direkt, wien, columbia, santiago, japan, spelet, grundades, vita, dels, slaget, late, hill, titeln, george, little, referens, hand, kabul, böcker, international, familj, fransk, johansson, fontsize, startade, olof, increment, modern, radio, lilla, spng, https, gridcolor, stöd, clements, slott, avsnitt, trummor, användas, bottom, magnus, udde, minst, liknande, height, ghar, colors, vertical, använder, orientation, structure, ålder, föddes, svenskt, enwiki, längs, nilsson, imagesize, timeaxis, plotarea, scalemajor, dateformat, helsingfors, alignbars, yyyy, tillhörde, textcolor, bilder, talets, monsunklimat, långt, danska, uppdrag, volution, äldsta, svart, avled, arrondissementet, kungen, visade, tidningen, tidigt, sommaren, grande, insee, chiffres, togs, hösten, alltid, vanligen, madrid, problem, börjar, samtliga, länge, bestod, kronor, system, sankt, richard, charles, spel, låtar, gavs, uppe, flertal, pasir, troligen, south, moderna, åter, musiker, jawa, bibliografi, förlag, byggnaden, delas, axel, enbart, utgör, lagen, gammal, olympiska, ingick, företag, centrum, utveckling, omfattande, laguna, eget, samhället, växte, rock, person, sett, nådde, betydelse, partiet, kallades, north, grekland, nere, gunnar, gjordes, villa, strax, life, viss, live, resultat, stadens, kalifornien, utseende, medverkade, filmer, ordning, jebel, bukit, ledare, betyder, status, marie, handling, silver, sångare, ursprungliga, geografi, såväl, vägen, snabbt, röda, släkte, konst, gård, uppgick, with, ihop, vapen, återvände, administrativ, brdo, bättre, washington, skedde, adolf, underarter, valet, larsson, valde, sarajevo, utbildning, ursprung, national, födda, records, historiska, månader, norska, platsen, ordet, listan, algeriet, general, örebro, höga, verksam, finalen, koudiat, spår, program, kustklimat, ägde, arabiska, august, schweiz, polen, bygga, museet, final, lärare, beroende, handlar, tekniska, byggnader, dåvarande, allra, utmärkelser, roman, delhi, saguenay, match, starkt, delvis, daniel, livet, länet, lappland, press, finska, djur, sten, gränsen, fotboll, följd, sker, samarbete, ekonomiska, föreningen, räknas, programmet, arkitekt, sonora, stället, phou, bakgrund, yngre, stil, härad, använde, främsta, bengt, bildade, cruz, juan, just, konstnär, wilhelm, svarta, emellertid, våren, unga, west, alger, colorado, singel, alexander, river, tjänst, dansk, litteratur, innebar, louis, reservoar, smith, music, landets, pris, mont, skapa, brukar, skillnad, viktig, nordiska, målare, lokala, emot, lista, däremot, mängd, bodde, kultur, drottning, hustru, lades, bruk, spanska, entiteten, känt, priset, allmänna, förste, christian, ledning, chihuahua, befolkningen, krig, belägen, material, star, tidiga, följde, francisco, vanliga, grupper, piano, georg, medeltida, därpå, information, möjligt, tävlingen, arroyo, köpenhamn, aires, forskning, riket, buenos, johannes, italienska, vanligt, sydkorea, ungern, oscar, kroatien, peru, hölls, leder, henry, relativt, nyheter, albert, inklusive, egypten, beslut, ansåg, flyttades, spelat, grekiska, kompositör, jorden, höll, petit, försökte, slog, nova, militär, försök, nederländerna, guvernementet, fotbollsspelare, premiär, teater, valdes, samman, wood, barrskog, diskografi, verksamheten, ford, bong, förening, koudyet, socknen, städer, eriksson, denne, group, länsväg, omfattar, skola, utsågs, kalmar, plan, morro, senaste, produktion, olsson, norsk, gården, exemplar, sovjetunionen, arbetet, grundade, kyrkans, hall, kvinna, texas, tour, federationen, björn, vanlig, hålla, matchen, venezuela, orgel, utnämndes, american, röst, ersattes, staten, församlingen, skåne, distrikt, nationella, beskrevs, jesus, marken, skrivit, dennes, bytte, öppna, umeå, debuterade, skriven, antingen, folk, invigdes, turn, resa, svårt, kontrakt, massa, byggde, förslag, engelsk, kvinnliga, mode, savann, läns, bygger, ansågs, skriva, aktiv, lägre, eric, statens, såldes, tagit, monte, befolkning, tilldelades, skrevs, färg, perioden, levande, egenskaper, royal, kritik, arbeta, lämna, sullivan, stift, inre, sändes, journalist, barcelona, bulu)
>     topics: Array[Array[(String, Double)]] = Array(Array((översättning,0.03584527137263872), (socken,0.02491996116865994), (församling,0.02490628738653775), (socknen,0.022825068088324012), (uppgick,0.017879608346786705)), Array((runt,0.03226894586119219), (meter,0.024409431158202783), (havet,0.022132711657064968), (nederbörd,0.02148558030874581), (månaden,0.021356313771342332)), Array((klassen,0.03223323621121513), (orden,0.032140352784615744), (kommendör,0.028628035554835788), (riddare,0.02572213421059921), (senast,0.022714847469118508)), Array((gallery,0.04043182297527037), (familjen,0.036426713096318626), (ingår,0.03335972843491673), (släkte,0.03294952066003985), (file,0.0329078354521061)), Array((album,0.009358444119624937), (albumet,0.009218605994143599), (bandet,0.008569286231897809), (låten,0.008494384319722029), (gitarr,0.006593890980715514)), Array((meter,0.034014980004080625), (olympiska,0.03299031954197243), (herrar,0.01740713855328453), (sommarspelen,0.016493036980131597), (guld,0.016473530616266375)), Array((senare,0.005202912736908954), (ryska,0.004980332650975352), (kriget,0.004956359036949585), (svenska,0.004107078219445283), (slaget,0.003994412233925799)), Array((ford,0.02865565696236134), (grand,0.01497731749999038), (avsnittet,0.012856303027619758), (ferrari,0.011020796630618354), (star,0.01088948153793848)), Array((runt,0.04217375386317798), (trakten,0.03443350635893168), (havet,0.029432163834692823), (meter,0.029164515781816613), (månaden,0.028263430039365416)), Array((paris,0.021849080720731134), (frankrike,0.02104582186322425), (franska,0.019143573459580128), (saint,0.01906342816207476), (jean,0.017466187171276955)), Array((mountain,0.10161487730061466), (avsnitt,0.04696683184763887), (serien,0.03283890738943248), (serie,0.01767809259682316), (volvo,0.016690717240719372)), Array((geonames,0.16173888357413663), (county,0.04772361521855838), (kanada,0.045209691062612106), (robotskapade,0.04064929862689341), (lake,0.03671487993757405)), Array((birds,0.02369681037340463), (iucn,0.020619560775043934), (world,0.01805647612510781), (clements,0.015933941106456802), (läst,0.013941192536265194)), Array((cheshmeh,0.11064594620845476), (jabal,0.05967252584628175), (hill,0.022062432102082228), (runt,0.02161165367091075), (källa,0.01971175642758329)), Array((creek,0.12269798734719728), (vattendrag,0.11548170564697531), (math,0.1080413631122436), (county,0.05567056593344491), (geonames,0.03005264844658276)), Array((arten,0.013686188564871101), (iucn,0.010069959578121875), (förekommer,0.009834209338061728), (arter,0.009684304135010991), (släktet,0.006769956157331183)), Array((kung,0.03260243910309328), (hamilton,0.013986789615136346), (england,0.0121153524586439), (drottning,0.011625755878276552), (skottland,0.009249631189674428)), Array((bulu,0.07027290860872792), (runt,0.01886574550707347), (juan,0.016179163575668384), (viejo,0.015799019016393998), (meter,0.014780885004736585)), Array((sjön,0.034502823564636326), (kvadratkilometer,0.02864792614705575), (lago,0.025037979653382458), (maja,0.024953856343253075), (havet,0.024907385096831488)), Array((trakten,0.04096386788951671), (lake,0.0345224866846443), (runt,0.02963580174185564), (månaden,0.028115313448807144), (meter,0.027858463237621884)), Array((landskommun,0.025398611367203856), (församling,0.02289216003761088), (stmi,0.01846838158518249), (januari,0.0176316418497597), (kommunen,0.013980852803650965)), Array((staden,0.023384488081661554), (talet,0.014684568820705514), (spelet,0.008041407554453981), (historia,0.005899803204835226), (invånare,0.005315004050335149)), Array((geonames,0.11557194603913014), (vattendrag,0.10396838795255336), (mexiko,0.07664488602777711), (periodiskt,0.044735247147267086), (robotskapade,0.032041160696272615)), Array((university,0.015555705383864719), (läst,0.014550019239920886), (john,0.008732132798153132), (york,0.007788712779924475), (amerikansk,0.006967143392245979)), Array((länsväg,0.12364353938874159), (gradina,0.041319606928897926), (makedonien,0.027513514818681497), (eupithecia,0.02290853181360536), (font,0.02141349760145409)), Array((kommun,0.10290042230605351), (socken,0.05728034134114776), (lappland,0.01811165620130087), (småland,0.010530347777947284), (värmland,0.007339475945269198)), Array((solen,0.0200670569291999), (bibcode,0.016284072132314146), (gånger,0.0135712033444977), (stjärna,0.013479451859270629), (magnitud,0.01336898859104554)), Array((punta,0.09684045533161303), (udde,0.09188909300387012), (bild,0.03486418022216705), (point,0.023982054573107807), (road,0.016686082802654822)), Array((talet,0.012694136012504064), (meter,0.010889970029104822), (området,0.00882690394783239), (mount,0.008722786999135347), (stora,0.006863758992484095)), Array((buttu,0.04844274548131795), (runt,0.019001409457080374), (trakten,0.01675423618341787), (qimmat,0.014601997139814487), (meter,0.014327415704946335)), Array((stockholm,0.015135856576968138), (född,0.01214761100144125), (svenska,0.011952592249192522), (svensk,0.010682496369850301), (sveriges,0.006188853427958379)), Array((tang,0.03048136070857475), (runt,0.022898886518763237), (morne,0.020878561387803588), (castle,0.020243918240258764), (dahr,0.018246897035616844)), Array((filmen,0.020781490320475385), (film,0.007085303531532155), (musik,0.0064215730197610225), (regi,0.006262473771143063), (john,0.006106569562524211)), Array((bosnien,0.03607418632541323), (hercegovina,0.035660992757308906), (pasir,0.025960037172071338), (runt,0.024025464515943364), (sarajevo,0.02315327816207366)), Array((olika,0.009401336618340807), (exempel,0.0074417216581975035), (talet,0.006309347400350304), (ofta,0.00599903713081023), (många,0.0059618095032171435)), Array((puerto,0.152053730570257), (group,0.04342233664608969), (drae,0.02784126696431022), (runt,0.017052063582336168), (pass,0.016573201370297795)), Array((loma,0.029949250854912395), (mexiko,0.02401769656994233), (runt,0.023552175762361592), (city,0.023491880302950876), (mexico,0.023446760999741703)), Array((vann,0.018247050041365145), (säsongen,0.01699606742987092), (matcher,0.010585508000319446), (spelade,0.010541376807987448), (spelare,0.010174683683362207)), Array((stockholm,0.023463309330417906), (museum,0.020588688992590618), (konst,0.01531361753508411), (malmö,0.013601805832605934), (målare,0.01339290590857053)), Array((morro,0.029847680557851572), (runt,0.02478222302763145), (sang,0.022600156012697558), (trakten,0.02176298711203436), (monte,0.020395277368909562)), Array((runt,0.04674932020852935), (trakten,0.0357544398258386), (meter,0.03514437047605392), (havet,0.03080664018465887), (månaden,0.030751659585869125)), Array((talet,0.020927168859939324), (kyrkan,0.016978453949576977), (kyrka,0.016398363668156845), (gallery,0.009234697415750651), (gamla,0.006700990866659774)), Array((partiet,0.009200084245357574), (regeringen,0.006781072683076032), (valet,0.006751636661360212), (file,0.006591367471565553), (politiska,0.006211111760647785)), Array((företaget,0.017632507535482355), (airport,0.016531267519608497), (piano,0.011765155277036494), (international,0.009193469609380736), (orkester,0.008789598592035592)), Array((svensk,0.027812732588726418), (född,0.020404036898973906), (amerikansk,0.016893938539467617), (skådespelare,0.015654966665244758), (politiker,0.010136789647076259)), Array((född,0.012110483884254163), (senare,0.0064196545583964214), (biografi,0.0046644671122361885), (tillsammans,0.004522535383402723), (barn,0.003858167880370399)), Array((ryssland,0.08655099647021003), (oblast,0.039588701471860624), (tubu,0.027362529200185913), (buntu,0.023957239335077818), (ozero,0.022674789610265166)), Array((bosnien,0.06494948578534294), (hercegovina,0.05479658672484445), (bästa,0.03202701607086499), (kroatien,0.03185579878936539), (serbien,0.020373304237372863)), Array((from,0.08411532601710255), (shift,0.06861519273333161), (small,0.0645509153944095), (value,0.046919681737576234), (linegrey,0.03393173559526991)), Array((cerro,0.2675005896664047), (berg,0.09592849535029228), (kulle,0.04349781576096616), (chile,0.0196392508406257), (grande,0.01294402564510321)))

In [ ]:
topics.zipWithIndex.filter({ case m => m._2 == 1944}).foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

  

Visualize Results
=================

In [ ]:
// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

  

>     termArray: Array[(Array[(String, Double)], Int)] = Array((Array((översättning,0.03584527137263872), (socken,0.02491996116865994), (församling,0.02490628738653775), (socknen,0.022825068088324012), (uppgick,0.017879608346786705)),0), (Array((runt,0.03226894586119219), (meter,0.024409431158202783), (havet,0.022132711657064968), (nederbörd,0.02148558030874581), (månaden,0.021356313771342332)),1), (Array((klassen,0.03223323621121513), (orden,0.032140352784615744), (kommendör,0.028628035554835788), (riddare,0.02572213421059921), (senast,0.022714847469118508)),2), (Array((gallery,0.04043182297527037), (familjen,0.036426713096318626), (ingår,0.03335972843491673), (släkte,0.03294952066003985), (file,0.0329078354521061)),3), (Array((album,0.009358444119624937), (albumet,0.009218605994143599), (bandet,0.008569286231897809), (låten,0.008494384319722029), (gitarr,0.006593890980715514)),4), (Array((meter,0.034014980004080625), (olympiska,0.03299031954197243), (herrar,0.01740713855328453), (sommarspelen,0.016493036980131597), (guld,0.016473530616266375)),5), (Array((senare,0.005202912736908954), (ryska,0.004980332650975352), (kriget,0.004956359036949585), (svenska,0.004107078219445283), (slaget,0.003994412233925799)),6), (Array((ford,0.02865565696236134), (grand,0.01497731749999038), (avsnittet,0.012856303027619758), (ferrari,0.011020796630618354), (star,0.01088948153793848)),7), (Array((runt,0.04217375386317798), (trakten,0.03443350635893168), (havet,0.029432163834692823), (meter,0.029164515781816613), (månaden,0.028263430039365416)),8), (Array((paris,0.021849080720731134), (frankrike,0.02104582186322425), (franska,0.019143573459580128), (saint,0.01906342816207476), (jean,0.017466187171276955)),9), (Array((mountain,0.10161487730061466), (avsnitt,0.04696683184763887), (serien,0.03283890738943248), (serie,0.01767809259682316), (volvo,0.016690717240719372)),10), (Array((geonames,0.16173888357413663), (county,0.04772361521855838), (kanada,0.045209691062612106), (robotskapade,0.04064929862689341), (lake,0.03671487993757405)),11), (Array((birds,0.02369681037340463), (iucn,0.020619560775043934), (world,0.01805647612510781), (clements,0.015933941106456802), (läst,0.013941192536265194)),12), (Array((cheshmeh,0.11064594620845476), (jabal,0.05967252584628175), (hill,0.022062432102082228), (runt,0.02161165367091075), (källa,0.01971175642758329)),13), (Array((creek,0.12269798734719728), (vattendrag,0.11548170564697531), (math,0.1080413631122436), (county,0.05567056593344491), (geonames,0.03005264844658276)),14), (Array((arten,0.013686188564871101), (iucn,0.010069959578121875), (förekommer,0.009834209338061728), (arter,0.009684304135010991), (släktet,0.006769956157331183)),15), (Array((kung,0.03260243910309328), (hamilton,0.013986789615136346), (england,0.0121153524586439), (drottning,0.011625755878276552), (skottland,0.009249631189674428)),16), (Array((bulu,0.07027290860872792), (runt,0.01886574550707347), (juan,0.016179163575668384), (viejo,0.015799019016393998), (meter,0.014780885004736585)),17), (Array((sjön,0.034502823564636326), (kvadratkilometer,0.02864792614705575), (lago,0.025037979653382458), (maja,0.024953856343253075), (havet,0.024907385096831488)),18), (Array((trakten,0.04096386788951671), (lake,0.0345224866846443), (runt,0.02963580174185564), (månaden,0.028115313448807144), (meter,0.027858463237621884)),19), (Array((landskommun,0.025398611367203856), (församling,0.02289216003761088), (stmi,0.01846838158518249), (januari,0.0176316418497597), (kommunen,0.013980852803650965)),20), (Array((staden,0.023384488081661554), (talet,0.014684568820705514), (spelet,0.008041407554453981), (historia,0.005899803204835226), (invånare,0.005315004050335149)),21), (Array((geonames,0.11557194603913014), (vattendrag,0.10396838795255336), (mexiko,0.07664488602777711), (periodiskt,0.044735247147267086), (robotskapade,0.032041160696272615)),22), (Array((university,0.015555705383864719), (läst,0.014550019239920886), (john,0.008732132798153132), (york,0.007788712779924475), (amerikansk,0.006967143392245979)),23), (Array((länsväg,0.12364353938874159), (gradina,0.041319606928897926), (makedonien,0.027513514818681497), (eupithecia,0.02290853181360536), (font,0.02141349760145409)),24), (Array((kommun,0.10290042230605351), (socken,0.05728034134114776), (lappland,0.01811165620130087), (småland,0.010530347777947284), (värmland,0.007339475945269198)),25), (Array((solen,0.0200670569291999), (bibcode,0.016284072132314146), (gånger,0.0135712033444977), (stjärna,0.013479451859270629), (magnitud,0.01336898859104554)),26), (Array((punta,0.09684045533161303), (udde,0.09188909300387012), (bild,0.03486418022216705), (point,0.023982054573107807), (road,0.016686082802654822)),27), (Array((talet,0.012694136012504064), (meter,0.010889970029104822), (området,0.00882690394783239), (mount,0.008722786999135347), (stora,0.006863758992484095)),28), (Array((buttu,0.04844274548131795), (runt,0.019001409457080374), (trakten,0.01675423618341787), (qimmat,0.014601997139814487), (meter,0.014327415704946335)),29), (Array((stockholm,0.015135856576968138), (född,0.01214761100144125), (svenska,0.011952592249192522), (svensk,0.010682496369850301), (sveriges,0.006188853427958379)),30), (Array((tang,0.03048136070857475), (runt,0.022898886518763237), (morne,0.020878561387803588), (castle,0.020243918240258764), (dahr,0.018246897035616844)),31), (Array((filmen,0.020781490320475385), (film,0.007085303531532155), (musik,0.0064215730197610225), (regi,0.006262473771143063), (john,0.006106569562524211)),32), (Array((bosnien,0.03607418632541323), (hercegovina,0.035660992757308906), (pasir,0.025960037172071338), (runt,0.024025464515943364), (sarajevo,0.02315327816207366)),33), (Array((olika,0.009401336618340807), (exempel,0.0074417216581975035), (talet,0.006309347400350304), (ofta,0.00599903713081023), (många,0.0059618095032171435)),34), (Array((puerto,0.152053730570257), (group,0.04342233664608969), (drae,0.02784126696431022), (runt,0.017052063582336168), (pass,0.016573201370297795)),35), (Array((loma,0.029949250854912395), (mexiko,0.02401769656994233), (runt,0.023552175762361592), (city,0.023491880302950876), (mexico,0.023446760999741703)),36), (Array((vann,0.018247050041365145), (säsongen,0.01699606742987092), (matcher,0.010585508000319446), (spelade,0.010541376807987448), (spelare,0.010174683683362207)),37), (Array((stockholm,0.023463309330417906), (museum,0.020588688992590618), (konst,0.01531361753508411), (malmö,0.013601805832605934), (målare,0.01339290590857053)),38), (Array((morro,0.029847680557851572), (runt,0.02478222302763145), (sang,0.022600156012697558), (trakten,0.02176298711203436), (monte,0.020395277368909562)),39), (Array((runt,0.04674932020852935), (trakten,0.0357544398258386), (meter,0.03514437047605392), (havet,0.03080664018465887), (månaden,0.030751659585869125)),40), (Array((talet,0.020927168859939324), (kyrkan,0.016978453949576977), (kyrka,0.016398363668156845), (gallery,0.009234697415750651), (gamla,0.006700990866659774)),41), (Array((partiet,0.009200084245357574), (regeringen,0.006781072683076032), (valet,0.006751636661360212), (file,0.006591367471565553), (politiska,0.006211111760647785)),42), (Array((företaget,0.017632507535482355), (airport,0.016531267519608497), (piano,0.011765155277036494), (international,0.009193469609380736), (orkester,0.008789598592035592)),43), (Array((svensk,0.027812732588726418), (född,0.020404036898973906), (amerikansk,0.016893938539467617), (skådespelare,0.015654966665244758), (politiker,0.010136789647076259)),44), (Array((född,0.012110483884254163), (senare,0.0064196545583964214), (biografi,0.0046644671122361885), (tillsammans,0.004522535383402723), (barn,0.003858167880370399)),45), (Array((ryssland,0.08655099647021003), (oblast,0.039588701471860624), (tubu,0.027362529200185913), (buntu,0.023957239335077818), (ozero,0.022674789610265166)),46), (Array((bosnien,0.06494948578534294), (hercegovina,0.05479658672484445), (bästa,0.03202701607086499), (kroatien,0.03185579878936539), (serbien,0.020373304237372863)),47), (Array((from,0.08411532601710255), (shift,0.06861519273333161), (small,0.0645509153944095), (value,0.046919681737576234), (linegrey,0.03393173559526991)),48), (Array((cerro,0.2675005896664047), (berg,0.09592849535029228), (kulle,0.04349781576096616), (chile,0.0196392508406257), (grande,0.01294402564510321)),49))

In [ ]:
// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

  

>     termRDD: org.apache.spark.rdd.RDD[(Array[(String, Double)], Int)] = ParallelCollectionRDD[1639] at parallelize at command-738986526041674:2
>     termRDD2: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[1640] at flatMap at command-738986526041674:3

In [ ]:
// Create DF with proper column names
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

  

>     termDF: org.apache.spark.sql.DataFrame = [term: string, probability: double ... 1 more field]

In [ ]:
display(termDF)

  

[TABLE]

Truncated to 30 rows

In [ ]:
// Create JSON data
val rawJson = termDF.toJSON.collect().mkString(",\n")

  

>     rawJson: String =
>     {"term":"översättning","probability":0.03584527137263872,"topicId":0},
>     {"term":"socken","probability":0.02491996116865994,"topicId":0},
>     {"term":"församling","probability":0.02490628738653775,"topicId":0},
>     {"term":"socknen","probability":0.022825068088324012,"topicId":0},
>     {"term":"uppgick","probability":0.017879608346786705,"topicId":0},
>     {"term":"runt","probability":0.03226894586119219,"topicId":1},
>     {"term":"meter","probability":0.024409431158202783,"topicId":1},
>     {"term":"havet","probability":0.022132711657064968,"topicId":1},
>     {"term":"nederbörd","probability":0.02148558030874581,"topicId":1},
>     {"term":"månaden","probability":0.021356313771342332,"topicId":1},
>     {"term":"klassen","probability":0.03223323621121513,"topicId":2},
>     {"term":"orden","probability":0.032140352784615744,"topicId":2},
>     {"term":"kommendör","probability":0.028628035554835788,"topicId":2},
>     {"term":"riddare","probability":0.02572213421059921,"topicId":2},
>     {"term":"senast","probability":0.022714847469118508,"topicId":2},
>     {"term":"gallery","probability":0.04043182297527037,"topicId":3},
>     {"term":"familjen","probability":0.036426713096318626,"topicId":3},
>     {"term":"ingår","probability":0.03335972843491673,"topicId":3},
>     {"term":"släkte","probability":0.03294952066003985,"topicId":3},
>     {"term":"file","probability":0.0329078354521061,"topicId":3},
>     {"term":"album","probability":0.009358444119624937,"topicId":4},
>     {"term":"albumet","probability":0.009218605994143599,"topicId":4},
>     {"term":"bandet","probability":0.008569286231897809,"topicId":4},
>     {"term":"låten","probability":0.008494384319722029,"topicId":4},
>     {"term":"gitarr","probability":0.006593890980715514,"topicId":4},
>     {"term":"meter","probability":0.034014980004080625,"topicId":5},
>     {"term":"olympiska","probability":0.03299031954197243,"topicId":5},
>     {"term":"herrar","probability":0.01740713855328453,"topicId":5},
>     {"term":"sommarspelen","probability":0.016493036980131597,"topicId":5},
>     {"term":"guld","probability":0.016473530616266375,"topicId":5},
>     {"term":"senare","probability":0.005202912736908954,"topicId":6},
>     {"term":"ryska","probability":0.004980332650975352,"topicId":6},
>     {"term":"kriget","probability":0.004956359036949585,"topicId":6},
>     {"term":"svenska","probability":0.004107078219445283,"topicId":6},
>     {"term":"slaget","probability":0.003994412233925799,"topicId":6},
>     {"term":"ford","probability":0.02865565696236134,"topicId":7},
>     {"term":"grand","probability":0.01497731749999038,"topicId":7},
>     {"term":"avsnittet","probability":0.012856303027619758,"topicId":7},
>     {"term":"ferrari","probability":0.011020796630618354,"topicId":7},
>     {"term":"star","probability":0.01088948153793848,"topicId":7},
>     {"term":"runt","probability":0.04217375386317798,"topicId":8},
>     {"term":"trakten","probability":0.03443350635893168,"topicId":8},
>     {"term":"havet","probability":0.029432163834692823,"topicId":8},
>     {"term":"meter","probability":0.029164515781816613,"topicId":8},
>     {"term":"månaden","probability":0.028263430039365416,"topicId":8},
>     {"term":"paris","probability":0.021849080720731134,"topicId":9},
>     {"term":"frankrike","probability":0.02104582186322425,"topicId":9},
>     {"term":"franska","probability":0.019143573459580128,"topicId":9},
>     {"term":"saint","probability":0.01906342816207476,"topicId":9},
>     {"term":"jean","probability":0.017466187171276955,"topicId":9},
>     {"term":"mountain","probability":0.10161487730061466,"topicId":10},
>     {"term":"avsnitt","probability":0.04696683184763887,"topicId":10},
>     {"term":"serien","probability":0.03283890738943248,"topicId":10},
>     {"term":"serie","probability":0.01767809259682316,"topicId":10},
>     {"term":"volvo","probability":0.016690717240719372,"topicId":10},
>     {"term":"geonames","probability":0.16173888357413663,"topicId":11},
>     {"term":"county","probability":0.04772361521855838,"topicId":11},
>     {"term":"kanada","probability":0.045209691062612106,"topicId":11},
>     {"term":"robotskapade","probability":0.04064929862689341,"topicId":11},
>     {"term":"lake","probability":0.03671487993757405,"topicId":11},
>     {"term":"birds","probability":0.02369681037340463,"topicId":12},
>     {"term":"iucn","probability":0.020619560775043934,"topicId":12},
>     {"term":"world","probability":0.01805647612510781,"topicId":12},
>     {"term":"clements","probability":0.015933941106456802,"topicId":12},
>     {"term":"läst","probability":0.013941192536265194,"topicId":12},
>     {"term":"cheshmeh","probability":0.11064594620845476,"topicId":13},
>     {"term":"jabal","probability":0.05967252584628175,"topicId":13},
>     {"term":"hill","probability":0.022062432102082228,"topicId":13},
>     {"term":"runt","probability":0.02161165367091075,"topicId":13},
>     {"term":"källa","probability":0.01971175642758329,"topicId":13},
>     {"term":"creek","probability":0.12269798734719728,"topicId":14},
>     {"term":"vattendrag","probability":0.11548170564697531,"topicId":14},
>     {"term":"math","probability":0.1080413631122436,"topicId":14},
>     {"term":"county","probability":0.05567056593344491,"topicId":14},
>     {"term":"geonames","probability":0.03005264844658276,"topicId":14},
>     {"term":"arten","probability":0.013686188564871101,"topicId":15},
>     {"term":"iucn","probability":0.010069959578121875,"topicId":15},
>     {"term":"förekommer","probability":0.009834209338061728,"topicId":15},
>     {"term":"arter","probability":0.009684304135010991,"topicId":15},
>     {"term":"släktet","probability":0.006769956157331183,"topicId":15},
>     {"term":"kung","probability":0.03260243910309328,"topicId":16},
>     {"term":"hamilton","probability":0.013986789615136346,"topicId":16},
>     {"term":"england","probability":0.0121153524586439,"topicId":16},
>     {"term":"drottning","probability":0.011625755878276552,"topicId":16},
>     {"term":"skottland","probability":0.009249631189674428,"topicId":16},
>     {"term":"bulu","probability":0.07027290860872792,"topicId":17},
>     {"term":"runt","probability":0.01886574550707347,"topicId":17},
>     {"term":"juan","probability":0.016179163575668384,"topicId":17},
>     {"term":"viejo","probability":0.015799019016393998,"topicId":17},
>     {"term":"meter","probability":0.014780885004736585,"topicId":17},
>     {"term":"sjön","probability":0.034502823564636326,"topicId":18},
>     {"term":"kvadratkilometer","probability":0.02864792614705575,"topicId":18},
>     {"term":"lago","probability":0.025037979653382458,"topicId":18},
>     {"term":"maja","probability":0.024953856343253075,"topicId":18},
>     {"term":"havet","probability":0.024907385096831488,"topicId":18},
>     {"term":"trakten","probability":0.04096386788951671,"topicId":19},
>     {"term":"lake","probability":0.0345224866846443,"topicId":19},
>     {"term":"runt","probability":0.02963580174185564,"topicId":19},
>     {"term":"månaden","probability":0.028115313448807144,"topicId":19},
>     {"term":"meter","probability":0.027858463237621884,"topicId":19},
>     {"term":"landskommun","probability":0.025398611367203856,"topicId":20},
>     {"term":"församling","probability":0.02289216003761088,"topicId":20},
>     {"term":"stmi","probability":0.01846838158518249,"topicId":20},
>     {"term":"januari","probability":0.0176316418497597,"topicId":20},
>     {"term":"kommunen","probability":0.013980852803650965,"topicId":20},
>     {"term":"staden","probability":0.023384488081661554,"topicId":21},
>     {"term":"talet","probability":0.014684568820705514,"topicId":21},
>     {"term":"spelet","probability":0.008041407554453981,"topicId":21},
>     {"term":"historia","probability":0.005899803204835226,"topicId":21},
>     {"term":"invånare","probability":0.005315004050335149,"topicId":21},
>     {"term":"geonames","probability":0.11557194603913014,"topicId":22},
>     {"term":"vattendrag","probability":0.10396838795255336,"topicId":22},
>     {"term":"mexiko","probability":0.07664488602777711,"topicId":22},
>     {"term":"periodiskt","probability":0.044735247147267086,"topicId":22},
>     {"term":"robotskapade","probability":0.032041160696272615,"topicId":22},
>     {"term":"university","probability":0.015555705383864719,"topicId":23},
>     {"term":"läst","probability":0.014550019239920886,"topicId":23},
>     {"term":"john","probability":0.008732132798153132,"topicId":23},
>     {"term":"york","probability":0.007788712779924475,"topicId":23},
>     {"term":"amerikansk","probability":0.006967143392245979,"topicId":23},
>     {"term":"länsväg","probability":0.12364353938874159,"topicId":24},
>     {"term":"gradina","probability":0.041319606928897926,"topicId":24},
>     {"term":"makedonien","probability":0.027513514818681497,"topicId":24},
>     {"term":"eupithecia","probability":0.02290853181360536,"topicId":24},
>     {"term":"font","probability":0.02141349760145409,"topicId":24},
>     {"term":"kommun","probability":0.10290042230605351,"topicId":25},
>     {"term":"socken","probability":0.05728034134114776,"topicId":25},
>     {"term":"lappland","probability":0.01811165620130087,"topicId":25},
>     {"term":"småland","probability":0.010530347777947284,"topicId":25},
>     {"term":"värmland","probability":0.007339475945269198,"topicId":25},
>     {"term":"solen","probability":0.0200670569291999,"topicId":26},
>     {"term":"bibcode","probability":0.016284072132314146,"topicId":26},
>     {"term":"gånger","probability":0.0135712033444977,"topicId":26},
>     {"term":"stjärna","probability":0.013479451859270629,"topicId":26},
>     {"term":"magnitud","probability":0.01336898859104554,"topicId":26},
>     {"term":"punta","probability":0.09684045533161303,"topicId":27},
>     {"term":"udde","probability":0.09188909300387012,"topicId":27},
>     {"term":"bild","probability":0.03486418022216705,"topicId":27},
>     {"term":"point","probability":0.023982054573107807,"topicId":27},
>     {"term":"road","probability":0.016686082802654822,"topicId":27},
>     {"term":"talet","probability":0.012694136012504064,"topicId":28},
>     {"term":"meter","probability":0.010889970029104822,"topicId":28},
>     {"term":"området","probability":0.00882690394783239,"topicId":28},
>     {"term":"mount","probability":0.008722786999135347,"topicId":28},
>     {"term":"stora","probability":0.006863758992484095,"topicId":28},
>     {"term":"buttu","probability":0.04844274548131795,"topicId":29},
>     {"term":"runt","probability":0.019001409457080374,"topicId":29},
>     {"term":"trakten","probability":0.01675423618341787,"topicId":29},
>     {"term":"qimmat","probability":0.014601997139814487,"topicId":29},
>     {"term":"meter","probability":0.014327415704946335,"topicId":29},
>     {"term":"stockholm","probability":0.015135856576968138,"topicId":30},
>     {"term":"född","probability":0.01214761100144125,"topicId":30},
>     {"term":"svenska","probability":0.011952592249192522,"topicId":30},
>     {"term":"svensk","probability":0.010682496369850301,"topicId":30},
>     {"term":"sveriges","probability":0.006188853427958379,"topicId":30},
>     {"term":"tang","probability":0.03048136070857475,"topicId":31},
>     {"term":"runt","probability":0.022898886518763237,"topicId":31},
>     {"term":"morne","probability":0.020878561387803588,"topicId":31},
>     {"term":"castle","probability":0.020243918240258764,"topicId":31},
>     {"term":"dahr","probability":0.018246897035616844,"topicId":31},
>     {"term":"filmen","probability":0.020781490320475385,"topicId":32},
>     {"term":"film","probability":0.007085303531532155,"topicId":32},
>     {"term":"musik","probability":0.0064215730197610225,"topicId":32},
>     {"term":"regi","probability":0.006262473771143063,"topicId":32},
>     {"term":"john","probability":0.006106569562524211,"topicId":32},
>     {"term":"bosnien","probability":0.03607418632541323,"topicId":33},
>     {"term":"hercegovina","probability":0.035660992757308906,"topicId":33},
>     {"term":"pasir","probability":0.025960037172071338,"topicId":33},
>     {"term":"runt","probability":0.024025464515943364,"topicId":33},
>     {"term":"sarajevo","probability":0.02315327816207366,"topicId":33},
>     {"term":"olika","probability":0.009401336618340807,"topicId":34},
>     {"term":"exempel","probability":0.0074417216581975035,"topicId":34},
>     {"term":"talet","probability":0.006309347400350304,"topicId":34},
>     {"term":"ofta","probability":0.00599903713081023,"topicId":34},
>     {"term":"många","probability":0.0059618095032171435,"topicId":34},
>     {"term":"puerto","probability":0.152053730570257,"topicId":35},
>     {"term":"group","probability":0.04342233664608969,"topicId":35},
>     {"term":"drae","probability":0.02784126696431022,"topicId":35},
>     {"term":"runt","probability":0.017052063582336168,"topicId":35},
>     {"term":"pass","probability":0.016573201370297795,"topicId":35},
>     {"term":"loma","probability":0.029949250854912395,"topicId":36},
>     {"term":"mexiko","probability":0.02401769656994233,"topicId":36},
>     {"term":"runt","probability":0.023552175762361592,"topicId":36},
>     {"term":"city","probability":0.023491880302950876,"topicId":36},
>     {"term":"mexico","probability":0.023446760999741703,"topicId":36},
>     {"term":"vann","probability":0.018247050041365145,"topicId":37},
>     {"term":"säsongen","probability":0.01699606742987092,"topicId":37},
>     {"term":"matcher","probability":0.010585508000319446,"topicId":37},
>     {"term":"spelade","probability":0.010541376807987448,"topicId":37},
>     {"term":"spelare","probability":0.010174683683362207,"topicId":37},
>     {"term":"stockholm","probability":0.023463309330417906,"topicId":38},
>     {"term":"museum","probability":0.020588688992590618,"topicId":38},
>     {"term":"konst","probability":0.01531361753508411,"topicId":38},
>     {"term":"malmö","probability":0.013601805832605934,"topicId":38},
>     {"term":"målare","probability":0.01339290590857053,"topicId":38},
>     {"term":"morro","probability":0.029847680557851572,"topicId":39},
>     {"term":"runt","probability":0.02478222302763145,"topicId":39},
>     {"term":"sang","probability":0.022600156012697558,"topicId":39},
>     {"term":"trakten","probability":0.02176298711203436,"topicId":39},
>     {"term":"monte","probability":0.020395277368909562,"topicId":39},
>     {"term":"runt","probability":0.04674932020852935,"topicId":40},
>     {"term":"trakten","probability":0.0357544398258386,"topicId":40},
>     {"term":"meter","probability":0.03514437047605392,"topicId":40},
>     {"term":"havet","probability":0.03080664018465887,"topicId":40},
>     {"term":"månaden","probability":0.030751659585869125,"topicId":40},
>     {"term":"talet","probability":0.020927168859939324,"topicId":41},
>     {"term":"kyrkan","probability":0.016978453949576977,"topicId":41},
>     {"term":"kyrka","probability":0.016398363668156845,"topicId":41},
>     {"term":"gallery","probability":0.009234697415750651,"topicId":41},
>     {"term":"gamla","probability":0.006700990866659774,"topicId":41},
>     {"term":"partiet","probability":0.009200084245357574,"topicId":42},
>     {"term":"regeringen","probability":0.006781072683076032,"topicId":42},
>     {"term":"valet","probability":0.006751636661360212,"topicId":42},
>     {"term":"file","probability":0.006591367471565553,"topicId":42},
>     {"term":"politiska","probability":0.006211111760647785,"topicId":42},
>     {"term":"företaget","probability":0.017632507535482355,"topicId":43},
>     {"term":"airport","probability":0.016531267519608497,"topicId":43},
>     {"term":"piano","probability":0.011765155277036494,"topicId":43},
>     {"term":"international","probability":0.009193469609380736,"topicId":43},
>     {"term":"orkester","probability":0.008789598592035592,"topicId":43},
>     {"term":"svensk","probability":0.027812732588726418,"topicId":44},
>     {"term":"född","probability":0.020404036898973906,"topicId":44},
>     {"term":"amerikansk","probability":0.016893938539467617,"topicId":44},
>     {"term":"skådespelare","probability":0.015654966665244758,"topicId":44},
>     {"term":"politiker","probability":0.010136789647076259,"topicId":44},
>     {"term":"född","probability":0.012110483884254163,"topicId":45},
>     {"term":"senare","probability":0.0064196545583964214,"topicId":45},
>     {"term":"biografi","probability":0.0046644671122361885,"topicId":45},
>     {"term":"tillsammans","probability":0.004522535383402723,"topicId":45},
>     {"term":"barn","probability":0.003858167880370399,"topicId":45},
>     {"term":"ryssland","probability":0.08655099647021003,"topicId":46},
>     {"term":"oblast","probability":0.039588701471860624,"topicId":46},
>     {"term":"tubu","probability":0.027362529200185913,"topicId":46},
>     {"term":"buntu","probability":0.023957239335077818,"topicId":46},
>     {"term":"ozero","probability":0.022674789610265166,"topicId":46},
>     {"term":"bosnien","probability":0.06494948578534294,"topicId":47},
>     {"term":"hercegovina","probability":0.05479658672484445,"topicId":47},
>     {"term":"bästa","probability":0.03202701607086499,"topicId":47},
>     {"term":"kroatien","probability":0.03185579878936539,"topicId":47},
>     {"term":"serbien","probability":0.020373304237372863,"topicId":47},
>     {"term":"from","probability":0.08411532601710255,"topicId":48},
>     {"term":"shift","probability":0.06861519273333161,"topicId":48},
>     {"term":"small","probability":0.0645509153944095,"topicId":48},
>     {"term":"value","probability":0.046919681737576234,"topicId":48},
>     {"term":"linegrey","probability":0.03393173559526991,"topicId":48},
>     {"term":"cerro","probability":0.2675005896664047,"topicId":49},
>     {"term":"berg","probability":0.09592849535029228,"topicId":49},
>     {"term":"kulle","probability":0.04349781576096616,"topicId":49},
>     {"term":"chile","probability":0.0196392508406257,"topicId":49},
>     {"term":"grande","probability":0.01294402564510321,"topicId":49}

  

Plot visualization

In [ ]:
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

circle {
  fill: rgb(31, 119, 180);
  fill-opacity: 0.5;
  stroke: rgb(31, 119, 180);
  stroke-width: 1px;
}

.leaf circle {
  fill: #ff7f0e;
  fill-opacity: 1;
}

text {
  font: 14px sans-serif;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<script>

var json = {
 "name": "data",
 "children": [
  {
     "name": "topics",
     "children": [
      ${rawJson}
     ]
    }
   ]
};

var r = 1000,
    format = d3.format(",d"),
    fill = d3.scale.category20c();

var bubble = d3.layout.pack()
    .sort(null)
    .size([r, r])
    .padding(1.5);

var vis = d3.select("body").append("svg")
    .attr("width", r)
    .attr("height", r)
    .attr("class", "bubble");

  
var node = vis.selectAll("g.node")
    .data(bubble.nodes(classes(json))
    .filter(function(d) { return !d.children; }))
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; })
    color = d3.scale.category20();
  
  node.append("title")
      .text(function(d) { return d.className + ": " + format(d.value); });

  node.append("circle")
      .attr("r", function(d) { return d.r; })
      .style("fill", function(d) {return color(d.topicName);});

var text = node.append("text")
    .attr("text-anchor", "middle")
    .attr("dy", ".3em")
    .text(function(d) { return d.className.substring(0, d.r / 3)});
  
  text.append("tspan")
      .attr("dy", "1.2em")
      .attr("x", 0)
      .text(function(d) {return Math.ceil(d.value * 10000) /10000; });

// Returns a flattened hierarchy containing all leaf nodes under the root.
function classes(root) {
  var classes = [];

  function recurse(term, node) {
    if (node.children) node.children.forEach(function(child) { recurse(node.term, child); });
    else classes.push({topicName: node.topicId, className: node.term, value: node.probability});
  }

  recurse(null, root);
  return {children: classes};
}
</script>
""")

  

  

Classify some articles
======================

In [ ]:
// Load the model
import org.apache.spark.mllib.clustering.LocalLDAModel
val model = LocalLDAModel.load(sc, "dbfs:/datasets/05_lda/lda_sv_20201208.model")



  

>     import org.apache.spark.mllib.clustering.LocalLDAModel
>     model: org.apache.spark.mllib.clustering.LocalLDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@2f6ba799

In [ ]:
// Load the countvector
val lda_countVector_mllib_loaded = sc.textFile("lda_countvector_sv_20201208") // This doesn't work, need to figure out how to parse the loaded rdd
print(lda_countVector_mllib_loaded)

  

>     lda_countvector_sv_20201208 MapPartitionsRDD[57] at textFile at command-165306931359183:2lda_countVector_mllib_loaded: org.apache.spark.rdd.RDD[String] = lda_countvector_sv_20201208 MapPartitionsRDD[57] at textFile at command-165306931359183:2

In [ ]:
// sample 5 articles
val numsample = 5
val sample = lda_countVector_mllib.takeSample(false, numsample)

  

>     numsample: Int = 5
>     sample: Array[(Long, org.apache.spark.mllib.linalg.Vector)] = Array((12136639304,(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,32,33,38,40,41,45,49,55,57,61,63,64,68,71,72,80,87,96,99,100,114,115,118,187,192,238,407,627,1797],[3.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (9662534067,(10000,[0,1,2,3,4,5,6,7,9,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,33,35,36,37,46,47,53,54,60,74,75,80,102,109,138,158,160,167,170,171,212,214,224,237,277,651,785,1148,2177],[2.0,3.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0])), (13118077644,(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,43,49,55,63,65,74,75,83,109,111,115,116,119,210,220,724,795,2819,2990],[3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (16303874732,(10000,[0,1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,29,30,32,34,36,37,38,40,41,45,55,57,59,62,65,66,73,87,145,150,154,155,174,181,193,207,577,664,8329],[3.0,3.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), (16623377689,(10000,[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,32,34,36,38,40,41,42,43,47,52,54,55,57,62,65,74,75,89,90,93,109,145,150,155,193,321,4352],[2.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])))

In [ ]:
// Look at the sample
//corpus_df.show(1, false)
lda_countVector_mllib.show(1, false)

In [ ]:
val sampleRdd = sc.makeRDD(sample)

  

>     sampleRdd: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = ParallelCollectionRDD[59] at makeRDD at command-165306931359186:1

  

Create topic distribution

In [ ]:
val topicDistributions = model.topicDistributions(sampleRdd)
topicDistributions.collect()

  

>     topicDistributions: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[62] at map at LDAModel.scala:373
>     res15: Array[(Long, org.apache.spark.mllib.linalg.Vector)] = Array((12136639304,[2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,0.015128546820869355,2.898550724637683E-4,2.898550724637683E-4,0.2535420383867245,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,0.7177062263866097,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4,2.898550724637683E-4]), (9662534067,[3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,0.7368149101533721,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,0.08909532655882103,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,0.09294923347030291,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,3.076923076923072E-4,0.06698668366365629]), (13118077644,[3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,0.1790640807505678,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,0.8051982143313992,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4,3.2786885245901623E-4]), (16303874732,[3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,0.968322787741821,0.01643911702008331,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4,3.174603174603174E-4]), (16623377689,[2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,0.9855882352941182,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4,2.9411764705882383E-4]))

In [ ]:
println("first topic distribution:"+topicDistributions.first._2.toArray.mkString(", "))

  

>     first topic distribution:2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 0.015128546820869355, 2.898550724637683E-4, 2.898550724637683E-4, 0.2535420383867245, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 0.7177062263866097, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4, 2.898550724637683E-4